Before performing the analysis, the preprocessing is done by Deciphering_CIMP_Preprocessing. Here we download directly the preprocessed data.

**IMPORTANT: As in the preprocessing file, placeholder are put in lieu of the directories - these should point to those used during the preprocessing**

# Package dependencies

In [ ]:
# general 
from __future__ import division
import numpy as np
import pandas as pd
import xenaPython as xena
import re
import os
from collections import Counter
import csv
from tqdm.notebook import trange, tqdm

# clustering
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.preprocessing import StandardScaler
import networkx as nx
import numpy.linalg as LA

# for clustering comparison
from sklearn.cluster import AgglomerativeClustering, KMeans, AffinityPropagation
from sklearn.mixture import GaussianMixture

# plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

# statistical tests
from scipy.stats import mannwhitneyu, kruskal, chisquare, ks_2samp, fisher_exact, ttest_ind, pointbiserialr, chi2_contingency, pearsonr, spearmanr

# statistical annotations
from statannot import add_stat_annotation

# dim reduction
import umap

# survival analysis
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from statsmodels.stats.multitest import multipletests
from lifelines import CoxPHFitter

# Random Forest analysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, accuracy_score

# Download preprocessed data

In [ ]:
info_dir = "path\to\filtered_illumina_TCGA"
clin_raw_dir = "path\to\clinical_raw"
meth_dir = "path\to\preprocessed_data"

In [ ]:
cancer_to_keep = os.listdir(meth_dir)

In [ ]:
cancer_to_keep = np.unique([cancer_to_keep[i].split('_')[0] for i in range(len(cancer_to_keep))]) 

In [ ]:
meth_data_pc = {}
for cancer in tqdm(cancer_to_keep):
    meth_data_pc[cancer] = pd.read_csv(os.path.join(meth_dir,cancer+"_meth_filtered.csv"))
    meth_data_pc[cancer] = meth_data_pc[cancer].set_index("Unnamed: 0")   

In [ ]:
cancer_clin_raw_pc = {}
for cancer in tqdm(cancer_to_keep):
    cancer_clin_raw_pc[cancer]= pd.read_csv(os.path.join(clin_raw_dir,"TCGA-"+cancer+".GDC_phenotype.tsv.gz"),sep="\t").set_index("submitter_id.samples")
    cancer_clin_raw_pc[cancer].index = [cancer_clin_raw_pc[cancer].index[i][:15] for i in range(len(cancer_clin_raw_pc[cancer].index))]

In [ ]:
pcpg_diagnosis = cancer_clin_raw_pc["PCPG"]["primary_diagnosis.diagnoses"].str.split(",").apply(lambda a: a[0]).str.split(" ").apply(lambda a: a[-1]).str.lower()
pheo_patients = pcpg_diagnosis[pcpg_diagnosis=="pheochromocytoma"].index
para_patients = pcpg_diagnosis[pcpg_diagnosis=="paraganglioma"].index
meth_data_pc["PCPG"] = meth_data_pc["PCPG"].loc[meth_data_pc["PCPG"].index.intersection(pheo_patients)]

In [ ]:
clin_data_pc = {}
for cancer in tqdm(cancer_to_keep):
    clin_data_pc[cancer] = pd.read_csv(os.path.join(info_dir,cancer+"_clin_mat.csv"))
    clin_data_pc[cancer] = clin_data_pc[cancer].set_index("Unnamed: 0")   

In [ ]:
for cancer in tqdm(cancer_to_keep):
    meth_data_pc[cancer] = meth_data_pc[cancer][~meth_data_pc[cancer].index.duplicated(keep='first')]
    clin_data_pc[cancer] = clin_data_pc[cancer][~clin_data_pc[cancer].index.duplicated(keep='first')]
    cancer_clin_raw_pc[cancer] = cancer_clin_raw_pc[cancer][~cancer_clin_raw_pc[cancer].index.duplicated(keep='first')]

In [ ]:
mut_dir = "path\to\mutations_MC3"
mut_pancan = {}
for cancer in tqdm(cancer_to_keep):
    if cancer in ["COAD","READ"]:
        mut_pancan[cancer] = pd.read_csv(os.path.join(info_dir,"COADREAD_mutations.csv"))
        mut_pancan[cancer] = mut_pancan[cancer].set_index("Unnamed: 0")  
        mut_pancan[cancer] = mut_pancan[cancer].loc[meth_data_pc[cancer].index.intersection(mut_pancan[cancer].index)]
    elif cancer=="PCPG":
        mut_pancan[cancer] = pd.read_csv(os.path.join(info_dir,"PCPG_mutations.csv"))
        mut_pancan[cancer] = mut_pancan[cancer].set_index("Unnamed: 0")  
        mut_pancan[cancer] = mut_pancan[cancer].loc[meth_data_pc[cancer].index.intersection(mut_pancan[cancer].index)]
    else:
        mut_pancan[cancer] = pd.read_csv(os.path.join(mut_dir,cancer+"_mutations.csv")).set_index("Unnamed: 0")
mut_pancan["LAML"] = mut_pancan["LAML"].rename(columns={"KDM7A":"JHDM1D"})

In [ ]:
msi_profile = pd.read_csv("MSI_status_nature_Cortes_Ciriano.csv",sep=";").set_index("Barcode")

### Preliminary: genes to use

The genes associated with DNA & histone methylation/demethylation are taken from the following sources: 
1. Hyun K, Jeon J, Park K, Kim J. Writing, erasing and reading histone lysine methylations. Exp Mol Med 2017; 49: e324
2. Moore LD, Le T, Fan G. DNA methylation and its basic function. Neuropsychopharmacology 2013; 38: 23–38
3. de Necochea-Campion R, Ghochikyan A, Josephs SF, Zacharias S, Woods E, Karimi-Busheri F et al. Expression of the epigenetic factor BORIS (CTCFL) in the human genome. J Transl Med 2011; 9: 213
4. Santoro R, Li J, Grummt I. The nucleolar remodeling complex NoRC mediates heterochromatin formation and silencing of ribosomal gene transcription. Nat Genet 2002; 32: 393–396
5. Lee GE, Kim JH, Taylor M, Muller MT. DNA methyltransferase 1-associated protein (DMAP1) is a co-repressor that stimulates DNA methylation globally and locally at sites of double strand break repair. J Biol Chem 2010; 285: 37630–37640
6. Bledea R, Vasudevaraja V, Patel S, Stafford J, Serrano J, Esposito G et al. Functional and topographic effects on DNA methylation in IDH1/2 mutant cancers. Sci Rep 2019; 9: 16830

In [ ]:
genes_writer_Hmeth = {"H3K4": ["SETD1A","SETD1B","KMT2A","KMT2D","KMT2C","KMT2B","SMYD1","SMYD2","SETD7","SETD9","PRDM9"],
                      "H3K9": ["SUV39H1","SUV39H2","EHMT2","EHMT1","SETDB1","PRDM1","PRDM2","MECOM","PRDM4","PRDM5","PRDM6","PRDM7","PRDM8","PRDM9","PRDM10","PRDM11","PRDM12","PRDM13","PRDM14","PRDM15","PRDM16"],
                      "H3K27": ["EZH1","EZH2"],
                      "H3K36": ["SETD2","NSD1","WHSC1","WHSC1L1","SMYD2","ASH1L","SETD3","SETMAR"],
                      "H3K79": ["DOT1L"],
                      "H4K20": ["SETD8","SUV420H1","SUV420H2"]}
all_genes_writer_Hmeth = list(genes_writer_Hmeth.values())
all_genes_writer_Hmeth = np.unique([item for sublist in all_genes_writer_Hmeth for item in sublist])

genes_eraser_Hmeth = {"H3K4": ["KDM1A","KDM1B","KDM5A","KDM5B","KDM5C","KDM5D"],
                      "H3K9": ["KDM3B","KDM3A","KDM4A","KDM4B","KDM4C","KDM4D","KDM4E","PHF8","JMJD1C"],
                      "H3K27": ["KDM6A","UTY","KDM6B","JHDM1D","PHF8"],
                      "H3K36": ["KDM2A","KDM2B","KDM4A","KDM4B","KDM4C","KDM4D"],
                      "H3K79": [],
                      "H4K20": ["PHF8","PHF2"]}
all_genes_eraser_Hmeth = list(genes_eraser_Hmeth.values())
all_genes_eraser_Hmeth = np.unique([item for sublist in all_genes_eraser_Hmeth for item in sublist])

In [ ]:
genes_meth = ["BAZ2A","CTCFL","DMAP1","DNMT1","DNMT3A","DNMT3B","GADD45A","GADD45B",
                       "GADD45G","GADD45GIP1","UHRF1","UHRF1BP1","UHRF1BP1L"]
genes_demeth = ["TET1","TET2","TET3","TDG","TDGF1","TDGF1P3","MBD1","MBD2","MBD4","APOBEC1","CTCF","IDH1","IDH2"]

co_expressed_genes = {"TET1": ["DNMT1", "DNMT3A","DNMT3B","EZH2","MECP2","NANOG","OGDH","PRMT5","SIN3A","TDG"],
                     "TET2": ["ASXL1","CXXC4","DNMT1","DNMT3A","EZH2","IDH1","IDH2","KMT2A","TDG","WT1"],
                     "TET3": ["CA2","DNMT1","DNMT3A","DNMT3B","DPPA3","NANOG","OGDH","EZH2","PRMT5","TDG"],
                     "TDG": ["AICDA","APEX1","CA2","GADD45A","SMUG1","SUMO1","SUMO2","SUMO3","TET1","UBE2I"],
                     "TDGF1": ["ACVR1B","ACVR1C","ACVR2B","GPC1","LEFTY1","LEFTY2","NANOG","NODAL","POU5F1","SMAD2"],
                     "MBD4": ["APEX1","DNMT1","DNMT3B","MLH1","SMUG1","TDG","TRDMT1","UHRF1","UNG","ZBTB33"],
                     "APOBEC1": ["A1CF","APOB","APOBEC2","APOBEC3A","APOBEC3B","APOBEC3C","APOBEC3H","APOBEC4","CDA","HNRNPAB"],
                     "CTCF": ["EP300","EZH2","H2AFZ","HIST2H2AC","MYC","POLR2A","POLR2F","RXRA","SUZ12","WDR5"],
                     "IDH1": ["ACO1","ACO2","CAT","HSD17B4","IDH2","IDH3A","IDH3B","IDH3G","OGDH","PGM1"],
                     "IDH2": ["ACO1","ACO2","GLUD1","IDH1","IDH3A","IDH3B","IDH3G","OGDH","SIRT3","SOD2"],
                     "BAZ2A": ["ARID4B","DNMT1","DNMT3B","H2AFV","H2AFX","H2AFZ","HDAC1","HIST2H2AC","SMARCA5","TTF1"],
                     "CTCFL": ["CBLN4","CHD8","COX7A1","IGF2","PHACTR3","PRMT7","RAD21","RAD21L1","RBM38","SMC3"],
                     "DMAP1": ["ACTL6A","CDC5L","KAT5","MEAF6","MORF4L1","MRGBP","RUVBL1","RUVBL2","VPS72","YEATS4"],
                     "DNMT1": ["DNMT3A","DNMT3B","EZH2","HDAC1","HDAC2","HIST2H3PS2","MBD2","RB1","SMARCA5","UHRF1"],
                     "DNMT3A": ["CBX4","DNMT1","DNMT3B","DNMT3L","EED","EZH2","HIST2H2AC","MYC","PRMT5","SUZ12"],
                     "DNMT3B": ["DNMT1","DNMT3A","EZH2","HDAC1","HDAC2","HIST1H2BA","HIST1H3A","HIST2H3PS2","SMARCA5","SUZ12"],
                     "GADD45A": ["AURKA","BRCA1","CCNB1","CCND1","CDK1","MAP2K6","MAP3K4","MAPK8","PCNA","TP53"],
                     "GADD45B": ["CCNB1","CCNB2","CDK1","GADD45A","GADD45G","MAP2K3","MAP2K6","MAP3K4","PCNA","TP53"],
                     "GADD45G": ["CCNB1","CCNB2","CDK1","GADD45A","GADD45B","MAP2K3","MAP2K6","MAP3K4","PCNA","TP53"],
                     "GADD45GIP1": ["MRPL11","MRPL28","MRPL4","MRPL40","MRPL41","MRPL42","MRPL47","MRPL50","MRPS30","MRPS34"],
                     "UHRF1": ["DNMT1","H2AFX","H2AFZ","HIST1H2BA","HIST1H2BB","HIST1H2BJ","HIST2H2AC","HIST2H2BE","HIST2H3PS2","MCM5"],
                     "UHRF1BP1": ["ANKS1A","C6orf106","FANCE","MORC1","PHRF1","SNRPC","TNIP1","WDFY4","ZNF76"],
                     "UHRF1BP1L": ["DEPDC4","FAM83A","GPATCH2","GPATCH2L","MAN2A1","PTPRH","STX6","TMEM160","VPS13C","ZSCAN23"]}

# get the list of all coexpressed genes
list_co_expressed_genes = list(co_expressed_genes.values())
list_co_expressed_genes = [item for sublist in list_co_expressed_genes for item in sublist]

# get the list of unique genes to keep for analysis
genes_to_keep = np.unique(genes_meth+genes_demeth+list_co_expressed_genes)
# genes_to_keep = gene_data_pc["ACC"].columns.intersection(genes_to_keep)

# genes that are in the GEX matrix
genes_demeth_gex = np.delete(genes_demeth,np.where(np.array(genes_demeth)=="TDGF1P3"))
genes_meth_gex = np.array(genes_meth)

### Delete normal samples

TCGA provides some samples from healthy tissue near the tumour ; we delete them from the original methylation and keep them to compare the distribution of average beta values. For ACC & MESO we have no normal samples from TCGA so we download those from GEO database (preprocessed).

In [ ]:
normal_sample_codes = ["10","11","12","13","14","20"]

In [ ]:
geo_normal_dir = "path\to\GEO_normal_tissue"

In [ ]:
geo_normal_meth = {"ACC": pd.read_csv(os.path.join(geo_normal_dir,"Adrenal_normal_full.csv")).set_index("Unnamed: 0"),
                  "LAML": pd.read_csv(os.path.join(geo_normal_dir,"Blood_normal_full.csv")).set_index("Unnamed: 0"),}

In [ ]:
normal_meth,new_meth_data_pc,new_gene_data_pc,new_clin_data_pc = {},{},{},{}
for cancer in tqdm(cancer_to_keep):
    ind_normal_samples = np.where(np.array([meth_data_pc[cancer].index[i].split("-")[-1] for i in range(meth_data_pc[cancer].shape[0])])=="11")[0]
    if cancer=="LGG":
        print("There are {} normal samples for cancer type {}".format(normal_meth["GBM"].shape[0],cancer))
        normal_meth[cancer] = normal_meth["GBM"][normal_meth["GBM"].columns.intersection(meth_data_pc[cancer].columns)]
    elif cancer=="MESO":
        print("There are {} normal samples for cancer type {}".format(normal_meth["LUAD"].shape[0],cancer))
        normal_meth[cancer] = normal_meth["LUAD"][normal_meth["LUAD"].columns.intersection(meth_data_pc[cancer].columns)]
    elif cancer in ["ACC","LAML"]:
        print("There are {} normal samples for cancer type {}".format(geo_normal_meth[cancer].shape[0],cancer))
        normal_meth[cancer] = geo_normal_meth[cancer]
    elif not(len(ind_normal_samples)==0):
        print("There are {} normal samples for cancer type {}".format(len(ind_normal_samples),cancer))
        normal_meth[cancer] = meth_data_pc[cancer].iloc[ind_normal_samples]
    else:
        print("No normal samples for cancer type {}".format(cancer))
        normal_meth[cancer] =  []
    new_meth_data_pc[cancer] = meth_data_pc[cancer].copy().drop(meth_data_pc[cancer].index[ind_normal_samples])
    new_clin_data_pc[cancer] = clin_data_pc[cancer].copy().drop(clin_data_pc[cancer].index.intersection(meth_data_pc[cancer].index[ind_normal_samples]))

In [ ]:
avg_samp,avg_cpg=[],[]
for cancer in new_meth_data_pc:
    print(cancer,"meth info: ",new_meth_data_pc[cancer].shape)
    avg_samp.append(new_meth_data_pc[cancer].shape[0])
    avg_cpg.append(new_meth_data_pc[cancer].shape[1])
print('average sample size : ',np.mean(avg_samp),np.quantile(avg_samp,0.25),np.quantile(avg_samp,0.75),
      'average cpg ids : ',np.mean(avg_cpg),np.quantile(avg_cpg,0.25),np.quantile(avg_cpg,0.75))


# Clustering

First we standardize the data to be able to perform dimensionality reduction

In [ ]:
std_sc_data = {}
for cancer in tqdm(cancer_to_keep):
    std_sc = StandardScaler()
    std_sc_data[cancer] = std_sc.fit_transform(new_meth_data_pc[cancer])


In [ ]:
df_transf_data = {}
for cancer in tqdm(cancer_to_keep):
    df_transf_data[cancer] = pd.DataFrame(std_sc_data[cancer], index=new_meth_data_pc[cancer].index, columns=new_meth_data_pc[cancer].columns)

### Dimensionality reduction

We use Uniform Manifold Approximation and Projection (UMAP) 
Ref : McInnes L, Healy J, Melville J. UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction. arXiv [stat.ML]. 2018.http://arxiv.org/abs/1802.03426.

In [ ]:
umap_red_data= {}
for cancer in tqdm(cancer_to_keep):
    reducer = umap.UMAP(n_components=2)
    umap_red_data[cancer] = reducer.fit_transform(df_transf_data[cancer])

### Check optimal number of clusters

To choose the optimal number of cluster, we compute the 10-nearest neighbor affinity matrix 

In [ ]:
def get_consensus_matrix_clustering(meth_data_pc,cancer_to_keep,triple_cluster_cancers,n_runs,kernel):
#     Parameters: meth_data_pc (dict): keys are cancer types, values are pd.DataFrame (n_pat,n_cpg) (rows=patients,columns=cpg)
#             cancer_to_keep (list): names of cancer types
#             triple_cluster_cancers (list): names of cancer types with 3 methylation groups
#             n_runs (int): number of runs for consensus clustering (avoid randomness due to init)
#             kernel (str): either "NN" or "aff", to compute from a preexisting affinity matrix or not
#     Returns: consensus_matrix (dict): keys are cancer types, values are pd.DataFrame (n_pat,n_runs) (rows=patients,columns=runs)
#             affinity_matrix (dict): keys are cancer types, values are pd.DataFrame (n_pat,n_pat) 
#                            corresponding to affinity between pairs of patients (rows=patients,columns=patients)
    if kernel=="NN":
        affinity_matrix = {}
    else:
        affinity_matrix = meth_data_pc
    consensus_matrix = {cancer: [] for cancer in cancer_to_keep}
    for r in tqdm(range(n_runs)):
        agglomerative_clustering_pc = {}
        for cancer in cancer_to_keep:
            print(cancer)
            if cancer in triple_cluster_cancers:
                if kernel=="NN":
                    agglomerative = SpectralClustering(n_clusters=3,affinity="nearest_neighbors",n_neighbors=10)
                else:   
                    agglomerative = SpectralClustering(n_clusters=3,affinity="precomputed")
            else:
                if kernel=="NN":
                      agglomerative = SpectralClustering(n_clusters=2,affinity="nearest_neighbors",n_neighbors=10)
                else:
                    agglomerative = SpectralClustering(n_clusters=2,affinity="precomputed")
            if kernel=="NN":       
                agglomerative.fit(meth_data_pc[cancer].values)
                affinity_matrix[cancer] = agglomerative.affinity_matrix_ 
                affinity_matrix[cancer] = affinity_matrix[cancer].todense()
            else:
                agglomerative.fit(meth_data_pc[cancer])
                
            agglomerative_clustering_pc[cancer] = pd.DataFrame(agglomerative.labels_,columns=["cluster"])
            agglomerative_clustering_pc[cancer] = agglomerative_clustering_pc[cancer]+1

            if len(consensus_matrix[cancer])==0:
                consensus_matrix[cancer] = agglomerative_clustering_pc[cancer]
            else:
                consensus_matrix[cancer] = pd.concat([consensus_matrix[cancer],agglomerative_clustering_pc[cancer]],axis=1)
    return consensus_matrix, affinity_matrix

In [ ]:
consensus_matrix_spectral, affinity_matrix = get_consensus_matrix_clustering(new_meth_data_pc,cancer_to_keep,[],1,"NN")

In [ ]:
# compute the eigenvalues for the signless Laplacian

plt.rc('font', size=15)
for cancer in tqdm(cancer_to_keep):
    graph = graph_of_patients(list(new_meth_data_pc[cancer].index),affinity_matrix[cancer])
    L = nx.linalg.laplacianmatrix.normalized_laplacian_matrix(graph)
    A = nx.linalg.graphmatrix.adjacency_matrix(graph)
    signL = L.todense() + A.todense() + A.todense()
    eigenvalues, eigenvectors = LA.eig(signL)
    eigengaps = [-eigenvalues[1]+eigenvalues[0],-eigenvalues[2]+eigenvalues[1],
                                                         -eigenvalues[3]+eigenvalues[2],-eigenvalues[4]+eigenvalues[3]]
    print("{} Gap1: {}, Gap2: {}, Gap3: {}, Gap4: {}".format(cancer,eigengaps[0],eigengaps[1],
                                                            eigengaps[2],eigengaps[3]))
    print("Optimal number of clusters {}".format(np.argmax(eigengaps[1:])+2))
    plt.figure()
    plt.title('Eigenvalues of L of '+cancer)
    plt.scatter(np.arange(30), eigenvalues[:30])
    plt.ylim()
    plt.grid()

In [ ]:
# list of cancers for which optimal value is 3
triple_cluster_cancers_aff = ["ACC","BLCA","BRCA","COAD","ESCA","GBM","HNSC","KIRC","LIHC","LUAD","LUSC","MESO","PAAD","PCPG","READ","SKCM","UCEC"]

In [ ]:
consensus_matrix_spectral, affinity_matrix = get_consensus_matrix_clustering(affinity_matrix,cancer_to_keep,triple_cluster_cancers_aff,10,"aff")

In [ ]:
# perform consensus clustering 
cluster_memb_dir = "path\to\cluster_memb"
from netneurotools import cluster
consensus_clustering_spectral = {}
for cancer in tqdm(cancer_to_keep):
#     consensus_clustering_spectral[cancer] = pd.DataFrame(cluster.find_consensus(consensus_matrix_spectral[cancer].values, seed=1234),columns=["cluster"],index=new_meth_data_pc[cancer].index)
#     consensus_clustering_spectral[cancer] = consensus_clustering_spectral[cancer].reset_index()
#     pd.DataFrame(consensus_clustering_spectral[cancer]).to_csv(os.path.join(cluster_memb_dir,cancer+"_cluster_memb_new.csv"))
# #####
    consensus_clustering_spectral[cancer] = pd.read_csv(os.path.join(cluster_memb_dir,cancer+"_cluster_memb_new.csv")).set_index("Unnamed: 0")
    consensus_clustering_spectral[cancer].columns=["id","cluster"]

In [ ]:
for cancer in tqdm(cancer_to_keep):
    consensus_clustering_spectral[cancer] = consensus_clustering_spectral[cancer].set_index("id")
consensus_clustering_spectral["CESC"] = consensus_clustering_spectral["CESC"].replace({4: 3})

In [ ]:
sign_cpg_dir = "path\to\sign_cpg_spectral"
def get_sign_kw_louvain(meth_data_pc, cancer_to_keep, consensus_clustering):
#     Parameters: meth_data_pc (dict): keys are cancer types, values are pd.DataFrame (n_pat,n_cpg) (rows=patients,columns=cpg)
#             cancer_to_keep (list): names of cancer types
#             consensus_clustering (dict): keys are cancer types, values are pd.DataFrame (n_pat,1) (rows=patients, column=cluster membership)
#     Returns: kw (dict): keys are cancer types, values are list of sign. cpg probes for 
    kw = {}
    for cancer in tqdm(cancer_to_keep):
        kw[cancer]=[]
        positions = meth_data_pc[cancer].columns
        ind_cl_1 = np.where(consensus_clustering[cancer]["cluster"].values==1)[0]
        ind_cl_2 = np.where(consensus_clustering[cancer]["cluster"].values==2)[0]
        ind_cl_3 = np.where(consensus_clustering[cancer]["cluster"].values==3)[0]
        print(cancer)
        if len(np.unique(consensus_clustering[cancer]["cluster"].values))==2:
            meth_cl_1 = meth_data_pc[cancer].iloc[ind_cl_1].copy()
            meth_cl_2 = meth_data_pc[cancer].iloc[ind_cl_2].copy()
            for pos in tqdm(positions):
                kw[cancer].append(kruskal(meth_cl_1[pos],
                        meth_cl_2[pos])[1])
        elif len(np.unique(consensus_clustering[cancer]["cluster"].values))==3:
            meth_cl_1 = meth_data_pc[cancer].iloc[ind_cl_1].copy()
            meth_cl_2 = meth_data_pc[cancer].iloc[ind_cl_2].copy()
            meth_cl_3 = meth_data_pc[cancer].iloc[ind_cl_3].copy()
            for pos in tqdm(positions):
                kw[cancer].append(kruskal(meth_cl_1[pos],
                        meth_cl_2[pos],
                        meth_cl_3[pos])[1])
        else:
            print("The cancer {} is not taken into consideration (too many clusters)".format(cancer))
            continue
        ind_sign = np.where(np.array(kw[cancer])<(0.05/len(kw[cancer])))[0]
        print("There are {} significantly different positions between clusters for cancer type {}".format(len(ind_sign),cancer))
        sign_pos=[meth_data_pc[cancer].columns[i] for i in ind_sign]
        pd.DataFrame(sign_pos).to_csv(os.path.join(sign_cpg_dir,cancer+"_sign_cpg_spectral.csv"))
    return kw

In [ ]:
kw_snf = get_sign_kw_louvain(new_meth_data_pc, ["LGG"], consensus_clustering_spectral)

In [ ]:
sign_cpg_dir = "path\to\sign_cpg_spectral"
sign_pos_spectral = {}
for cancer in cancer_to_keep:
    sign_pos_spectral[cancer] = pd.read_csv(os.path.join(sign_cpg_dir,cancer+"_sign_cpg_spectral.csv")).set_index("Unnamed: 0")

In [ ]:
avg_size_sign_pos = []
for cancer in sign_pos_spectral:
    print(cancer,sign_pos_spectral[cancer].shape[0])
    avg_size_sign_pos.append(sign_pos_spectral[cancer].shape[0])
print("avg : ",np.mean(avg_size_sign_pos),"25% : ",np.quantile(avg_size_sign_pos,0.25),"75% : ",np.quantile(avg_size_sign_pos,0.75))

In [ ]:
def get_avg_methylation_snf(sign_pos, meth_data_pc, consensus_clustering, cancer_order):
#     Parameters: meth_data_pc (dict): keys are cancer types, values are pd.DataFrame (n_pat,n_cpg) (rows=patients,columns=cpg)
#             sign_pos (dict): keys are cancer types, values are pd.DataFrame (n_sign_pos,1) (rows=sign_pos,)
#             consensus_clustering (dict): keys are cancer types, values are pd.DataFrame (n_pat,1) (rows=patients, column=cluster membership)
#             cancer_order (list): list of cancer types in the KS order
#     Returns: avg_meth (pd.Df): the average beta values over significant positions (n_cancers,n_clusters) (rows=cancer,column=cluster number)
    avg_meth = {}
    for cancer in cancer_order:
        avg_meth[cancer] = []
        for i in np.unique(consensus_clustering[cancer]["cluster"].values):
            avg_meth[cancer].append(np.round(np.mean(meth_data_pc[cancer].iloc[np.where(consensus_clustering[cancer]["cluster"].values==i)[0]][sign_pos[cancer].values.ravel()].mean()),2))
    return pd.DataFrame.from_dict(avg_meth, orient='index')

In [ ]:
avg_meth_snf_spectral = get_avg_methylation_snf(sign_pos_spectral, new_meth_data_pc, consensus_clustering_spectral, cancer_to_keep)
avg_meth_snf_spectral.columns=[1,2,3]
avg_meth_snf_spectral

In [ ]:
# make sure the cluster number corresponds to the average beta value (1=lowest, 3=highest)
for cancer in avg_meth_snf_spectral.index:
    sorted_arg = np.argsort(avg_meth_snf_spectral.loc[cancer])
    cl_order = {}
    for i in range(avg_meth_snf_spectral.loc[cancer].dropna().shape[0]):
        cl_order[sorted_arg[i+1]+1] = i+1
    consensus_clustering_spectral[cancer] = consensus_clustering_spectral[cancer].replace({"cluster": cl_order})

In [ ]:
df_umap_spectral,umap_red_data_pd = {},{}
for cancer in cancer_to_keep:
    umap_red_data_pd[cancer] = pd.DataFrame(umap_red_data[cancer],index=consensus_clustering_spectral[cancer].index,columns=["Comp1","Comp2"])
    if cancer in triple_cluster_cancers_aff:
        df_umap_spectral[cancer] = pd.concat([umap_red_data_pd[cancer],consensus_clustering_spectral[cancer]],axis=1).replace({1: "Low meth cluster", 2: "Intermediate meth cluster", 3: "High meth cluster"})
    else:
        df_umap_spectral[cancer] = pd.concat([umap_red_data_pd[cancer],consensus_clustering_spectral[cancer]],axis=1).replace({1: "Low meth cluster", 2: "High meth cluster"})
    df_umap_spectral[cancer].columns = ["Comp1","Comp2","Cluster"]

In [ ]:
# compute the associated silhouette score in the 2D UMAP reduced space
sil_scores = []
sil_scores_means= []
for cancer in tqdm(cancer_to_keep):
    sil_pd = pd.concat([new_meth_data_pc[cancer],consensus_clustering_spectral[cancer]],axis=1).dropna()
    sil = silhouette_samples(sil_pd.drop("cluster",axis=1),sil_pd["cluster"].values.ravel(),metric="euclidean")
    mn = np.mean(sil)
    print(cancer,mn)
    sil_scores.append(sil)
    sil_scores_means.append(mn)
print("Mean {} IQR [{}-{}]".format(np.mean(sil_scores_means),np.quantile(sil_scores_means,0.25),np.quantile(sil_scores_means,0.75)))

In [ ]:
plt.rc('font', size=15)
fig, ax = plt.subplots(7,4,figsize=(15,20))
flatax = ax.flatten()

In [ ]:
# plot the silhouette score per sample for each cancer type 
# code inspired from https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

import matplotlib.cm as cm
seapal = sns.color_palette("bright")
plt.rc('font', size=15)
for j in range(len(sil_scores)):

    cluster_labels = consensus_clustering_spectral[cancer_to_keep[j]].values.ravel()
    sample_silhouette_values = sil_scores[j]
    silhouette_avg = np.mean(sample_silhouette_values)
    n_clusters =  np.max(cluster_labels)
    if n_clusters == 2:
        colors = {0: seapal[0], 1: seapal[3]}
    else:
        colors = {0: seapal[0], 1: seapal[8], 2: seapal[3]}
    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i+1]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = colors[i]
        flatax[j].fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    flatax[j].set_title("The silhouette plot for "+cancer_to_keep[j],fontsize=15)
    flatax[j].set_xlabel("SC",fontsize=10)
    flatax[j].set_ylabel("Cluster label",fontsize=10)

    # The vertical line for average silhouette score of all the values
    flatax[j].axvline(x=silhouette_avg, color="red", linestyle="--")

    flatax[j].set_yticks([])  # Clear the yaxis labels / ticks
    flatax[j].set_xticks([-0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
    flatax[j].tick_params(labelsize=10)

In [ ]:
flatax[-1].axis("off")
flatax[-2].axis("off")
fig.tight_layout()
fig

In [ ]:
fig.savefig("silhouette_coefficients_full.svg",bbox_inches="tight")
fig.savefig("silhouette_coefficients_full.png",bbox_inches="tight")

In [ ]:
def get_plot_df(df_umap_red,sil_scores):
#     Parameters: df_umap_red (dict): keys are cancer types, values are pd.Df (n_patients,3) with columns 
#             the 3 UMAP components coordinates and the cluster membership
#             sil_scores (list): list of size n_cancers with np.array of size n_patients with silhouette score per sample
#     Returns: plot_df_umap_red (pd.Df): rows are patients, columns are the 2 UMAP components coordinates, 
#                        the cluster membership, the cancer type and the silhouette coefficient (n_total_patients,5)
    plt.rc('font', size=15)
    plot_df_umap_red = []
    for i,cancer in enumerate(df_umap_red):
        df = df_umap_red[cancer].copy()
        df["cancer"] = [cancer]*len(df.index)
        sscore = np.array(["Positive" if sil_scores[i][j]>=0 else "Negative" for j in range(len(sil_scores[i]))]).reshape(-1,1)
        df = pd.concat([df,pd.DataFrame(sil_scores[i],index=df.index,columns=["Sil_orig"]),pd.DataFrame(sscore,index=df.index,columns=["Silhouette"])],axis=1)
        if len(plot_df_umap_red)==0:
            plot_df_umap_red = df
        else:
            plot_df_umap_red = pd.concat([plot_df_umap_red,df])
    return plot_df_umap_red


In [ ]:
def annotate(data, **kws):
    print(data)
    n = np.round(data["Sil_orig"].mean(),2)
    ax = plt.gca()
    

def get_plot_clustering(plot_df_umap_red):
#     Parameters: plot_df_umap_red (pd.Df): rows are patients, columns are the 2 UMAP components coordinates, 
#             the cluster membership, the cancer type and the silhouette coefficient (n_total_patients,5)
#     Returns: None (plots the figure)
    seapal = sns.color_palette("bright")
    plt.rc('font', size=30)
    facet = sns.relplot(data=plot_df_umap_red, x='Comp1', y='Comp2', hue='Cluster', col="cancer", size="Silhouette", sizes={"Positive": 50, "Negative": 20}, col_wrap=7, palette={"Low meth cluster": seapal[0], "High meth cluster": seapal[3],"Intermediate meth cluster": seapal[8]},
                   facet_kws={'sharey': False, 'sharex': False})
    for i,ax in enumerate(facet.axes):
        df = plot_df_umap_red[plot_df_umap_red["cancer"]==cancer_to_keep[i]]
        ax.text(.9, .9, np.round(df["Sil_orig"].mean(),2), transform=ax.transAxes)
    facet.set_titles('{col_name}')
    facet.set(xticklabels=[])
    facet.set(yticklabels=[])
    facet.fig.subplots_adjust(top=0.9,right=0.85)
    plt.setp(facet._legend.get_title(), fontsize=30)
    plt.setp(facet._legend.get_texts(), fontsize=30)
    plt.setp(facet._legend,bbox_to_anchor=(1.01,0.5))
    plt.savefig("UMAP_clusters.svg",bbox_inches="tight")
    plt.savefig("UMAP_clusters.png",bbox_inches="tight")
    return facet

In [ ]:
plot_df_spectral = get_plot_df(df_umap_spectral,sil_scores)

In [ ]:
%matplotlib inline
facet = get_plot_clustering(plot_df_spectral)

## Correlation between clustering and clinical features

We compute significant associations between clinical variables and the cluster membership

In [ ]:
for cancer in tqdm(cancer_to_keep):
    print(cancer)
    cancer_clin_raw = cancer_clin_raw_pc[cancer]
    common_pat_clustering = consensus_clustering_spectral[cancer].loc[consensus_clustering_spectral[cancer].index.intersection(cancer_clin_raw.index)]
    ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
    ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
    ind_cl_3 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
    
    for clin_cat in cancer_clin_raw.columns:
        # Kruskal wallis for continuous variables
        if pd.api.types.is_numeric_dtype(cancer_clin_raw[clin_cat]):
            values_clin_cat = cancer_clin_raw[clin_cat].loc[common_pat_clustering.index].fillna(-1)
            if len(Counter(values_clin_cat))==1:
                continue
            val_cl1,val_cl2,val_cl3 = values_clin_cat.loc[ind_cl_1].values, values_clin_cat.loc[ind_cl_2].values, values_clin_cat.loc[ind_cl_3].values
            if cancer in triple_cluster_cancers_aff:
                kr_p_value = kruskal(val_cl1,val_cl2,val_cl3)[1]
            else:
                kr_p_value = kruskal(val_cl1,val_cl2)[1]
            if kr_p_value<0.05/cancer_clin_raw.shape[1]:
                print("Kruskal {}: {}".format(clin_cat,kr_p_value))
        else:
            # chi square for categorical variables
            values_clin_cat = cancer_clin_raw[clin_cat].fillna("NA")
            if len(Counter(values_clin_cat))==1:
                continue
            freq_cl_1 = pd.DataFrame.from_dict(dict(Counter(values_clin_cat.loc[ind_cl_1].values)),orient="index",columns=["Low"])
            freq_cl_2 = pd.DataFrame.from_dict(dict(Counter(values_clin_cat.loc[ind_cl_2].values)),orient="index",columns=["Int"])
            if cancer in triple_cluster_cancers_aff:
                freq_cl_3 = pd.DataFrame.from_dict(dict(Counter(values_clin_cat.loc[ind_cl_3].values)),orient="index",columns=["High"])
                cont_table = pd.concat([freq_cl_1,freq_cl_2,freq_cl_3],axis=1).fillna(0).T
            else:
                cont_table = pd.concat([freq_cl_1,freq_cl_2],axis=1).fillna(0).T
            if cont_table.shape[1]==1:
                continue
            if "NA" in cont_table.columns:
                cont_table = cont_table.drop("NA",axis=1)
                if cont_table.shape[1]==1:
                    continue
            # check assumption for chi square
            if (cont_table.values<5).any():
                continue
            chi_p_value = chi2_contingency(cont_table)[1]
            if (chi_p_value<0.05/cancer_clin_raw.shape[1]).any():
                print(cont_table)
                print("Chi square {}: {}".format(clin_cat,chi_p_value))

# Cluster characterization through methylation distribution

In [ ]:
def get_dist_methylation_snf(sign_pos, meth_data_pc, consensus_clustering, cancer_order):
#     Parameters: meth_data_pc (dict): keys are cancer types, values are pd.DataFrame (n_pat,n_cpg) (rows=patients,columns=cpg)
#             sign_pos (dict): keys are cancer types, values are pd.DataFrame (n_sign_pos,1) (rows=sign_pos,)
#             consensus_clustering (dict): keys are cancer types, values are pd.DataFrame (n_pat,1) (rows=patients, column=cluster membership)
#             cancer_order (list): list of cancer types in the KS order
#     Returns: df_list (pd.Df): the average beta values over significant positions with cluster membership and cancer type (n_total_patients,3)
    df_list = []
    for cancer in cancer_order:
        print(cancer)
        values = []
        n_clusters = len(np.unique(consensus_clustering[cancer]["cluster"].values))
        for i in range(n_clusters):
            if n_clusters==2:
                mapping_leg = {0: "Low meth cluster", 1: "High meth cluster"}
            else:
                mapping_leg = {0: "Low meth cluster", 1: "Intermediate meth cluster", 2: "High meth cluster"}
            values.append(pd.concat([meth_data_pc[cancer].iloc[np.where(consensus_clustering[cancer]["cluster"].values==i+1)[0]][sign_pos[cancer].values.ravel()].mean(),pd.DataFrame([mapping_leg[i]]*len(sign_pos[cancer]),index=sign_pos[cancer].values.ravel(), columns=["Cluster"])],axis=1))
        df = pd.concat(values)
        df = pd.concat([df,pd.DataFrame([cancer]*df.shape[0],columns=["cancer"],index=df.index)],axis=1)
        df.columns = ["beta","cluster","cancer"]
        df_list.append(df)
    return pd.concat(df_list)

In [ ]:
dist_meth_spectral_1 = get_dist_methylation_snf(sign_pos_spectral, new_meth_data_pc, consensus_clustering_spectral, cancer_to_keep[:19])
dist_meth_spectral_2 = get_dist_methylation_snf(sign_pos_spectral, new_meth_data_pc, consensus_clustering_spectral, cancer_to_keep[19:])

In [ ]:
def get_dist_meth_with_normal(dist_meth_spectral,normal_meth,cancer_to_keep):
#     Parameters: dist_meth_spectral (pd.Df): the average beta values over significant positions 
#             with cluster membership and cancer type (n_total_patients,3)
#             normal_meth (dict): keys are cancer types, values are pd.Df (n_normal_samples,n_cpg) (rows=normal samples, columns=cpg)
#             cancer_to_keep (list): list of cancer types
#     Returns: new_dist_meth (pd.Df): same as dist_meth_spectral  but with normal tissues added
    new_dist_meth = dist_meth_spectral.copy()
    for cancer in tqdm(cancer_to_keep):
        nmavg = normal_meth[cancer].mean()
        normalpd = pd.concat([nmavg,pd.DataFrame(["Normal tissue"]*nmavg.shape[0],index=nmavg.index),pd.DataFrame([cancer]*nmavg.shape[0],index=nmavg.index)],axis=1)
        normalpd.columns=["beta","cluster","cancer"]
        new_dist_meth = pd.concat([new_dist_meth,normalpd])
    return new_dist_meth

In [ ]:
new_dist_meth_1 = get_dist_meth_with_normal(dist_meth_spectral_1,normal_meth,cancer_to_keep[:19])
new_dist_meth_2 = get_dist_meth_with_normal(dist_meth_spectral_2,normal_meth,cancer_to_keep[19:])

In [ ]:
diff_mean_cl,diff_mean_normal = [],[]
for i,cancer in enumerate(cancer_to_keep):
    if i<19:
        df = new_dist_meth_1[new_dist_meth_1["cancer"]==cancer]
    else:
        df = new_dist_meth_2[new_dist_meth_2["cancer"]==cancer]
    mean_low = df[df["cluster"]=="Low meth cluster"]["beta"].mean()
    mean_high = df[df["cluster"]=="High meth cluster"]["beta"].mean()
    mean_normal = df[df["cluster"]=="Normal tissue"]["beta"].mean()
    diff_mean = mean_high - mean_low
    diff_mean_n = mean_high - mean_normal
    diff_mean_cl.append(diff_mean)
    diff_mean_normal.append(diff_mean_n)


In [ ]:
# order cancer types according to average between low and high
cancer_order = np.array(list(cancer_to_keep))[np.argsort(list(diff_mean_cl))[::-1]]

In [ ]:
dist_meth_spectral_1 = get_dist_methylation_snf(sign_pos_spectral, new_meth_data_pc, consensus_clustering_spectral, cancer_order[:19])
dist_meth_spectral_2 = get_dist_methylation_snf(sign_pos_spectral, new_meth_data_pc, consensus_clustering_spectral, cancer_order[19:])

new_dist_meth_1 = get_dist_meth_with_normal(dist_meth_spectral_1,normal_meth,cancer_order[:19])
new_dist_meth_2 = get_dist_meth_with_normal(dist_meth_spectral_2,normal_meth,cancer_order[19:])

In [ ]:
plt.rc("font", size=20)
seapal = sns.color_palette("deep")
fig = plt.figure(figsize=(13,3))
ax = plt.axes()
sns.lineplot(x=cancer_order,y=np.sort(diff_mean_cl)[::-1],ax=ax,color=seapal[0],label="High vs low")
sns.scatterplot(x=cancer_order,y=np.sort(diff_mean_cl)[::-1],ax=ax,s=200,color=seapal[0])

sns.lineplot(x=cancer_order,y=np.sort(diff_mean_normal)[::-1],ax=ax,color=seapal[3],label="High vs normal")
sns.scatterplot(x=cancer_order,y=np.sort(diff_mean_normal)[::-1],ax=ax,s=200,color=seapal[3])

ax.set_xticklabels(cancer_order,rotation=65,rotation_mode="default",fontsize=25)
ax.set_yticks([0,0.1,0.2,0.3,0.4,0.5,0.6])
ax.set_yticklabels(np.round(ax.get_yticks(),2),rotation=0,rotation_mode="default",fontsize=25)
ax.set_ylabel("Difference \ngroup means",fontsize=25)

plt.axvline(x=18.5,color=seapal[7],linewidth=4)
plt.axhline(y=0.2,color=seapal[7],linestyle="--")

plt.legend(loc='right', bbox_to_anchor=(1.35,0.5), title="Groups compared",frameon=False,fontsize=20)

ax.text(19,0.5,"CIMP limit",fontsize=25,color=seapal[7])

fig.savefig("mean_comparison_panel.png",bbox_inches="tight")
fig.savefig("mean_comparison_panel.svg",bbox_inches="tight")

In [ ]:
x_ticks = []
for cancer in cancer_order:
    cl1 = consensus_clustering_spectral[cancer][consensus_clustering_spectral[cancer]["cluster"]==1].index
    cl2 = consensus_clustering_spectral[cancer][consensus_clustering_spectral[cancer]["cluster"]==2].index
    cl3 = consensus_clustering_spectral[cancer][consensus_clustering_spectral[cancer]["cluster"]==3].index
    label = cancer+" \n $n_{low}$="+str(len(cl1))
    if cancer in triple_cluster_cancers_aff:
        label += " \n $n_{inter}$="+str(len(cl2))+"\n $n_{high}$="+str(len(cl3))
    else:
        label += "\n $n_{high}$="+str(len(cl2))
    label += "\n $n_{ref}$="+str(normal_meth[cancer].shape[0])
    x_ticks.append(label)

In [ ]:
%matplotlib inline
plt.rc('font', size=50)
plt.figure(figsize=(65,13))
seapal = sns.color_palette("bright")
g = sns.violinplot(x="cancer",y="beta",hue="cluster",
                   hue_order=["Low meth cluster","Intermediate meth cluster","High meth cluster","Normal tissue"],
                   data=new_dist_meth_1, legend_out=False,
                   palette={"Low meth cluster": seapal[0],"High meth cluster": seapal[3],"Intermediate meth cluster": seapal[8],"Normal tissue": seapal[7]},
                  linewidth=4)


g.set_xticklabels(x_ticks[:13], fontsize=33)
g.set_ylabel("Average beta-values")
add_stat_annotation(g, data=new_dist_meth_1, x="cancer", y="beta", hue="cluster", hue_order=["Low meth cluster","Intermediate meth cluster","High meth cluster","Normal tissue"],
                                box_pairs=[(("ACC", "Low meth cluster"), ("ACC","High meth cluster")),
                                           (("ACC", "Low meth cluster"), ("ACC","Intermediate meth cluster")),
                                           (("ACC", "Intermediate meth cluster"), ("ACC","High meth cluster")),
                                   (("COAD", "Low meth cluster"), ("COAD","High meth cluster")),
                                           (("COAD", "Low meth cluster"), ("COAD","Intermediate meth cluster")),
                                           (("COAD", "Intermediate meth cluster"), ("COAD","High meth cluster")),
                                          
                                   (("KIRP", "Low meth cluster"), ("KIRP","High meth cluster")),
                                   (("LAML", "Low meth cluster"), ("LAML","High meth cluster")),
                                   (("LIHC", "Low meth cluster"), ("LIHC","High meth cluster")),
                                           (("LIHC", "Low meth cluster"), ("LIHC","Intermediate meth cluster")),
                                           (("LIHC", "Intermediate meth cluster"), ("LIHC","High meth cluster")),
                                           (("LUAD", "Low meth cluster"), ("LUAD","High meth cluster")),
                                           (("LUAD", "Low meth cluster"), ("LUAD","Intermediate meth cluster")),
                                           (("LUAD", "Intermediate meth cluster"), ("LUAD","High meth cluster")),
                                           (("MESO", "Low meth cluster"), ("MESO","High meth cluster")),
                                           (("MESO", "Low meth cluster"), ("MESO","Intermediate meth cluster")),
                                           (("MESO", "Intermediate meth cluster"), ("MESO","High meth cluster")),
                                   (("PCPG", "Low meth cluster"), ("PCPG","High meth cluster")),
                                    (("PCPG", "Low meth cluster"), ("PCPG","Intermediate meth cluster")),
                                    (("PCPG", "Intermediate meth cluster"), ("PCPG","High meth cluster")),
                                    (("READ", "Low meth cluster"), ("READ","High meth cluster")),
                                           (("READ", "Low meth cluster"), ("READ","Intermediate meth cluster")),
                                           (("READ", "Intermediate meth cluster"), ("READ","High meth cluster")),   
                                   (("SARC", "Low meth cluster"), ("SARC","High meth cluster")),
                                            (("SKCM", "Low meth cluster"), ("SKCM","High meth cluster")),
                                   (("SKCM", "Low meth cluster"), ("SKCM","Intermediate meth cluster")),
                                   (("SKCM", "Intermediate meth cluster"), ("SKCM","High meth cluster")),  
                                    (("STAD", "Low meth cluster"), ("STAD","High meth cluster")),
                                          (("THCA", "Low meth cluster"), ("THCA","High meth cluster")),
                                          ],
                    test='Mann-Whitney', text_format='star', loc='inside', verbose=2)


plt.legend(loc='center', bbox_to_anchor=(0.5, 1.2), title="Cluster",ncol=4)
plt.xlabel('')

plt.savefig("dist_methylation_part1_new.svg",bbox_inches='tight')
plt.savefig("dist_methylation_part1_new.png",bbox_inches='tight')

In [ ]:
plt.rc('font', size=50)
plt.figure(figsize=(65,13))
seapal = sns.color_palette("bright")
g = sns.violinplot(x="cancer",y="beta",hue="cluster",
                   hue_order=["Low meth cluster","Intermediate meth cluster","High meth cluster","Normal tissue"],
                   data=new_dist_meth_2, legend_out=False,
                   palette={"Low meth cluster": seapal[0],"High meth cluster": seapal[3],"Intermediate meth cluster": seapal[8],"Normal tissue": seapal[7]},
                  linewidth=4)

g.set_xticklabels(x_ticks[13:], fontsize=33)
g.set_ylabel("Average beta-values")
add_stat_annotation(g, data=new_dist_meth_2, x="cancer", y="beta", hue="cluster", hue_order=["Low meth cluster","Intermediate meth cluster","High meth cluster","Normal tissue"],
                                box_pairs=[
                                   (("CESC", "Low meth cluster"), ("CESC","High meth cluster")),
                                    (("GBM", "Low meth cluster"), ("GBM","High meth cluster")),
                                           (("GBM", "Low meth cluster"), ("GBM","Intermediate meth cluster")),
                                           (("GBM", "Intermediate meth cluster"), ("GBM","High meth cluster")),
                                   (("HNSC", "Low meth cluster"), ("HNSC","High meth cluster")),
                                   (("HNSC", "Low meth cluster"), ("HNSC","Intermediate meth cluster")),
                                   (("HNSC", "Intermediate meth cluster"), ("HNSC","High meth cluster")),
                                            (("KIRC", "Low meth cluster"), ("KIRC","High meth cluster")),
                                           (("KIRC", "Low meth cluster"), ("KIRC","Intermediate meth cluster")),
                                           (("KIRC", "Intermediate meth cluster"), ("KIRC","High meth cluster")),
                                   (("LGG", "Low meth cluster"), ("LGG","High meth cluster")),
                                            (("LUSC", "Low meth cluster"), ("LUSC","High meth cluster")),
                                   (("LUSC", "Low meth cluster"), ("LUSC","Intermediate meth cluster")),
                                           (("LUSC", "Intermediate meth cluster"), ("LUSC","High meth cluster")),
                                    (("BLCA", "Low meth cluster"), ("BLCA","High meth cluster")),
                                    (("BLCA", "Intermediate meth cluster"), ("BLCA","High meth cluster")),
                                    (("BLCA", "Low meth cluster"), ("BLCA","Intermediate meth cluster")),
                                           (("BRCA", "Low meth cluster"), ("BRCA","Intermediate meth cluster")),
                                   (("BRCA", "Low meth cluster"), ("BRCA","High meth cluster")),
                                   (("BRCA", "Intermediate meth cluster"), ("BRCA","High meth cluster")),
                                    (("ESCA", "Low meth cluster"), ("ESCA","High meth cluster")),
                                           (("ESCA", "Low meth cluster"), ("ESCA","Intermediate meth cluster")),
                                           (("ESCA", "Intermediate meth cluster"), ("ESCA","High meth cluster")),
                                           (("PAAD", "Low meth cluster"), ("PAAD","High meth cluster")),
                                           (("PAAD", "Low meth cluster"), ("PAAD","Intermediate meth cluster")),
                                           (("PAAD", "Intermediate meth cluster"), ("PAAD","High meth cluster")),
                                   (("PRAD", "Low meth cluster"), ("PRAD","High meth cluster")),
                                           (("THYM", "Low meth cluster"), ("THYM","High meth cluster")),
                                           (("UCEC", "Low meth cluster"), ("UCEC","High meth cluster")),
                                           (("UCEC", "Low meth cluster"), ("UCEC","Intermediate meth cluster")),
                                   (("UCEC", "Intermediate meth cluster"), ("UCEC","High meth cluster")),],
                    test='Mann-Whitney', text_format='star', loc='inside', verbose=2)

g.get_legend().remove()
plt.savefig("dist_methylation_part2_new.svg",bbox_inches='tight')
plt.savefig("dist_methylation_part2_new.png",bbox_inches='tight')

### CIMP status characterization

In [ ]:
# look for differences in group average beta value to characterize into CIMP and non-CIMP
cimp_cancer_wdiff = []
for cancer in cancer_order[:13]:
    distmethcancer = dist_meth_spectral_1[dist_meth_spectral_1["cancer"]==cancer]
    cl1_avg = distmethcancer[distmethcancer["cluster"]=="Low cluster"]["beta"].mean()
    cl2_avg = distmethcancer[distmethcancer["cluster"]=="High cluster"]["beta"].mean()
    print("Cancer {} diff between cluster avg : {} & high group value : {}".format(cancer,cl2_avg-cl1_avg,cl2_avg))
    if cl2_avg-cl1_avg>0.195:
        cimp_cancer_wdiff.append(cancer)
cimp_cancer_wdiff

In [ ]:
cimp_cancer_wdiff= []
for cancer in cancer_order[13:]:
    distmethcancer = dist_meth_spectral_2[dist_meth_spectral_2["cancer"]==cancer]
    cl1_avg = distmethcancer[distmethcancer["cluster"]=="Low cluster"]["beta"].mean()
    cl2_avg = distmethcancer[distmethcancer["cluster"]=="High cluster"]["beta"].mean()
    print("Cancer {} diff between cluster avg : {} & high group value : {}".format(cancer,cl2_avg-cl1_avg,cl2_avg))
    if cl2_avg-cl1_avg>0.195:
        cimp_cancer_wdiff.append(cancer)
cimp_cancer_wdiff

In [ ]:
up_probes_perc,down_probes_perc = [],[]
for cancer in tqdm(cancer_order):
    ind_cl_1 = consensus_clustering_spectral[cancer][consensus_clustering_spectral[cancer]["cluster"]==1].index
    ind_cl_2 = consensus_clustering_spectral[cancer][consensus_clustering_spectral[cancer]["cluster"]==2].index
    ind_cl_3 = consensus_clustering_spectral[cancer][consensus_clustering_spectral[cancer]["cluster"]==3].index

    if cancer in triple_cluster_cancers_aff:
        ind_cl_2 = ind_cl_3
        
    methdf = new_meth_data_pc[cancer][sign_pos_spectral[cancer].values.ravel()]
    perc = (methdf.loc[ind_cl_2].mean()>methdf.loc[ind_cl_1].mean()).sum()/(methdf.shape[1])
    perc = np.round(perc*100,1)
    up_probes_perc.append(perc)
    down_probes_perc.append(100-perc)

In [ ]:
seapal = sns.color_palette("deep")
plt.rc("font",size=20)
width = 0.6 
fig, ax = plt.subplots(figsize=(13,3))

ax.bar(cancer_order, up_probes_perc, width, label='Up',color=seapal[3])
ax.bar(cancer_order, down_probes_perc, width, bottom=up_probes_perc,
       label='Down',color=seapal[0])

ax.set_xticklabels(cancer_order,rotation=60,rotation_mode="default",fontsize=25)
ax.set_ylabel("(%)",fontsize=25)

plt.legend(loc='right', bbox_to_anchor=(1.35,0.5), title="Methylation direction \n         of probes",frameon=False,fontsize=18)
ax.axvline(x=18.5, color=seapal[7],linewidth=4)
ax.text(18,110,"CIMP limit",color=seapal[7])

fig.savefig("probe_direction_panel.png",bbox_inches="tight")
fig.savefig("probe_direction_panel.svg",bbox_inches="tight")

In [ ]:
CIMP_cancers = ['KIRP','SARC','MESO','ACC','READ','LAML','STAD','PCPG','CESC',
                'LUSC','LIHC','COAD','KIRC','THCA','GBM','SKCM','LUAD','HNSC','LGG']

# Predicting CIMP status with probes

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
# create and save the train and testing sets
X_train_pc,y_train_pc,X_test_pc,y_test_pc = {},{},{},{}
for cancer in tqdm(sorted(CIMP_cancers)):
    X = new_meth_data_pc[cancer][sign_pos_spectral[cancer].values.ravel()]
    if cancer in triple_cluster_cancers_aff:
        y = consensus_clustering_spectral[cancer].replace({3: 1, 2: 0, 1: 0}).values.ravel()
    else:
        y = consensus_clustering_spectral[cancer].replace({2: 1, 1: 0}).values.ravel()
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
    
    X_train_pc[cancer]=X_train
    y_train_pc[cancer]=y_train
    X_test_pc[cancer]=X_test
    y_test_pc[cancer]=y_test

for cancer in X_train_pc:
    pd.DataFrame(X_train_pc[cancer].index).to_csv(os.path.join(bac_pred_dir,cancer+"_train_pat.csv"),index=False)
    pd.DataFrame(X_test_pc[cancer].index).to_csv(os.path.join(bac_pred_dir,cancer+"_test_pat.csv"),index=False)

In [ ]:
bac_pred_dir = "path\to\save\bac_pred"

X_train_pc,y_train_pc,X_test_pc,y_test_pc = {},{},{},{}
for cancer in tqdm(sorted(CIMP_cancers)):
    pat_train = pd.read_csv(os.path.join(bac_pred_dir,cancer+"_train_pat.csv")).values.ravel()
    pat_test = pd.read_csv(os.path.join(bac_pred_dir,cancer+"_test_pat.csv")).values.ravel()
    # get the train and testing set
    X_train_pc[cancer]=new_meth_data_pc[cancer][sign_pos_spectral[cancer].values.ravel()].loc[pat_train]
    X_test_pc[cancer]=new_meth_data_pc[cancer][sign_pos_spectral[cancer].values.ravel()].loc[pat_test]
    if cancer in triple_cluster_cancers_aff:
        y = consensus_clustering_spectral[cancer].replace({3: 1, 2: 0, 1: 0})
    else:
        y = consensus_clustering_spectral[cancer].replace({2: 1, 1: 0})
    y_train_pc[cancer]= y.loc[pat_train]
    y_test_pc[cancer]= y.loc[pat_test]

In [ ]:
from itertools import combinations
from sklearn.base import clone

class SequentialForwardSelection():
     
    '''
    Instantiate with Estimator and given number of features
    '''
    def __init__(self, estimator, k_features):
        self.estimator = clone(estimator)
        self.k_features = k_features
         
    '''
    X_train - Training data Pandas dataframe
    X_test - Test data Pandas dataframe
    y_train - Training label Pandas dataframe
    y_test - Test data Pandas dataframe
    '''  
        
    def fit_cv(self, X_train, y_train, n_cv):
        max_indices = tuple(range(X_train.shape[1]))
        total_features_count = len(max_indices)
        self.subsets_ = []
        self.scores_ = []
        self.indices_ = []
        
        # create the Stratified k-fold 
        kf = StratifiedKFold(n_splits=n_cv)
        X, y = X_train, y_train
        
        scores = []
        subsets = []
        for p in combinations(max_indices, r=1):
            s = 0
            for train_index, test_index in kf.split(X,y):
                score = self._calc_score(X.iloc[train_index].values, X.iloc[test_index].values, 
                                         y.iloc[train_index].values, y.iloc[test_index].values, p)
                s += score/n_cv
            scores.append(s)
            subsets.append(p)
        '''
        Iterate through the feature space to find the first feature
        which gives the maximum model performance
        '''
        #
        # Find the single feature having best score
        #
        best_score_index = np.argmax(scores)
        self.scores_.append(scores[best_score_index])
        self.indices_ = list(subsets[best_score_index])
        self.subsets_.append(self.indices_)
        print("Found first feature...")
        #
        # Add a feature one by one until k_features is reached
        #
        dim = 1
        while dim < self.k_features:
            print(f"Testing with feature {dim}...")
            scores = []
            subsets = []
            current_feature = dim
            '''
            Add the remaining features one-by-one from the remaining feature set
            Calculate the score for every feature combinations
            '''
            idx = 0
            while idx < total_features_count:
                if idx not in self.indices_:
                    indices = list(self.indices_)
                    indices.append(idx)
                    s=0
                    for train_index, test_index in kf.split(X,y):
                        score = self._calc_score(X.iloc[train_index].values, X.iloc[test_index].values, 
                                                 y.iloc[train_index].values, y.iloc[test_index].values, indices)
                        s += score/n_cv
                    scores.append(s)
                    subsets.append(indices)
                idx += 1
             
            #
            # Get the index of best score
            #
            best_score_index = np.argmax(scores)
            #
            # Record the best score
            #
            self.scores_.append(scores[best_score_index])
            #
            # Get the indices of features which gave best score
            #
            self.indices_ = list(subsets[best_score_index])
            #
            # Record the indices of features for best score
            #
            self.subsets_.append(self.indices_)
             
            dim += 1
             
        self.k_score_ = self.scores_[-1]
      
    '''
    Transform training, test data set to the data set
    havng features which gave best score
    '''
    def transform(self, X):
        return X.values[:, self.indices_]
     
    '''
    Train models with specific set of features
    indices - indices of features
    '''
    def _calc_score(self, X_train, X_test, y_train, y_test, indices):
        self.estimator.fit(X_train[:, indices], y_train.ravel())
        y_pred = self.estimator.predict(X_test[:, indices])
        score = balanced_accuracy_score(y_test, y_pred)
        return score

In [ ]:
k_features = 5
n_cv = 5

best_probes_pc, best_scores_pc = {},{}

for cancer in tqdm(sorted(CIMP_cancers)):
    # train on 5 fold cvs the Sequential Forward Selection on a logistic regression with default regularization
    lr = LogisticRegression(C=1.0, random_state=1, class_weight="balanced")
    sfs = SequentialForwardSelection(lr, k_features)
    print(cancer)
    sfs.fit_cv(X_train_pc[cancer], y_train_pc[cancer], n_cv)

    probes = list(X_train_pc[cancer].columns[sfs.indices_])

    print(f"{cancer} Best probes {probes}, best ABAC {sfs.scores_}")
    
    best_probes_pc[cancer] = probes
    best_scores_pc[cancer] = sfs.scores_
    # save the best probes and their associated 5-fold ABAC on validation
    df = pd.DataFrame(
            np.array(best_scores_pc[cancer]).reshape(-1,1),
            index=best_probes_pc[cancer],
            columns=["ABAC"]
    )
    df.to_csv(os.path.join(bac_pred_dir,cancer+"_sfs_probes_scores.csv"))

In [ ]:
best_probes_df = {}
for cancer in sorted(CIMP_cancers):
    df = pd.read_csv(os.path.join(bac_pred_dir,cancer+"_sfs_probes_scores.csv"))
    df.columns = ["probes","ABAC"]
    best_probes_df[cancer] = df

In [ ]:
avg_abac_train, avg_abac_test, avg_acc_test = [],[],[]
# get the ABAC on the validation and test set, and the accuracy on the test set
for cancer in sorted(CIMP_cancers):
    lr = LogisticRegression(C=1.0, random_state=1, class_weight="balanced")
    bp = best_probes_df[cancer].probes.ravel()
    bs = best_probes_df[cancer].ABAC.ravel()
    abac = []
    acc = []
    for i in range(len(bp)):
        lr.fit(X_train_pc[cancer][bp[:i+1]],y_train_pc[cancer].values.ravel())
        y_pred = lr.predict(X_test_pc[cancer][bp[:i+1]])
        abac.append(balanced_accuracy_score(y_pred,y_test_pc[cancer].values.ravel()))
        acc.append(accuracy_score(y_pred,y_test_pc[cancer].values.ravel()))
    final_nprobes = np.argmax(np.array(abac)+bs)+1
    print(cancer,final_nprobes,bp[:final_nprobes],bs[final_nprobes-1],abac[final_nprobes-1],acc[final_nprobes-1])
    avg_abac_train.append(bs[final_nprobes-1])
    avg_abac_test.append(abac[final_nprobes-1])
    avg_acc_test.append(acc[final_nprobes-1])
print(f"Avg ABAC train {np.mean(avg_abac_train)} [{np.quantile(avg_abac_train,0.25)}-{np.quantile(avg_abac_train,0.75)}], \
      avg ABAC test {np.mean(avg_abac_test)} [{np.quantile(avg_abac_test,0.25)}-{np.quantile(avg_abac_test,0.75)}], \
      avg ACC test {np.mean(avg_acc_test)} [{np.quantile(avg_acc_test,0.25)}-{np.quantile(avg_acc_test,0.75)}]")

# Survival

In [ ]:
# clean the clinical data
for cancer in new_clin_data_pc:
    new_clin_data_pc[cancer]["OS.time"] = new_clin_data_pc[cancer]["OS.time"].replace("[Discrepancy]", "NaN")
    new_clin_data_pc[cancer]["OS.time"] = new_clin_data_pc[cancer]["OS.time"].astype(float)
    new_clin_data_pc[cancer]["OS"] = new_clin_data_pc[cancer]["OS"].replace("[Discrepancy]", "NaN")
    new_clin_data_pc[cancer]["OS"] = new_clin_data_pc[cancer]["OS"].astype(float)

In [ ]:
# unify the stage indications in clinical data
mapping_stage = {"Stage I": ["Stage I", "Stage 0", "I/II NOS", "Stage IA", "Stage IA1", "Stage IA2", "Stage IB", "Stage IB1", "Stage IB2", "Stage IC", "T1c", "T1a", "T1b"],
                "Stage II": ["Stage II", "Stage IIA", "Stage IIA1", "Stage IIA2", "Stage IIB", "Stage IIC", "T2a", "T2b", "T2c", "T2", "T3a", "T3b", "T4"],
                "Stage III": ["Stage III", "Stage IIIA","Stage IIIB", "Stage IIIC", "Stage IIIC1", "Stage IIIC2"],
                "Stage IV": ["Stage IV", "Stage IVA", "Stage IVB","Stage IVC","Stage X"]}
replace_map = {}
for stage in mapping_stage:
    for el in mapping_stage[stage]:
        replace_map[el]=stage

### Kaplan Meier

In [ ]:
def get_survival_analysis_metrics_snf(clin_data_csv,meth_data_pc,consensus_clustering,cancer,feature):
#     Parameters: clin_data_csv (dict): keys are cancer types, values are pd.Df (n_patients,n_clin_info)
#             meth_data_pc (dict): keys are cancer types, values are pd.DataFrame (n_pat,n_cpg) (rows=patients,columns=cpg)
#             consensus_clustering (dict): keys are cancer types, values are pd.DataFrame (n_pat,1) (rows=patients, column=cluster membership)
#             cancer (str): cancer type
#             feature (str): feature to use to break down into groups
#     Returns: duration (np.array): array of survival times
#             event (np.array): array of death/life

    clinmat, duration, event = {},{},{}
    for i in range(len(np.unique(consensus_clustering[cancer][feature].values))):
        clinmat[i] = clin_data_csv[cancer]
        if feature=="cluster":
            pat_cl = consensus_clustering[cancer].index[np.where(consensus_clustering[cancer][feature].values==i+1)]
        else:
            pat_cl = consensus_clustering[cancer].index[np.where(consensus_clustering[cancer][feature].values==i)]
        common_pat = clinmat[i].index.intersection(pat_cl)
        clinmat[i] = clinmat[i].loc[common_pat]
        clinmat[i] = clinmat[i].dropna()
        duration[i], event[i] = [],[]
        for j in range(clinmat[i].shape[0]):
            duration[i].append(float(clinmat[i]["OS.time"].iloc[j]))
            event[i].append(int(clinmat[i]["OS"].iloc[j]))
        duration[i] = np.array(duration[i])
        event[i] = np.array(event[i])
    return duration, event

In [ ]:
def get_kaplan_meier(clin_data_csv, meth_data_pc, consensus_clustering, 
                     triple_cluster_cancers, cancer_list, 
                     flatax1, flatax2, flatax3, 
                     sign_KM_cancers, nsign_KM_cancers, 
                     nCIMP_KM_cancers, feature):
#     Parameters: clin_data_csv (dict): keys are cancer types, values are pd.Df (n_patients,n_clin_info)
#             meth_data_pc (dict): keys are cancer types, values are pd.DataFrame (n_pat,n_cpg) (rows=patients,columns=cpg)
#             consensus_clustering (dict): keys are cancer types, values are pd.DataFrame (n_pat,1) (rows=patients, column=cluster membership)
#             triple_cluster_cancers (list): cancer types with 3 meth clusters
#             cancer_list (list): cancers considered
#             flatax1 (list): list of pyplot axes for plotting significant associations
#             flatax2 (list): list of pyplot axes for plotting non significant associations
#             flatax3 (list): list of pyplot axes for plotting the associations for CIMP negative
#             sign_KM_cancers (list): list cancers with significant associations
#             nsign_KM_cancers (list): list cancers with non-significant associations
#             nCIMP_KM_cancers (list): list CIMP negative cancers 
#             feature (str): feature to use to break down into groups
#     Returns: p_values (list): list of associated p values
    p_values = {}
    fignum1,fignum2,fignum3=0,0,0
    seapal = sns.color_palette("bright")
    for i,cancer in enumerate(cancer_list):
        print(cancer)
        if cancer in triple_cluster_cancers:
            color_pal = [seapal[0], seapal[8], seapal[3]]
        else:
            color_pal = [seapal[0], seapal[3]]
        duration, event = get_survival_analysis_metrics_snf(clin_data_csv, meth_data_pc, consensus_clustering, cancer, feature)
        if len(list(duration.keys()))>3:
            print("More than 3 meth clusters, not taken into account!")
            continue
        ## create a kmf object
        kmf = KaplanMeierFitter() 
        ## Fit the data into the model
        kmf.fit(duration[0], event[0], label='Low meth cluster')
        
        if cancer in sign_KM_cancers:
            fnum=fignum1
            flatax=flatax1
            fignum1+=1
        elif cancer in nsign_KM_cancers:
            fnum=fignum2
            flatax=flatax2
            fignum2+=1
        elif cancer in nCIMP_KM_cancers:
            fnum=fignum3
            flatax=flatax3
            fignum3+=1
            
        ## Create an estimate
        kmf.plot(show_censors=True,c=color_pal[0],ax=flatax[fnum],ci_alpha=0.1)
        flatax[fnum].set_ylim([0,1])
        ## Fit the data into the model
        if not(cancer in(triple_cluster_cancers)):
            kmf.fit(duration[1], event[1], label='High meth cluster')
            ## Create an estimate
            kmf.plot(show_censors=True,c=color_pal[1],ax=flatax[fnum],ci_alpha=0.1)
        else:
            ## Fit the data into the model
            kmf.fit(duration[1], event[1], label='Intermediate meth cluster')
            ## Create an estimate
            kmf.plot(show_censors=True,c=color_pal[1],ax=flatax[fnum])
            ## Fit the data into the model
            kmf.fit(duration[2], event[2], label='High meth cluster')
            ## Create an estimate
            pl1 = kmf.plot(show_censors=True,c=color_pal[2],ax=flatax[fnum],ci_alpha=0.1)
        flatax[fnum].set_title(cancer,fontsize=20)
        flatax[fnum].legend().remove()
        print("Results between meth cluster 0 and 1")
        results=logrank_test(duration[0],duration[1],event_observed_A=event[0], event_observed_B=event[1])
        p_values[cancer] = [results.p_value]
        results.print_summary()
        if cancer in triple_cluster_cancers:
            print("Results between meth cluster 0 and 2")
            results=logrank_test(duration[0],duration[2],event_observed_A=event[0], event_observed_B=event[2])
            p_values[cancer].append(results.p_value)
            results.print_summary()
            print("Results between meth cluster 1 and 2")
            results=logrank_test(duration[1],duration[2],event_observed_A=event[1], event_observed_B=event[2])
            p_values[cancer].append(results.p_value)
            results.print_summary()
    return p_values

In [ ]:
from math import floor,log10
def scinum(num, decimal_digits=1, precision=None, exponent=None):
    """
    Returns a string representation of the scientific
    notation of the given number formatted for use with
    LaTeX or Mathtext, with specified number of significant
    decimal digits and precision (number of decimal digits
    to show). The exponent to be used can also be specified
    explicitly.
    """
    if exponent is None:
        exponent = int(floor(log10(abs(num))))
    coeff = round(num / float(10**exponent), decimal_digits)
    if precision is None:
        precision = decimal_digits

    return r"${0:.{2}f}\cdot10^{{{1:d}}}$".format(coeff, exponent, precision)

In [ ]:
# save patients with silhouette coefficient > 0
high_conf_patients = {cancer: 
                      plot_df_spectral.loc[(plot_df_spectral["cancer"]==cancer) & \ 
                                           (plot_df_spectral["Sil_orig"]>=0)].index for cancer in cancer_to_keep}

In [ ]:
# proportion of HC patients 
avg_prop_HC = []
for cancer in cancer_to_keep:
    prop = high_conf_patients[cancer].shape[0]/new_meth_data_pc[cancer].shape[0]*100
    avg_prop_HC.append(prop)
    print(cancer,f"{prop:.2f}")
print(f"Mean HC prop {np.mean(avg_prop_HC):.2f}, [{np.quantile(avg_prop_HC,0.25):.2f}-{np.quantile(avg_prop_HC,0.75):.2f}]")

In [ ]:
# meth data and cluster membership of HC patients
filtered_meth_data = {cancer: new_meth_data_pc[cancer].loc[high_conf_patients[cancer]] for cancer in cancer_to_keep}
filtered_consensus_clustering = {cancer: consensus_clustering_spectral[cancer].loc[high_conf_patients[cancer]] for cancer in cancer_to_keep}

In [ ]:
plt.rc('font', size=20)

In [ ]:
sign_KM_cancers = ["ACC","GBM","HNSC","KIRC","KIRP","LGG","MESO","SKCM"]
nsign_KM_cancers = ["CESC","COAD","LAML","LIHC","LUAD","LUSC","PCPG","READ","SARC","STAD","THCA"]
nCIMP_KM_cancers = ["BLCA","BRCA","ESCA","PAAD","PRAD","THYM","UCEC"]

In [ ]:
fig1, ax1 = plt.subplots(2,4,figsize=(15,5.5))
flatax1 = ax1.flatten()

fig2, ax2 = plt.subplots(3,4,figsize=(15,8))
flatax2 = ax2.flatten()

fig3, ax3 = plt.subplots(2,4,figsize=(15,5.5))
flatax3 = ax3.flatten()

In [ ]:
km_pval_spectral = get_kaplan_meier(new_clin_data_pc, filtered_meth_data, filtered_consensus_clustering, 
                                    triple_cluster_cancers_aff, cancer_to_keep, 
                                    flatax1, flatax2, flatax3, 
                                    sign_KM_cancers, nsign_KM_cancers, nCIMP_KM_cancers, 
                                    "cluster")

In [ ]:
km_pval_spectral_lin_CIMP,km_pval_spectral_lin = [],[]
for cancer in km_pval_spectral:
    if cancer in CIMP_cancers:
        km_pval_spectral_lin_CIMP+=km_pval_spectral[cancer]
    km_pval_spectral_lin+=km_pval_spectral[cancer]

In [ ]:
# FDR correction with Benjamini Hochberg - for all cancer types and for only CIMP positive 
bool_p,pval,_,_ = multipletests(km_pval_spectral_lin,method='fdr_bh')
bool_p_CIMP,pval_CIMP,_,_ = multipletests(km_pval_spectral_lin_CIMP,method='fdr_bh')

In [ ]:
fdr_pvalues = {}
count=0
# get the associated fdr p values - corrected only for CIMP for CIMP positive, and fully corrected for CIMP negative
for cancer in cancer_to_keep:
    if cancer in triple_cluster_cancers_aff:
        fdr_pvalues[cancer] = pval[count:count+3]
        count+=3
    else:
        fdr_pvalues[cancer] = np.array([pval[count]])
        count+=1
count=0
for cancer in sorted(CIMP_cancers):
    if cancer in triple_cluster_cancers_aff:
        fdr_pvalues[cancer] = pval_CIMP[count:count+3]
        count+=3
    else:
        fdr_pvalues[cancer] = np.array([pval_CIMP[count]])
        count+=1

In [ ]:
lines, labels = flatax2[1].get_legend_handles_labels()
fig2.legend(lines, labels, loc = 'lower right', bbox_to_anchor = (1.75,0.25,1,1),
            bbox_transform = plt.gcf().transFigure,fontsize=20,frameon=False)
count=0
fignum1,fignum2,fignum3=0,0,0
for i,cancer in enumerate(cancer_to_keep):
    
    if cancer in sign_KM_cancers:
        fnum=fignum1
        flatax=flatax1
        fignum1+=1
    elif cancer in nsign_KM_cancers:
        fnum=fignum2
        flatax=flatax2
        fignum2+=1
    elif cancer in nCIMP_KM_cancers:
        fnum=fignum3
        flatax=flatax3
        fignum3+=1
        
    if cancer in ["GBM","LUAD","LUSC","MESO","SKCM","BLCA","ESCA","PAAD"]:
        x,y = 0.45,0.45
    elif cancer in ["LGG"]:
        x,y=0.45,0.7
    elif cancer in ["ACC","KIRP"]:
        x,y = 0.45,0.05
    else:
        x,y = 0.05,0.05
    
    if cancer in triple_cluster_cancers_aff:
        p1,p2,p3 = fdr_pvalues[cancer]
        text = 'L-I:{},\nL-H:{},\nI-H:{}'.format(scinum(p1,1),scinum(p2,1),scinum(p3,1)) 
        count+=3
    else:
        p1 = fdr_pvalues[cancer][0]
        text = 'L-H:{}'.format(scinum(p1,1))
        count+=1

    flatax[fnum].text(x, y, text, transform=flatax[fnum].transAxes,fontsize=15)
    flatax[fnum].set_xlabel("")

flatax2[-1].axis("off")


In [ ]:
fig1.text(0.5, -0.02, 'Days', ha='center')
fig1.text(-0.02, 0.5, 'OS', ha='center',fontsize=25,rotation=90)
fig1.tight_layout()
fig1

In [ ]:
fig2.text(0.5, -0.02, 'Days', ha='center')
fig2.text(-0.02, 0.5, 'OS', ha='center',fontsize=25,rotation=90)
fig2.tight_layout()
fig2

In [ ]:
fig3.text(0.5, -0.02, 'Days', ha='center')
fig3.text(-0.02, 0.5, 'OS', ha='center',fontsize=25,rotation=90)
flatax3[-1].axis("off")
fig3.tight_layout()
fig3

In [ ]:
fig1.savefig("KM_sign.png",bbox_inches="tight")
fig1.savefig("KM_sign.svg",bbox_inches="tight")
fig2.savefig("KM_nsign.png",bbox_inches="tight")
fig2.savefig("KM_nsign.svg",bbox_inches="tight")
fig3.savefig("KM_nCIMP.png",bbox_inches="tight")
fig3.savefig("KM_nCIMP.svg",bbox_inches="tight")

### Survival linked to IDH1 mutation

In [ ]:
from math import floor,log10
def scinum(num, decimal_digits=1, precision=None, exponent=None):
    """
    Returns a string representation of the scientific
    notation of the given number formatted for use with
    LaTeX or Mathtext, with specified number of significant
    decimal digits and precision (number of decimal digits
    to show). The exponent to be used can also be specified
    explicitly.
    """
    if exponent is None:
        exponent = int(floor(log10(abs(num))))
    coeff = round(num / float(10**exponent), decimal_digits)
    if precision is None:
        precision = decimal_digits

    return r"${0:.{2}f}\cdot10^{{{1:d}}}$".format(coeff, exponent, precision)

plt.rc("font",size=30)
    
fig, ax = plt.subplots(1,4,figsize=(20,5))
flatax = ax.flatten()

p_values = {}

seapal = sns.color_palette("bright")
color_pal = [seapal[0], seapal[3]]

for i,cancer in enumerate(["GBM","LAML","LGG","SKCM"]):
    print(cancer)
    duration, event = get_survival_analysis_metrics_snf(new_clin_data_pc, new_meth_data_pc, mut_pancan, cancer, "IDH1")
    
    if len(list(duration.keys()))>3:
        print("More than 3 meth clusters, not taken into account!")
        continue
    ## create a kmf object
    kmf = KaplanMeierFitter() 
    ## Fit the data into the model
    kmf.fit(duration[0], event[0], label='WT')

    ## Create an estimate
    kmf.plot(show_censors=True,c=color_pal[0],ax=flatax[i],ci_alpha=0.1)
    flatax[i].set_ylim([0,1])
    ## Fit the data into the model

    kmf.fit(duration[1], event[1], label='IDH1mut')
    ## Create an estimate
    kmf.plot(show_censors=True,c=color_pal[1],ax=flatax[i],ci_alpha=0.1)

    flatax[i].set_title(cancer,fontsize=30)
    flatax[i].legend().remove()
    flatax[i].set_xlabel("")
    print("Results between meth cluster 0 and 1")
    results=logrank_test(duration[0],duration[1],event_observed_A=event[0], event_observed_B=event[1])
    p_values[cancer] = [results.p_value]
    results.print_summary()
    if cancer=="GBM":
        flatax[i].text(1400,0.85,"p={}".format(scinum(results.p_value,1)),fontsize=23)
    elif cancer=="LGG":
        flatax[i].text(2000,0.85,"p={}".format(scinum(results.p_value,1)),fontsize=23)
    else:
        flatax[i].text(0,0.05,"p={}".format(scinum(results.p_value,1)),fontsize=23)

lines, labels = flatax[0].get_legend_handles_labels()
fig.legend(lines, labels, loc = 'lower right', bbox_to_anchor = (0.16,0.35,1,1),
            bbox_transform = plt.gcf().transFigure,fontsize=30,frameon=False)
fig.text(0.5, -0.02, 'Days', ha='center')

In [ ]:
fig.tight_layout()
fig.savefig("IDH1_survival_KM.png",bbox_inches="tight")
fig.savefig("IDH1_survival_KM.svg",bbox_inches="tight")
fig

### Cox model

In [ ]:
from sklearn.preprocessing import LabelBinarizer
# list of cancers with provided stage information in GDC
cancer_with_stage = ['ACC','BLCA','BRCA','CESC','ESCA','HNSC','KIRC','KIRP','LIHC','LUAD','LUSC',
 'MESO','PAAD','PRAD','SKCM','STAD','THCA','UCEC','COAD','READ']

In [ ]:
def fit_cox_model(clin_data_pc,consensus_clustering,cancer,cancer_with_stage):
#     Parameters: clin_data_pc (dict): keys are cancer types, values are pd.Df (n_patients,n_clin_info)
#             consensus_clustering (dict): keys are cancer types, values are pd.DataFrame (n_pat,1) (rows=patients, column=cluster membership)
#             cancer (str): cancer type
#             cancer_with_stage (list): list of cancers with stage information
#     Returns: cph (lifelines.CoxFitter): fitted regressor
#             augclin (pd.Df): clinical data formatted for the Cox Fitter

        augclin = clin_data_pc[cancer].merge(consensus_clustering[cancer],left_index=True,right_index=True)
        augclin = augclin.replace({'[Discrepancy]':0})
        augclin['age'] = -augclin['age']
        augclin['age'] = (augclin['age']-augclin['age'].mean())/augclin['age'].std()
        augclin = augclin.replace({'MALE':0, 'male': 0, 'FEMALE':1, 'female': 1,})
        if cancer in ["CESC","PRAD","UCEC","BRCA"]:
            augclin = augclin.drop(['gender'],axis=1)
        if cancer in ["LAML","PCPG","SARC","THYM"]:
            augclin = augclin.drop(['stage'],axis=1)
        
        if cancer in triple_cluster_cancers_aff:
            
            lb = LabelBinarizer()
            data = lb.fit_transform(augclin["cluster"].values.ravel())
            
            column_mapping={1: "Low meth cluster", 2: "inter meth cluster", 3: "high meth cluster"}
            col_labels = [column_mapping[lb.classes_[i]] for i in range(len(lb.classes_))]
            df = pd.DataFrame(data,columns=col_labels,index=augclin.index)
            df = df.drop(["Low meth cluster"],axis=1)
            augclin = pd.concat([augclin,df],axis=1)
            
        else:
            augclin["high meth cluster"]=augclin["cluster"].replace({1:0, 2:1})
    
        
        if cancer in cancer_with_stage:
            augclin = augclin.replace(replace_map)
            augclin = augclin.replace({'Stage I':1,'Stage II':2,'Stage III':3, 'Stage IV':4, "not reported": -1})
            
        
        cph = CoxPHFitter(penalizer=0.1)
        augclin = augclin.dropna(axis=0)
        cph.fit(augclin.drop("cluster",axis=1), duration_col='OS.time', event_col='OS')
        return cph,augclin   

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(20,6))
flatax = ax.flatten()
seapal = sns.color_palette("bright")
fignum=0

plt.rc('font', size=20)

cox_p_spectral = []
for cancer in sorted(CIMP_cancers):
    print(cancer)
    cph,augclin = fit_cox_model(new_clin_data_pc,filtered_consensus_clustering,cancer,cancer_with_stage)
    augclin_cl1 = augclin[augclin["cluster"]==1]
    if cancer in triple_cluster_cancers_aff:
        augclin_cl2 = augclin[augclin["cluster"]==3]
    else:
        augclin_cl2 = augclin[augclin["cluster"]==2]
    median_df_cl1 = cph.predict_median(augclin_cl1).sort_values()
    median_df_cl2 = cph.predict_median(augclin_cl2).sort_values()
    print("Median survival time cl1: {} vs cl2: {}".format(median_df_cl1.iloc[int(median_df_cl1.shape[0]/2)],
                                                          median_df_cl2.iloc[int(median_df_cl2.shape[0]/2)]))
    print(cph.summary)

    cox_p_spectral.append(cph.summary["p"]["high meth cluster"])
    if cancer in ["ACC","HNSC","KIRP","LGG","MESO"]:
        cph.plot(hazard_ratios=True,ax=flatax[fignum])
        flatax[fignum].set_title(cancer,fontsize=30)
        flatax[fignum].set_xlabel(flatax[fignum].get_xlabel(),fontsize=20)
        fignum+=1

flatax[-1].axis("off")
fig.tight_layout()

In [ ]:
fig.savefig("Cox_hazard_full.png",bbox_inches="tight")
fig.savefig("Cox_hazard_full.svg",bbox_inches="tight")

# Mutation analysis

### MSI Status

We compute the enrichment in MSI in the high methyalted clusters for COAD & UCEC

In [ ]:
seapal = sns.color_palette("bright")
size_plot = {"mss": 30, "msi-h": 300, "msi-l": 50, "indeterminate": 1, -1: 1}
for cancer in ["COAD","UCEC"]:
    print(cancer)
    
    # data taken from the Ciriano-Cortes Nature paper 
    red_msi = msi_profile[msi_profile["Cancer_type"]==cancer]
    ind_red_clust = [consensus_clustering_spectral[cancer].index[i][:-3] for i in range(consensus_clustering_spectral[cancer].shape[0])]
    red_clust = consensus_clustering_spectral[cancer].copy()
    red_clust.index = ind_red_clust
    red_clust = red_clust.loc[red_msi.index.intersection(ind_red_clust)]
    red_clust = red_clust[~red_clust.index.duplicated()]
    red_msi = red_msi.loc[red_msi.index.intersection(ind_red_clust)]
    red_msi = red_msi[~red_msi.index.duplicated()]
    msi_concat = pd.concat([red_msi,red_clust],axis=1)
    msi_concat.index = msi_concat.index+"-01"
    msi_concat = msi_concat.rename(columns={"MSI_category_nb_from_TCGA_consortium": "MSI status"})
    
    df = pd.concat([plot_df_spectral[plot_df_spectral["cancer"]==cancer],msi_concat],axis=1).fillna("indeterminate")
    
    # create the contingency table between MSI-H and MSS between low and high 
    dictcl1 = dict(Counter(df[df["Cluster"]=="Low meth cluster"]["MSI status"]))
    dictcl2 = dict(Counter(df[df["Cluster"]=="High meth cluster"]["MSI status"]))
    if not("msi-h" in dictcl1):
        dictcl1["msi-h"]=0
    if not("msi-h" in dictcl2):
        dictcl2["msi-h"]=0
    print(dictcl1,dictcl2)
    contingency = pd.DataFrame(np.zeros((2,2)),index=["msi-h","mss"],columns=["Low meth cluster", "High meth cluster"])
    for ms in contingency.index:
        contingency.loc[ms]["Low meth cluster"] = dictcl1[ms]
        contingency.loc[ms]["High meth cluster"] = dictcl2[ms]
    print(fisher_exact(contingency))
    
    edgecolor_list = df["MSI status"].replace({"mss": "white", "indeterminate": "grey", "msi-h": "black", "msi-l": "black"}).values
    linewidth_list = df["MSI status"].replace({"mss": 0, "indeterminate": 2, "msi-h": 1, "msi-l": 1}).values
    
    fig,ax = plt.subplots(1,1,figsize=(15,7))
    ax.set_title(cancer+" MSI status",fontsize=20)

    plt.rc('font', size=20)
    sns.scatterplot(data=df,x="Comp1",y="Comp2",hue="Cluster",size="MSI status",
                    sizes=size_plot,legend="auto", ax=ax,
                    palette={"Low meth cluster": seapal[0], "High meth cluster": seapal[3],"Intermediate meth cluster": seapal[8]},
                   edgecolor=edgecolor_list,linewidth=linewidth_list)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    plt.legend(loc='right', bbox_to_anchor=(1.4,0.5),frameon=False,fontsize=20)

    plt.savefig(cancer+"_MSI_status.png",dpi=100)
    plt.savefig(cancer+"_MSI_status.svg")

### IDH1/SETD2 mutation analysis total

We compute the continuous CIMP score as the average of beta values over all significantly differentially expressed positions for each cancer type

In [ ]:
CIMPness_dir = "path/to/CIMP/score"
CIMPness = {}
for cancer in tqdm(cancer_to_keep):
    CIMPness[cancer] = new_meth_data_pc[cancer][sign_pos_spectral[cancer].values.ravel()].T.mean().sort_values()
    CIMPness[cancer].to_csv(os.path.join(CIMPness_dir,cancer+"_CIMP_score.csv"))

In [ ]:
def get_list_patients_mut(mut_pancan, list_mut, cancer_to_keep):
#     Parameters: mut_pancan (dict): keys are cancer types, values are pd.Df (n_patients,n_genes_mut)
#             list_mut (list): list of mutations of interest - we compute the patients that have at least one of these
#             cancer_to_keep (list): list of cancers 
#     Returns: patients_pc (dict): keys are cancer types, values are pd.Index the list of patients that have at least 
#         one mutation in the genes of list_mut

    patients_pc = {}
    for cancer in cancer_to_keep:
        patients_pc[cancer] = mut_pancan[cancer][mut_pancan[cancer][list_mut].sum(axis=1)>0].index
    return patients_pc

In [ ]:
def return_rank_mut_patients(CIMPness, patients_mut_pc, cancer_to_keep):
#     Parameters: CIMPness (dict): keys are cancer types, values are pd.Df (n_patients,1) the CIMP score
#             patients_mut_pc (list): (dict): keys are cancer types, values are pd.Index the list of patients that have at least 
#         one mutation in the genes of list_mut
#             cancer_to_keep (list): list of cancers 
#     Returns: index_mut_pc (dict): keys are cancer types, values are list of ranks of patients with the mutations

    index_mut_pc = {}
    for cancer in cancer_to_keep:
        indname = "index" if cancer=="PCPG" else "Unnamed: 0" 
        locname = "level_0" if cancer=="PCPG" else "index"
        df = CIMPness[cancer].reset_index().reset_index().set_index(indname)
        common_pat = CIMPness[cancer].index.intersection(patients_mut_pc[cancer])
        index_mut_pc[cancer] = df.loc[common_pat][locname].ravel()
    return index_mut_pc

In [ ]:
def enrichement_mutation_CIMP(CIMPness, patients_mut_pc, cancer_to_keep, high_conf_patients):
#     Parameters: CIMPness (dict): keys are cancer types, values are pd.Df (n_patients,1) the CIMP score
#             patients_mut_pc (list): (dict): keys are cancer types, values are pd.Index the list of patients that have at least 
#         one mutation in the genes of list_mut
#             cancer_to_keep (list): list of cancers
#             high_conf_patients (dict): keys are cancer types, values are lists of patients with SSC>0
#     Returns: rs (dict): keys are cancer types, values are point biserial correlation coefficients
#             ps (dict): keys are cancer types, values are correlation associated p values

    rs,ps = {},{}
    for cancer in cancer_to_keep:
        df = CIMPness[cancer]
        mut_df = pd.DataFrame([0]*df.shape[0],index=df.index,columns=["mutated"])
        common_pat = CIMPness[cancer].index.intersection(patients_mut_pc[cancer])
        common_pat = common_pat.intersection(high_conf_patients[cancer])
        mut_df.loc[common_pat] = 1
        df = pd.concat([df,mut_df],axis=1)
        df.columns = ["CIMPness","mutated"]
        r,p = pointbiserialr(df.CIMPness.ravel(),df.mutated.ravel())
        rs[cancer] = r
        ps[cancer] = p
    return rs,ps

In [ ]:
def get_cancers_enriched_mut(CIMPness, patients_mut_pc, cancer_to_keep, high_conf_patients):
#     Parameters: CIMPness (dict): keys are cancer types, values are pd.Df (n_patients,1) the CIMP score
#             patients_mut_pc (list): (dict): keys are cancer types, values are pd.Index the list of patients that have at least 
#         one mutation in the genes of list_mut
#             cancer_to_keep (list): list of cancers
#             high_conf_patients (dict): keys are cancer types, values are lists of patients with SSC>0
#     Returns: enriched_info_pc (dict): keys are cancer types, values are dictionaries with 4 keys - p for 
#         the fdr corrected p value, r for the associated point biserial correlation coefficient, rank for the 
#         list of ranks of the patients with the mutation, size_gr for the number of patients with the mutation, size
#         for the total number of samples for the cancer type, perc for the percentage of patients in the cancer type 
#         presenting the mutation
    
    enriched_info_pc = {}
    
    mut_rs, mut_ps = enrichement_mutation_CIMP(CIMPness,patients_mut_pc,cancer_to_keep, high_conf_patients)
    corr_ps = multipletests(np.nan_to_num(list(mut_ps.values()),nan=1),method="fdr_bh")[1]
    cancer_enriched = cancer_to_keep[corr_ps<0.105]
    
    index_mut_pc = return_rank_mut_patients(CIMPness,patients_mut_pc,cancer_to_keep)
    
    for i,cancer in enumerate(cancer_to_keep):
        if cancer in cancer_enriched:
            enriched_info_pc[cancer] = {"p": corr_ps[i], "r": mut_rs[cancer], 
                                        "rank": index_mut_pc[cancer], "size_gr": index_mut_pc[cancer].shape[0], 
                                        "size": CIMPness[cancer].shape[0], 
                                        "perc": index_mut_pc[cancer].shape[0]/CIMPness[cancer].shape[0]*100}
    return enriched_info_pc

In [ ]:
patients_mut_IDH1 = get_list_patients_mut(mut_pancan,["IDH1",],cancer_to_keep)
patients_mut_SETD2 = get_list_patients_mut(mut_pancan,["SETD2"],cancer_to_keep)

In [ ]:
enriched_info_IDH1 = get_cancers_enriched_mut(CIMPness, patients_mut_IDH1, cancer_to_keep, high_conf_patients)
enriched_info_SETD2 = get_cancers_enriched_mut(CIMPness, patients_mut_SETD2, cancer_to_keep, high_conf_patients)

In [ ]:
enriched_cancers = np.unique(list(enriched_info_IDH1.keys()) + list(enriched_info_SETD2.keys()))

In [ ]:
fig, ax = plt.subplots(3,len(enriched_cancers)//3+1,figsize=(30,7))
flatax = ax.flatten()

for i,cancer in tqdm(enumerate(enriched_cancers)):
    common_pat = mut_pancan[cancer].index.intersection(CIMPness[cancer].index)
    common_pat = common_pat.intersection(high_conf_patients[cancer])
    df1 = mut_pancan[cancer].loc[common_pat][["IDH1"]].replace({0: "whitesmoke", 1: "blue"})
    df2 = mut_pancan[cancer].loc[common_pat][["SETD2"]].replace({0: "whitesmoke", 1: "red"})
    
    a = pd.concat([df1,df2],axis=1)
    plotdf = pd.concat([CIMPness[cancer],a],axis=1,join="inner").sort_values(0).reset_index().reset_index().set_index("index")

    plotdf.columns = ["x","CIMP score","IDH1","SETD2"]
    plotdf["x"] = plotdf["x"]*2
    plotdf = pd.concat([plotdf,pd.DataFrame([0.8]*plotdf.shape[0],index=plotdf.index,columns=["y1"])],axis=1)
    plotdf = pd.concat([plotdf,pd.DataFrame([1]*plotdf.shape[0],index=plotdf.index,columns=["y2"])],axis=1)
    
    sns.scatterplot(data = plotdf, x="x", y="y1", c=plotdf.IDH1.ravel(), alpha=1, linewidth=0.6, s=1800, ax=flatax[i], marker="|")
    sns.scatterplot(data = plotdf, x="x", y="y2", c=plotdf.SETD2.ravel(), alpha=1, linewidth=0.6, s=1800, ax=flatax[i], marker="|")
    if cancer in enriched_info_IDH1:
        r1 = f"{enriched_info_IDH1[cancer]['r']:.1f}"
        p1 = enriched_info_IDH1[cancer]["p"]
    else:
        r1 = 0
        p1 = 1
    if cancer in enriched_info_SETD2:
        r2 = f"{enriched_info_SETD2[cancer]['r']:.1f}"
        p2 = enriched_info_SETD2[cancer]["p"]
    else:
        r2 = 0
        p2 = 1
    r1stars = "(NS)" if p1>=0.105 else ("*" if 0.01<=p1<0.105 else ("**" if 0.001<=p1<0.01 else ("***" if 0.0001<=p1<0.001 else "****")))
    r2stars = "(NS)" if p2>=0.105 else ("*" if 0.01<=p2<0.105 else ("**" if 0.001<=p2<0.01 else ("***" if 0.0001<=p2<0.001 else "****")))
    flatax[i].set_title(f"{cancer} \n IDH1 r={r1}{r1stars}, SETD2 r={r2}{r2stars}",fontsize=15)
    flatax[i].axis("off")
flatax[-1].axis("off")
fig.savefig("IDH1_SETD2_enrichment.png",bbox_inches="tight")
fig.savefig("IDH1_SETD2_enrichment.svg",bbox_inches="tight")

### GOI mutations

We now analyze mutations in genes of interest (DNA & Histone methylation/demethylation genes) and compute associated p-values

In [ ]:
for cancer in tqdm(sorted(CIMP_cancers)):
    all_p_fisher = []
    print(cancer)
    common_pat_clustering = filtered_consensus_clustering[cancer].loc[filtered_consensus_clustering[cancer].index.intersection(mut_pancan[cancer].index)]
    ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
    if cancer in triple_cluster_cancers_aff:
        ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
    else:
        ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
    genes = all_genes_writer_Hmeth
    mutated_genes,mutated_index = [],[]
    for i,gene in enumerate(genes):
        n_mut_1 = (mut_pancan[cancer][gene].loc[ind_cl_1]>0).sum()
        n_mut_2 = (mut_pancan[cancer][gene].loc[ind_cl_2]>0).sum()
        perc_mut_1 = n_mut_1/len(ind_cl_1)*100
        perc_mut_2 = n_mut_2/len(ind_cl_2)*100
        p_fisher = fisher_exact(np.array([[n_mut_1,n_mut_2],[len(ind_cl_1)-n_mut_1,len(ind_cl_2)-n_mut_2]]))[1]
        all_p_fisher.append(p_fisher)
        if np.abs(perc_mut_1-perc_mut_2)>10:
            mutated_genes.append(gene)
            mutated_index.append(i)
            print(gene,perc_mut_1,perc_mut_2,p_fisher)
    print("percent of patients with at least one sign mut genes cl1 {} & cl2 {}".format((mut_pancan[cancer][mutated_genes].loc[ind_cl_1].T.sum()>0).sum()/len(ind_cl_1)*100,(mut_pancan[cancer][mutated_genes].loc[ind_cl_2].T.sum()>0).sum()/len(ind_cl_2)*100))
    if len(all_p_fisher)>0:
        print(multipletests(all_p_fisher,alpha=0.1,method="fdr_bh")[1][mutated_index])

In [ ]:
for cancer in tqdm(sorted(CIMP_cancers)):
    all_p_fisher = []
    print(cancer)
    common_pat_clustering = filtered_consensus_clustering[cancer].loc[filtered_consensus_clustering[cancer].index.intersection(mut_pancan[cancer].index)]
    ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
    if cancer in triple_cluster_cancers_aff:
        ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
    else:
        ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
    genes = all_genes_eraser_Hmeth
    mutated_genes,mutated_index = [],[]
    for i,gene in enumerate(genes):
        n_mut_1 = (mut_pancan[cancer][gene].loc[ind_cl_1]>0).sum()
        n_mut_2 = (mut_pancan[cancer][gene].loc[ind_cl_2]>0).sum()
        perc_mut_1 = n_mut_1/len(ind_cl_1)*100
        perc_mut_2 = n_mut_2/len(ind_cl_2)*100
        p_fisher = fisher_exact(np.array([[n_mut_1,n_mut_2],[len(ind_cl_1)-n_mut_1,len(ind_cl_2)-n_mut_2]]))[1]
        all_p_fisher.append(p_fisher)
        if np.abs(perc_mut_1-perc_mut_2)>10:
            mutated_genes.append(gene)
            mutated_index.append(i)
            print(gene,perc_mut_1,perc_mut_2,p_fisher)
    print("percent of patients with at least one sign mut genes cl1 {} & cl2 {}".format((mut_pancan[cancer][mutated_genes].loc[ind_cl_1].T.sum()>0).sum()/len(ind_cl_1)*100,(mut_pancan[cancer][mutated_genes].loc[ind_cl_2].T.sum()>0).sum()/len(ind_cl_2)*100))
    if len(all_p_fisher)>0:
        print(multipletests(all_p_fisher,alpha=0.1,method="fdr_bh")[1][mutated_index])

In [ ]:
for cancer in tqdm(sorted(CIMP_cancers)):
    all_p_fisher = []
    print(cancer)
    common_pat_clustering = filtered_consensus_clustering[cancer].loc[filtered_consensus_clustering[cancer].index.intersection(mut_pancan[cancer].index)]
    ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
    if cancer in triple_cluster_cancers_aff:
        ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
    else:
        ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
    genes = genes_demeth_gex
    mutated_genes,mutated_index = [],[]
    for i,gene in enumerate(genes):
        n_mut_1 = (mut_pancan[cancer][gene].loc[ind_cl_1]>0).sum()
        n_mut_2 = (mut_pancan[cancer][gene].loc[ind_cl_2]>0).sum()
        perc_mut_1 = n_mut_1/len(ind_cl_1)*100
        perc_mut_2 = n_mut_2/len(ind_cl_2)*100
        p_fisher = fisher_exact(np.array([[n_mut_1,n_mut_2],[len(ind_cl_1)-n_mut_1,len(ind_cl_2)-n_mut_2]]))[1]
        all_p_fisher.append(p_fisher)
        if np.abs(perc_mut_1-perc_mut_2)>10:
            mutated_genes.append(gene)
            mutated_index.append(i)
            print(gene,perc_mut_1,perc_mut_2,p_fisher)
    print("percent of patients with at least one sign mut genes cl1 {} & cl2 {}".format((mut_pancan[cancer][mutated_genes].loc[ind_cl_1].T.sum()>0).sum()/len(ind_cl_1)*100,(mut_pancan[cancer][mutated_genes].loc[ind_cl_2].T.sum()>0).sum()/len(ind_cl_2)*100))
    if len(all_p_fisher)>0:
        print(multipletests(all_p_fisher,alpha=0.1,method="fdr_bh")[1][mutated_index])

In [ ]:
for cancer in tqdm(sorted(CIMP_cancers)):
    all_p_fisher = []
    print(cancer)
    common_pat_clustering = filtered_consensus_clustering[cancer].loc[filtered_consensus_clustering[cancer].index.intersection(mut_pancan[cancer].index)]
    ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
    if cancer in triple_cluster_cancers_aff:
        ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
    else:
        ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
    genes = genes_meth_gex
    mutated_genes,mutated_index = [],[]
    for i,gene in enumerate(genes):
        n_mut_1 = (mut_pancan[cancer][gene].loc[ind_cl_1]>0).sum()
        n_mut_2 = (mut_pancan[cancer][gene].loc[ind_cl_2]>0).sum()
        perc_mut_1 = n_mut_1/len(ind_cl_1)*100
        perc_mut_2 = n_mut_2/len(ind_cl_2)*100
        p_fisher = fisher_exact(np.array([[n_mut_1,n_mut_2],[len(ind_cl_1)-n_mut_1,len(ind_cl_2)-n_mut_2]]))[1]
        all_p_fisher.append(p_fisher)
        if np.abs(perc_mut_1-perc_mut_2)>10:
            mutated_genes.append(gene)
            mutated_index.append(i)
            print(gene,perc_mut_1,perc_mut_2,p_fisher)
    print("percent of patients with at least one sign mut genes cl1 {} & cl2 {}".format((mut_pancan[cancer][mutated_genes].loc[ind_cl_1].T.sum()>0).sum()/len(ind_cl_1)*100,(mut_pancan[cancer][mutated_genes].loc[ind_cl_2].T.sum()>0).sum()/len(ind_cl_2)*100))
    if len(all_p_fisher)>0:
        print(multipletests(all_p_fisher,alpha=0.1,method="fdr_bh")[1][mutated_index])

### MGME analysis

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
# download gencode annotations
gencode_annot_gene = pd.read_csv("path\to\gencode_annotation_and_length")
gagval = gencode_annot_gene["gene_id"].values
gagval = [gagval[i].split(".")[0] for i in range(len(gagval))]
gencode_annot_gene["gene_id_simplf"] = gagval
uhrf1_pd = pd.DataFrame([[0,"UHRF1","ENSG00000276043.5",4000,"ENSG00000276043"]],columns=gencode_annot_gene.columns)
gencode_annot_gene = pd.concat([gencode_annot_gene,uhrf1_pd])

# download mapping between official gene notations and gencode
merged_table = pd.read_csv("path\to\gencode_mapping_table").set_index("gene_name")
merged_table.columns = ["gene_id_simplf","gencode_id_gex","gencode_id"]
uhrf1_pd = pd.DataFrame([["ENSG00000276043","ENSG00000276043.3","ENSG00000276043.5"]],index=["UHRF1"],columns=merged_table.columns)
merged_table = pd.concat([merged_table,uhrf1_pd])
merged_table = merged_table[~merged_table.index.duplicated(keep='first')]

In [ ]:
def get_coordinates_genes_cancer(gencode_annot_gene,gene_data_pc,mut_pancan,cancer):
    gex_cancer = gene_data_pc[cancer].mean()
    gex_cancer = gex_cancer.rename("gex_cancer")
    gex_cancer = pd.concat([gex_cancer,inv_merged_table.loc[gene_data_pc[cancer].columns.intersection(inv_merged_table.index)]["index"]],axis=1).dropna()
    gex_cancer.columns=["gex_cancer","gene_name"]
    gex_cancer = gex_cancer.reset_index()
    merged_df = gencode_annot_gene.merge(gex_cancer,how="inner",left_on="gene_name",right_on="gene_name")[["gene_name","length","gex_cancer"]]
    merged_df = merged_df.set_index("gene_name")
    merged_df.columns=["length","gex_cancer"]
    merged_df = merged_df[~merged_df.index.duplicated()]
    merged_df = merged_df.loc[mut_pancan[cancer].columns.intersection(merged_df.index)]
    std = StandardScaler()
    coordinates = pd.DataFrame(std.fit_transform(merged_df[["length","gex_cancer"]].values),index=merged_df.index,columns=["x","y"])
    return coordinates

In [ ]:
def get_mutational_burden_comparison(list_cancer,df_louvain_snf,mut_pancan,genes_of_interest):
    mut_burden,goi_mut_burden = [],[]
    
    for cancer in list_cancer:

        ind_cl_1 = df_louvain_snf[cancer][df_louvain_snf[cancer]["cluster"]==1].index
        ind_cl_2 = df_louvain_snf[cancer][df_louvain_snf[cancer]["cluster"]==2].index
        ind_cl_3 = df_louvain_snf[cancer][df_louvain_snf[cancer]["cluster"]==3].index
        if cancer in triple_cluster_cancers_aff:
            ind_cl_2 = ind_cl_3
        
        ind_cl_1 = mut_pancan[cancer].index.intersection(ind_cl_1)
        ind_cl_2 = mut_pancan[cancer].index.intersection(ind_cl_2)
        
        mut_burden.append([mut_pancan[cancer].loc[ind_cl_1].T.sum().sum()+1, \
                            mut_pancan[cancer].loc[ind_cl_2].T.sum().sum()+1])
        if len(genes_of_interest)==1:
            goi_mut_burden.append([int(mut_pancan[cancer][genes_of_interest[0]].loc[ind_cl_1].sum())+1, \
                            int(mut_pancan[cancer][genes_of_interest[0]].loc[ind_cl_2].sum())+1])
        else:
            goi_mut_burden.append([mut_pancan[cancer][genes_of_interest].loc[ind_cl_1].T.sum().sum()+1, \
                            mut_pancan[cancer][genes_of_interest].loc[ind_cl_2].T.sum().sum()+1])
    mut_burden = pd.DataFrame(mut_burden,columns=["Low meth cluster","High meth cluster"],index=list_cancer)
    goi_mut_burden = pd.DataFrame(goi_mut_burden,columns=["Low meth cluster","High meth cluster"],index=list_cancer)
    return mut_burden,goi_mut_burden

In [ ]:
def get_df_mut_comp_pergene(list_cancer,df_louvain_snf,mut_pancan,genes_of_interest):
    df_mut_comp_pergene = []
    
    for gene in tqdm(genes_of_interest):
       
        mut_burden, goi_mut_burden = get_mutational_burden_comparison(list_cancer,df_louvain_snf,mut_pancan,[gene])
        mut_burden_comp = pd.concat([goi_mut_burden["Low meth cluster"]/mut_burden["Low meth cluster"]*10000,
                                      goi_mut_burden["High meth cluster"]/mut_burden["High meth cluster"]*10000],axis=1)
        comp_mut_burden = mut_burden_comp["High meth cluster"]/mut_burden_comp["Low meth cluster"]
        if len(df_mut_comp_pergene)==0:
            df_mut_comp_pergene = comp_mut_burden
        else:
            df_mut_comp_pergene = pd.concat([df_mut_comp_pergene,comp_mut_burden],axis=1)
    df_mut_comp_pergene.columns = genes_of_interest
    return df_mut_comp_pergene

In [ ]:
def find_1NN_genes(genes_of_interest,coordinates_val_lasso_pc,cancer,n_neighbors):
    gene_list = []
    neigh = NearestNeighbors(n_neighbors=n_neighbors)
    neigh.fit(coordinates_val_lasso_pc[cancer].values)
    coord_gene = coordinates_val_lasso_pc[cancer].loc[genes_of_interest].values
    neighbor_inds = neigh.kneighbors(coord_gene,return_distance=False)
    for i in range(neighbor_inds.shape[0]):
        neigh_ind = neighbor_inds[i,np.random.randint(n_neighbors)]
        gene_list.append(coordinates_val_lasso_pc[cancer].index[neigh_ind])
    return gene_list

In [ ]:
def get_bootstrap_comp_mut_burden(coordinates_val_lasso_pc,n_neighbors,list_cancer,
                                  df_louvain_snf,mut_pancan,genes_of_interest):
    mut_burden = []
    for cancer in list_cancer:
        ind_cl_1 = df_louvain_snf[cancer][df_louvain_snf[cancer]["cluster"]==1].index
        ind_cl_2 = df_louvain_snf[cancer][df_louvain_snf[cancer]["cluster"]==2].index
        ind_cl_3 = df_louvain_snf[cancer][df_louvain_snf[cancer]["cluster"]==3].index
        if cancer in triple_cluster_cancers_aff:
            ind_cl_2 = ind_cl_3
        ind_cl_1 = mut_pancan[cancer].index.intersection(ind_cl_1)
        ind_cl_2 = mut_pancan[cancer].index.intersection(ind_cl_2)
        
        mut_burden.append([mut_pancan[cancer].loc[ind_cl_1].T.sum().sum()+1, \
                            mut_pancan[cancer].loc[ind_cl_2].T.sum().sum()+1])
    mut_burden = pd.DataFrame(mut_burden,columns=["Low meth cluster","High meth cluster"],index=list_cancer)
    print("Done calculating mutational burden")
    
    mut_bootstrap = []
    for i in tqdm(range(500)):
        
        genes_bootstrap = find_1NN_genes(genes_of_interest,coordinates_val_lasso_pc,cancer,n_neighbors)
        
        goi_mut_burden = []
        for cancer in list_cancer:
            ind_cl_1 = df_louvain_snf[cancer][df_louvain_snf[cancer]["cluster"]==1].index
            ind_cl_2 = df_louvain_snf[cancer][df_louvain_snf[cancer]["cluster"]==2].index
            ind_cl_3 = df_louvain_snf[cancer][df_louvain_snf[cancer]["cluster"]==3].index
            if cancer in triple_cluster_cancers_aff:
                ind_cl_2 = ind_cl_2
            ind_cl_1 = mut_pancan[cancer].index.intersection(ind_cl_1)
            ind_cl_2 = mut_pancan[cancer].index.intersection(ind_cl_2)  
                
            if len(genes_of_interest)==1:
                goi_mut_burden.append([int(mut_pancan[cancer][genes_bootstrap[0]].loc[ind_cl_1].sum())+1, \
                                int(mut_pancan[cancer][genes_bootstrap[0]].loc[ind_cl_2].sum())+1])
            else:
                goi_mut_burden.append([mut_pancan[cancer][genes_bootstrap].loc[ind_cl_1].T.sum().sum()+1, \
                                mut_pancan[cancer][genes_bootstrap].loc[ind_cl_2].T.sum().sum()+1])
        goi_mut_burden = pd.DataFrame(goi_mut_burden,columns=["Low meth cluster","High meth cluster"],index=list_cancer)
        
        mut_burden_comp = pd.concat([goi_mut_burden["Low meth cluster"]/mut_burden["Low meth cluster"]*10000,
                                  goi_mut_burden["High meth cluster"]/mut_burden["High meth cluster"]*10000],axis=1)
        comp_mut_burden = mut_burden_comp["High meth cluster"]/mut_burden_comp["Low meth cluster"]
        mut_bootstrap.append(comp_mut_burden.values)
    
    return mut_bootstrap

In [ ]:
def get_empirical_p_value_mutations(mut_bootstrap,comp_mut_burden,meth_or_demeth,list_cancer):
    p_value = {cancer: 0 for cancer in list_cancer}
    comp_burd = comp_mut_burden[meth_or_demeth]
    for i in tqdm(range(len(mut_bootstrap))):
        for j in range(len(mut_bootstrap[i])):
            if comp_burd.iloc[j]>1 and mut_bootstrap[i][j]>comp_burd.iloc[j]:
                p_value[list_cancer[j]]+=1/1000
            elif comp_burd.iloc[j]<1 and mut_bootstrap[i][j]<comp_burd.iloc[j]:
                p_value[list_cancer[j]]+=1/1000
    return p_value

In [ ]:
COAD_gene = {"COAD": pd.read_csv("path\to\gene_fpkm\COADREAD_gene_fpkm.csv")
        }

In [ ]:
COAD_gene["COAD"]["Unnamed: 0"] = COAD_gene["COAD"]["Unnamed: 0"].apply(lambda x: x[:15])
COAD_gene["COAD"] = COAD_gene["COAD"].set_index("Unnamed: 0")
COAD_gene["COAD"] = COAD_gene["COAD"][~COAD_gene["COAD"].index.duplicated()]
COAD_gene["COAD"] = COAD_gene["COAD"].loc[consensus_clustering_spectral["COAD"].index.intersection(COAD_gene["COAD"].index)]

In [ ]:
coordinates_val_lasso_pc = {}
for cancer in ["COAD"]:
    coordinates_val_lasso_pc[cancer] = get_coordinates_genes_cancer(gencode_annot_gene,COAD_gene,mut_pancan,cancer)

In [ ]:
df_mut_comp_pergene_meth = get_df_mut_comp_pergene(["COAD"],consensus_clustering_spectral,mut_pancan,mutations_to_test["COAD"])

In [ ]:
df_mut_comp_pergene_meth

In [ ]:
df_p_value_pergene_meth = []
for gene in tqdm(mutations_to_test["COAD"]):
    mut_bootstrap_pergene_meth = get_bootstrap_comp_mut_burden(coordinates_val_lasso_pc,1000,["COAD"],
                                  consensus_clustering_spectral,mut_pancan,[gene])
    p_value_mut_pergene = get_empirical_p_value_mutations(mut_bootstrap_pergene_meth,df_mut_comp_pergene_meth,gene,["COAD"])
    if len(df_p_value_pergene_meth)==0:
        df_p_value_pergene_meth = pd.DataFrame.from_dict(p_value_mut_pergene, orient='index',columns=[gene])
    else:
        df_p_value_pergene_meth = pd.concat([df_p_value_pergene_meth,pd.DataFrame.from_dict(p_value_mut_pergene, orient='index',columns=[gene])],axis=1)


In [ ]:
len(mutations_to_test["COAD"])

In [ ]:
mask_multipletest_meth = []
for cancer in ["COAD"]:
    multtest = multipletests(df_p_value_pergene_meth.loc[cancer],method='fdr_bh')
    mask_multipletest_meth.append(multtest[0])
    for i,gene in enumerate(mutations_to_test["COAD"]):
        print(gene,multtest[0][i],multtest[1][i])

# RF for feature selection

We train Random Forest (RF) classifiers on the entire mutational information. We keep RF that perform better than random (ABAC > 0 on 10-fold CV) and have persistency of features over 10 folds. 

In [ ]:
def common_features(lol):
    # common features in list of lists
    return list(set(lol[0]).intersection(*lol[1:]))

In [ ]:
def plot_important_features_RF(mut_or_gene_pc,cancer,consensus_clustering,n_splits):
#     Parameters: mut_or_gene_pc (dict): keys are cancer types, values are pd.Df with mutational or gene expression info (n_patients,n_genes or n_mut)
#             cancer (str): cancer type
#             consensus_clustering (dict): keys are cancer types, values are pd.Df (n_patients,1) (rows=patients,columns=cluster membership)
#             n_splits (int): number of CV splits
#     Returns: best_estimator (sklearn RandomForestClassifier): trained RF classifier with best score
#             common_features (list): list of positions of best features per cancer
    RF_scores = {}
    
    X = mut_or_gene_pc[cancer]
    X = X.replace({0: np.nan}).dropna(axis=1,how="all").replace({np.nan: 0})
    y = consensus_clustering[cancer]
    common_pat = X.index.intersection(y.index)
    X = X.loc[common_pat]
    y = y.loc[common_pat].values.ravel()
    
    outer_cv = StratifiedKFold(n_splits=n_splits, shuffle=True,random_state=33)
    bac_scores, estimators, sel_10_feat = [],[],[]
    for train_index,test_index in outer_cv.split(X,y):
        
        X_train, X_test, y_train, y_test = X.iloc[train_index],X.iloc[test_index], \
                                            y[train_index],y[test_index]

        sel = SelectFromModel(RandomForestClassifier(n_estimators = 500,class_weight="balanced"))
        sel.fit(X_train, y_train)
        y_pred = sel.estimator_.predict(X_test)
        bac_scores.append(balanced_accuracy_score(y_test,y_pred,adjusted=True))
        estimators.append(sel.estimator_)
        sel_10_feat.append(np.argsort(sel.estimator_.feature_importances_)[::-1][:10])
    
    print("{}-fold BAC on test set of {}: {}".format(n_splits,cancer,np.mean(bac_scores)))
    print("Common 10 best features selected for {} folds : {}".format(n_splits,X.columns[common_features(sel_10_feat)]))
    best_estimator = estimators[np.argmax(bac_scores)]
    RF_score = np.max(bac_scores)
    #Define size of bar plot
    plt.figure(figsize=(10,8))
    #Plot Searborn bar chart
    sns.barplot(x=best_estimator.feature_importances_[np.argsort(best_estimator.feature_importances_)[::-1][:10]],
                y=X_train.columns[np.argsort(best_estimator.feature_importances_)[::-1][:10]])
    #Add chart labels
    plt.title('Feature importance of best estimator for '+cancer)
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    return best_estimator,common_features(sel_10_feat),RF_score

In [ ]:
common_features_pc = {cancer: [] for cancer in sorted(CIMP_cancers)}
RF_models = {cancer: [] for cancer in sorted(CIMP_cancers)}
RF_scores = {cancer: [] for cancer in sorted(CIMP_cancers)}

for cancer in tqdm(sorted(CIMP_cancers)):
    RF_models[cancer],common_features_pc[cancer],RF_scores[cancer] = plot_important_features_RF(mut_pancan,cancer,consensus_clustering_spectral,10)

In [ ]:
# Scores of RF
cancer_RF_selected = [cancer for (cancer,val) in RF_scores.items() if val>0]
cancer_RF_selected = [cancer for cancer in cancer_RF_selected if len(common_features_pc[cancer])>0]
val_RF_selected = [RF_scores[cancer] for cancer in cancer_RF_selected]
np.mean(val_RF_selected),np.quantile(val_RF_selected,0.25), np.quantile(val_RF_selected,0.75)

In [ ]:
plt.rc('font', size=15)
# to create plots for cancer types that fit the selection criteria
for cancer in tqdm(cancer_RF_selected+["STAD"]):   
    best_estimator = RF_models[cancer]
    X = mut_pancan[cancer]
    # remove columns with all 0
    X = X.replace({0: np.nan}).dropna(axis=1,how="all").replace({np.nan: 0})
    y = consensus_clustering_spectral[cancer]
    common_pat = X.index.intersection(y.index)
    X = X.loc[common_pat]
    y = y.loc[common_pat].values.ravel()
    # select only the features persistently selected
    mut_exp = X.iloc[:,common_features_pc[cancer]]
    common_pat_clustering = consensus_clustering_spectral[cancer].loc[consensus_clustering_spectral[cancer].index.intersection(mut_pancan[cancer].index)]
    ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
    ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
    ind_cl_3 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
    if cancer in triple_cluster_cancers_aff:
        ind_cl_2 = ind_cl_3
    cl1_gex = mut_exp.loc[ind_cl_1]
    cl2_gex = mut_exp.loc[ind_cl_2]
    perc_cl1 = cl1_gex.sum()/len(ind_cl_1)*100
    perc_cl2 = cl2_gex.sum()/len(ind_cl_2)*100
    
    # check if the enrichment is in the low or high methylated meth cluster
    diff_mask = perc_cl1<perc_cl2
    fisher_p = []
    # check significance of enrichment through corrected Fisher p
    for gene in cl1_gex.columns:
        fisher_p.append(fisher_exact(np.array([[cl1_gex[gene].sum(),cl2_gex[gene].sum()],[cl1_gex.shape[0]-cl1_gex[gene].sum(),cl2_gex.shape[0]-cl2_gex[gene].sum()]]))[1])
    bh_p = multipletests(fisher_p,alpha=0.1)[1]
    
    # seaborn representation
    df = pd.DataFrame(np.transpose([best_estimator.feature_importances_[np.argsort(best_estimator.feature_importances_)[::-1][:10]],
                           list(X.columns[np.argsort(best_estimator.feature_importances_)[::-1][:10]])]),
             columns=["feature_imp","gene"])
    df = df.set_index("gene")
    df = pd.concat([df,diff_mask],axis=1).replace({False: "Low meth cluster", True: "High meth cluster", np.nan: "NA"})
    df = pd.concat([df,pd.DataFrame(bh_p,index=cl1_gex.columns,columns=["p"])],axis=1).reset_index()
    df.columns = ["gene","Features importance","Cluster enriched","BH p"]
    df["Features importance"] = df["Features importance"].astype(float)
    plt.figure(figsize=(7,5))
    seapal = sns.color_palette("bright")
    
    ax = sns.barplot(data=df,x="Features importance",y="gene",hue="Cluster enriched",dodge=False,palette={"Low meth cluster": seapal[0], "High meth cluster": seapal[3], "NA": "grey"})
    plt.legend(frameon=False)
    feat_max = np.max(df["Features importance"].values.ravel())
    for i in range(df["gene"].shape[0]):
        feature = df["gene"].iloc[i]
        pos = df["Features importance"].iloc[i]
        sign = "" if df["BH p"].iloc[i]<0.11 else " (NS)"
        if feature in list(perc_cl1.index):
            ax.text(1.01*pos,i+0.2,str(np.round(perc_cl1.loc[feature],0))+"%,"+str(np.round(perc_cl2.loc[feature],0))+"%"+sign)
    plt.title('Feature importance of best estimator for '+cancer)
    ax.set_xlim([0,1.6*feat_max])
    ax.set_xticklabels([])
    plt.xlabel('Feature importance')
    plt.ylabel('')
    ax.figure.savefig("RF_"+cancer+".svg",bbox_inches="tight")

#### Visual representation

In [ ]:
genes_rf = {"TP53": 5, "KRAS": 1, 
            "PIK3CA": 1, "PTEN": 1, "NF1": 1, 
            "IDH1": 2, "SETD2": 2, "NSD1": 1,
            "ATRX": 2, "ARID1A": 1, "PBRM1": 1,
            "BAP1": 2,  
            "CTNNB1": 2, "ALB": 1, "BRAF": 1, "CIC": 1, "EGFR": 1, "FUBP1": 1, "LATS2": 1, "CYP8B1": 1, "CDH13": 1, "MUC16": 1}

genes_rf_pd = pd.DataFrame.from_dict(genes_rf, orient="index").reset_index()
genes_rf_pd.columns = ["gene","count"]

In [ ]:
seapal = sns.color_palette("bright")
plt.rc("font", size=20)
fig,ax = plt.subplots(1,1,figsize=(15,10))
sns.barplot(data=genes_rf_pd,x="count",y="gene",
            palette=[seapal[0]]*2+[seapal[1]]*3+[seapal[2]]*3+[seapal[3]]*3+[seapal[4]]+[seapal[8]]*10)
ax.set_xlabel("Number of cancer types")
ax.set_ylabel("")
ax.text(3, 1.5, "RAS/TP53 pathway", color=seapal[0], fontsize=25)
ax.text(3, 3.5, "PTEN/PIK3 pathway", color=seapal[1], fontsize=25)
ax.text(3, 5.5, "Methylation/demethylation", color=seapal[2], fontsize=25)
ax.text(3, 8.8, "SWI/SNF complex", color=seapal[3], fontsize=25)
ax.text(3, 11.3, "Other chromatin remodeling", color=seapal[4], fontsize=25)
ax.text(3, 16.5, "Other", color=seapal[8], fontsize=25)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)

fig.savefig("RF_selected_genes_viz.png",bbox_inches="tight")
fig.savefig("RF_selected_genes_viz.svg",bbox_inches="tight")

#### Visualization

In [ ]:
david_path = {"Nervous system development/Neurogenesis": 7,  "Pattern specification": 7,
              "Cell-cell signaling": 7, "Cell differentiation/fate commitment": 7, "Cell adhesion": 5, 
              "Cell proliferation": 3, "Cation transport": 3}

david_path_pd = pd.DataFrame.from_dict(david_path, orient="index").reset_index()
david_path_pd.columns = ["Pathway","count"]

In [ ]:
seapal = sns.color_palette("bright")
plt.rc("font", size=35)
fig,ax = plt.subplots(1,1,figsize=(15,10))
sns.barplot(data=david_path_pd,x="count",y="Pathway")
ax.set_xlabel("Number of cancer types")
ax.set_ylabel("")

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)

fig.savefig("DAVID_viz.png",bbox_inches="tight")
fig.savefig("DAVID_viz.svg",bbox_inches="tight")

# CIMP score vs mutation

In [ ]:
# these are the mutations that were significantly enriched either in the GOI (meth/demeth) or using the RF classifiers
mutations_to_test = {"ACC": ["MUC16"],
                     "BLCA": ["KMT2C",'KDM6A', 'FGFR3', 'TP53', 'ERBB2', 'RB1', 'STAG2'],
                    "BRCA": ['TP53', 'CDH1', 'GATA3', 'PIK3CA'],
                    "CESC": ["KMT2D"],
                    "COAD": ["ASH1L","EHMT1", "EHMT2", "KMT2A","KMT2B","KMT2C","KMT2D","MECOM","NSD1","PRDM1","PRDM2","PRDM9","PRDM10","PRDM13","PRDM14","PRDM15","PRDM16","SETD1A","SETD1B","SETD2","SETDB1","BAZ2A","DNMT1","UHRF1BP1L","TET1","TET2","TET3","MBD1","APOBEC1","CTCF","KDM2B","KDM3B","KDM4A","KDM4B","KDM4C","KDM5A","KDM5B","KDM5C","KDM6A","KDM6B","PHF2","BRAF","KRAS"],
                    "ESCA": ["LAMA1","NFE2L2"],
                    "GBM": ["IDH1","TP53","ATRX"],
                    "HNSC": ["NSD1","PRDM9"],
                     "KIRC": ["SETD2","PBRM1"],
                     "KIRP": [],
                    "LAML": ["IDH2","IDH1",],
                    "LGG": ["IDH1","ATRX","EGFR","CIC","FUBP1","TP53","NF1","PTEN"],
                     "LIHC": ['TP53', 'CTNNB1', 'BAP1', 'ALB'],
                     "LUAD": [],
                     "LUSC": ["NSD1","CYP8B1","SETD1A"],
                    "MESO": ["KMT2B", "SETD2", "BAP1","LATS2"],
                     "PAAD": ['TP53', 'RNF43', 'CDKN2A', 'TTN', 'KRAS', 'SMAD4'],
                     "PCPG": [],
                     "PRAD": ["TTN","SPOP","FOXA1","TP53"],
                     "READ": ["CDH13"],
                     "SARC": [],
                    "SKCM": ["IDH1"],
                    "STAD": ['ARID1A', 'TP53', 'CTNNB1', 'PIK3CA'],
                    "THYM": [],
                     "THCA": [],
                    "UCEC": ["ARID1A","ASH1L","DOT1L", "EHMT1", "EHMT2", "EZH1", "EZH2","KMT2A","KMT2B","KMT2C","KMT2D","NSD1","PRDM2","PRDM4","PRDM5","PRDM9","PRDM10","PRDM11","PRDM14","PRDM15","PRDM16","SETD1B","SETD2","SETDB1","SMYD1","SUV39H1","SUV420H1","WHSC1L1","BAZ2A","CTCFL","DNMT1","DNMT3A","DNMT3B","UHRF1BP1L","TET1","TET2","TET3","TDG","MBD1","MBD4","CTCF","IDH1","IDH2","JMJD1C","KDM2A","KDM2B","KDM3A","KDM3B","KDM4A","KDM4B","KDM4C","KDM5A","KDM5B","KDM5C","KDM6A","KDM6B","PHF2","PHF8","PTEN","TP53","CTNNB1","KRAS","PPP2R1A"]}

In [ ]:
mutations_to_test_CIMP = {cancer: mutations_to_test[cancer] for cancer in CIMP_cancers}
mutations_to_test_nCIMP = {cancer: mutations_to_test[cancer] for cancer in nCIMP_KM_cancers}

In [ ]:
# to test per gene and not per cancer
invert_mutations_to_test = {}
for key, val in mutations_to_test_CIMP.items():
    for mut in val:
        if mut in list(invert_mutations_to_test.keys()):
            invert_mutations_to_test[mut].append(key)
        else:
            invert_mutations_to_test[mut] = [key]

In [ ]:
fig, axs = plt.subplots(12, 9, figsize=(35,20))
for a in axs.flatten():
    a.axis("off")

In [ ]:
# get the FDR corrected p values
mutation_ps = []
for j,mut in tqdm(enumerate(sorted(invert_mutations_to_test))):
    for i,cancer in enumerate(invert_mutations_to_test[mut]):
        common_pat = mut_pancan[cancer].index.intersection(CIMPness[cancer].index)
        common_pat = common_pat.intersection(high_conf_patients[cancer])
        
        y = mut_pancan[cancer].loc[common_pat][mut].ravel()
        X = CIMPness[cancer].loc[common_pat].values.ravel()
        
        corr,p = pointbiserialr(y,X)
        mutation_ps.append(p)
        
corrected_ps = multipletests(mutation_ps,method="fdr_bh")[1]

In [ ]:
# compute the point biserial correlation between the CIMP score and a mutation
flatax = axs.flatten()
fignum = 0
for j,mut in tqdm(enumerate(sorted(invert_mutations_to_test))):
    for i,cancer in enumerate(invert_mutations_to_test[mut]):
        common_pat = mut_pancan[cancer].index.intersection(CIMPness[cancer].index)
        common_pat = common_pat.intersection(high_conf_patients[cancer])
        df1 = mut_pancan[cancer].loc[common_pat][[mut]]
        

        plotdf = pd.concat([CIMPness[cancer],df1],axis=1,join="inner").sort_values(0).reset_index().reset_index().set_index("index")
        plotdf.columns = ["x","CIMP score","mutation"]
        
        X = plotdf["CIMP score"].ravel()
        y = plotdf["mutation"].ravel()
        
        plotdf["mutation"] = plotdf["mutation"].replace({0: "whitesmoke", 1: "black"})
        
        corr,_ = pointbiserialr(y,X)
        p = corrected_ps[fignum]
        print(cancer,mut,corr,p)
        plotdf["x"] = plotdf["x"]*2
        plotdf = pd.concat([plotdf,pd.DataFrame([1]*plotdf.shape[0],index=plotdf.index,columns=["y"])],axis=1)
        
        sns.scatterplot(data = plotdf, x="x", y="y", c=plotdf.mutation.ravel(), alpha=1, linewidth=0.6, s=1800, ax=flatax[fignum], marker="|")
            
        rstars = "(NS)" if p>=0.1 else ("*" if 0.01<=p<0.1 else ("**" if 0.001<=p<0.01 else ("***" if 0.0001<=p<0.01 else "****")))
            
        flatax[fignum].set_title(mut+" in "+cancer+"\n r="+str(round(corr,2))+rstars,fontsize=25)
        fignum+=1

In [ ]:
ax = flatax[-1]
ax.axis("on")
ax.set_xlabel("CIMP score",fontsize=25)
ax.set_xticks([0,1])
ax.set_xticklabels(["Min score","Max score"],rotation=0,rotation_mode="default")
ax.set_ylabel("Mut",fontsize=25)
ax.set_yticks([])
fig.tight_layout()

In [ ]:
fig.savefig("mutation_vs_CIMP_score.png",bbox_inches="tight")
fig.savefig("mutation_vs_CIMP_score.svg",bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(8, 3, figsize=(10,10))
for a in axs.flatten():
    a.axis("off")

In [ ]:
# identical as before but only high corr
flatax = axs.flatten()
fignum = 0
p_count = 0
for j,mut in tqdm(enumerate(sorted(invert_mutations_to_test))):
    for i,cancer in enumerate(invert_mutations_to_test[mut]):
        common_pat = mut_pancan[cancer].index.intersection(CIMPness[cancer].index)
        common_pat = common_pat.intersection(high_conf_patients[cancer])
        df1 = mut_pancan[cancer].loc[common_pat][[mut]]
        

        plotdf = pd.concat([CIMPness[cancer],df1],axis=1,join="inner").sort_values(0).reset_index().reset_index().set_index("index")
        plotdf.columns = ["x","CIMP score","mutation"]
        
        X = plotdf["CIMP score"].ravel()
        y = plotdf["mutation"].ravel()
        
        plotdf["mutation"] = plotdf["mutation"].replace({0: "whitesmoke", 1: "black"})
        
        corr,_ = pointbiserialr(y,X)
        p = corrected_ps[p_count]
        p_count+=1
        if corr>0.25:
            plotdf["x"] = plotdf["x"]*2
            plotdf = pd.concat([plotdf,pd.DataFrame([1]*plotdf.shape[0],index=plotdf.index,columns=["y"])],axis=1)

            sns.scatterplot(data = plotdf, x="x", y="y", c=plotdf.mutation.ravel(), alpha=1, linewidth=0.6, s=1800, ax=flatax[fignum], marker="|")

            rstars = "(NS)" if p>=0.1 else ("*" if 0.01<=p<0.1 else ("**" if 0.001<=p<0.01 else ("***" if 0.0001<=p<0.01 else "****")))

            flatax[fignum].set_title(mut+" in "+cancer+"\n r="+str(round(corr,2))+rstars,fontsize=15)
            fignum+=1

In [ ]:
ax = flatax[-1]
ax.axis("on")
ax.set_xlabel("CIMP score")
ax.set_xticks([0,1])
ax.set_xticklabels(["Min score","Max score"],rotation=0,rotation_mode="default")
ax.set_ylabel("Mut")
ax.set_yticks([])
fig.tight_layout()
fig.tight_layout()

In [ ]:
fig.savefig("mutation_vs_CIMP_score_highcorr.png",bbox_inches="tight")
fig.savefig("mutation_vs_CIMP_score_highcorr.svg",bbox_inches="tight")

In [ ]:
gene_cancer_mapping = {"IDH1": ["GBM","LAML", "LGG","SKCM"], "SETD2": ["COAD","KIRC","MESO"]}
nsd1_mapping = ["COAD","HNSC","LUSC"]

In [ ]:
plt.rc("font",size=20)
seapal = sns.color_palette("bright")
fig, ax = plt.subplots(3,3,figsize=(10,7))
flatax = ax.flatten()
fignum=0
size_plot = {"mut": 40, "non-mut": 10}

for gene in gene_cancer_mapping.keys():
    for cancer in gene_cancer_mapping[gene]:
        if fignum==4:
            fignum+=2
        df = pd.concat([df_umap_spectral[cancer],mut_pancan[cancer][gene].replace({0: "non-mut",1: "mut"})],axis=1,join="inner")
        df.columns = np.append(df_umap_spectral[cancer].columns,["Mutation"])
        edgecolor_list = df["Mutation"].replace({"non-mut": "white", "mut": "black"}).values
        linewidth_list = df["Mutation"].replace({"non-mut": 0, "mut": 1}).values
          
        sns.scatterplot(data=df,x="Comp1",y="Comp2",hue="Cluster",size="Mutation",
                        sizes=size_plot,ax=flatax[fignum],legend="auto",
                        palette={"Low meth cluster": seapal[0], "High meth cluster": seapal[3],"Intermediate meth cluster": seapal[8]},
                       edgecolor=edgecolor_list,linewidth=linewidth_list)
        if fignum in range(4):
            flatax[fignum].set_title(cancer+" (1)",fontsize=15)

        else:
            flatax[fignum].set_title(cancer,fontsize=15)
        flatax[fignum].legend().remove()
        flatax[fignum].set_xlabel('')

        if fignum in [0,3]:
            flatax[fignum].set_ylabel("IDH1/2")
        elif fignum==6:
            flatax[fignum].set_ylabel(str(gene))
        else:
            flatax[fignum].set_ylabel('')
        flatax[fignum].set_xticks([])
        flatax[fignum].set_yticks([])
        fignum+=1

# special IDH2 case
df = pd.concat([df_umap_spectral["LAML"],mut_pancan["LAML"]["IDH2"].replace({0: "non-mut",1: "mut"})],axis=1,join="inner")
df.columns = np.append(df_umap_spectral["LAML"].columns,["Mutation"])
edgecolor_list = df["Mutation"].replace({"non-mut": "white", "mut": "black"}).values
linewidth_list = df["Mutation"].replace({"non-mut": 0, "mut": 1}).values
sns.scatterplot(data=df,x="Comp1",y="Comp2",hue="Cluster",size="Mutation",
                            sizes=size_plot,ax=flatax[4],legend="auto",
                            palette={"Low meth cluster": seapal[0], "High meth cluster": seapal[3],"Intermediate meth cluster": seapal[8]},
                           edgecolor=edgecolor_list,linewidth=linewidth_list)
flatax[4].set_title("LAML (2)",fontsize=15)
flatax[4].legend().remove()
flatax[4].set_xlabel('')
flatax[4].set_ylabel('')
flatax[4].set_xticks([])
flatax[4].set_yticks([])

flatax[5].axis("off")

lines, labels = flatax[0].get_legend_handles_labels()
fig.legend(lines, labels, loc = 'center right', bbox_to_anchor = (0.05,0,1,1),
            bbox_transform = plt.gcf().transFigure,fontsize=15,frameon=False)
fig.tight_layout()

In [ ]:
fig.savefig("UMAP_mutation_goi.png",bbox_inches="tight")
fig.savefig("UMAP_mutation_goi.svg",bbox_inches="tight")

In [ ]:
seapal = sns.color_palette("bright")
fig, ax = plt.subplots(1,4,figsize=(10,2))
flatax = ax.flatten()
fignum=8
size_plot = {"mut": 40, "non-mut": 10}

for i,cancer in enumerate(nsd1_mapping):
    df = pd.concat([df_umap_spectral[cancer],mut_pancan[cancer][gene].replace({0: "non-mut",1: "mut"})],axis=1,join="inner")
    df.columns = np.append(df_umap_spectral[cancer].columns,["Mutation"])
    edgecolor_list = df["Mutation"].replace({"non-mut": "white", "mut": "black"}).values
    linewidth_list = df["Mutation"].replace({"non-mut": 0, "mut": 1}).values

    sns.scatterplot(data=df,x="Comp1",y="Comp2",hue="Cluster",size="Mutation",
                        sizes=size_plot,ax=ax[i],legend="auto",
                        palette={"Low meth cluster": seapal[0], "High meth cluster": seapal[3],"Intermediate meth cluster": seapal[8]},
                       edgecolor=edgecolor_list,linewidth=linewidth_list)
    ax[i].set_title(cancer,fontsize=15)
    ax[i].legend().remove()
    ax[i].set_xlabel('')
    if i==0:
        print(gene)
        ax[i].set_ylabel(str(gene))
    else:
        ax[i].set_ylabel('')
    ax[i].set_xticks([])
    ax[i].set_yticks([])
        
lines, labels = flatax[0].get_legend_handles_labels()
fig.legend(lines, labels, loc = 'center right', bbox_to_anchor = (0.3,0.0,1,1),
            bbox_transform = plt.gcf().transFigure,fontsize=12,frameon=False)
fig.tight_layout()

In [ ]:
fig.savefig("NSD1_mutations.svg",bbox_inches="tight")
fig.savefig("NSD1_mutations.png",bbox_inches="tight")

### For non-CIMP

In [ ]:
# to test per gene and not per cancer
invert_mutations_to_test_nCIMP = {}
for key, val in mutations_to_test_nCIMP.items():
    for mut in val:
        if mut in list(invert_mutations_to_test_nCIMP.keys()):
            invert_mutations_to_test_nCIMP[mut].append(key)
        else:
            invert_mutations_to_test_nCIMP[mut] = [key]

In [ ]:
fig, axs = plt.subplots(10, 9, figsize=(35,20))
for a in axs.flatten():
    a.axis("off")

In [ ]:
mutation_ps = []
for j,mut in tqdm(enumerate(sorted(invert_mutations_to_test_nCIMP))):
    for i,cancer in enumerate(invert_mutations_to_test_nCIMP[mut]):
        common_pat = mut_pancan[cancer].index.intersection(CIMPness[cancer].index)
        common_pat = common_pat.intersection(high_conf_patients[cancer])
        y = mut_pancan[cancer].loc[common_pat][mut].ravel()
        X = CIMPness[cancer].loc[common_pat].values.ravel()
        corr,p = pointbiserialr(y,X)
        mutation_ps.append(p)
corrected_ps = multipletests(mutation_ps,method="fdr_bh")[1]

In [ ]:
# compute the point biserial correlation between the CIMP score and a mutation
flatax = axs.flatten()
fignum = 0
for j,mut in tqdm(enumerate(sorted(invert_mutations_to_test_nCIMP))):
    for i,cancer in enumerate(invert_mutations_to_test_nCIMP[mut]):
        common_pat = mut_pancan[cancer].index.intersection(CIMPness[cancer].index)
        common_pat = common_pat.intersection(high_conf_patients[cancer])
        df1 = mut_pancan[cancer].loc[common_pat][[mut]]
        

        plotdf = pd.concat([CIMPness[cancer],df1],axis=1,join="inner").sort_values(0).reset_index().reset_index().set_index("index")
        plotdf.columns = ["x","CIMP score","mutation"]
        
        X = plotdf["CIMP score"].ravel()
        y = plotdf["mutation"].ravel()
        
        plotdf["mutation"] = plotdf["mutation"].replace({0: "whitesmoke", 1: "black"})
        
        corr,_ = pointbiserialr(y,X)
        p = corrected_ps[fignum]
        print(cancer,mut,corr,p)
        plotdf["x"] = plotdf["x"]*2
        plotdf = pd.concat([plotdf,pd.DataFrame([1]*plotdf.shape[0],index=plotdf.index,columns=["y"])],axis=1)
        
        sns.scatterplot(data = plotdf, x="x", y="y", c=plotdf.mutation.ravel(), alpha=1, linewidth=0.6, s=1800, ax=flatax[fignum], marker="|")
            
        rstars = "(NS)" if p>=0.1 else ("*" if 0.01<=p<0.1 else ("**" if 0.001<=p<0.01 else ("***" if 0.0001<=p<0.01 else "****")))
            
        flatax[fignum].set_title(mut+" in "+cancer+"\n r="+str(round(corr,2))+rstars,fontsize=25)
        fignum+=1

In [ ]:
ax = flatax[-1]
ax.axis("on")
ax.set_xlabel("CIMP score",fontsize=25)
ax.set_xticks([0,1])
ax.set_xticklabels(["Min score","Max score"],rotation=0,rotation_mode="default")
ax.set_ylabel("Mut",fontsize=25)
ax.set_yticks([])
fig.tight_layout()

In [ ]:
fig

In [ ]:
fig.savefig("mutation_vs_nCIMP_score.png",bbox_inches="tight")
fig.savefig("mutation_vs_nCIMP_score.svg",bbox_inches="tight")

### CIMP score vs xCell

In [ ]:
immune_scores = pd.read_csv("path\to\Thorsson\Scores_160_Signatures.tsv.gz", delimiter = "\t")
immune_scores = immune_scores.set_index("SetName").loc[["LIexpression_score","CSF1_response","TGFB_score_21050467","Module3_IFN_score","CHANG_CORE_SERUM_RESPONSE_UP"]]
immune_scores = immune_scores.drop("Source",axis=1).T
immune_scores = immune_scores.reset_index()
immune_scores["index"] = immune_scores["index"].apply(lambda x: x[:15])
immune_scores = immune_scores.set_index("index")
immune_scores = immune_scores[~immune_scores.index.duplicated()]

In [ ]:
immune_scores.columns = ["Lymphocyte infiltration","CSF1 response","TGF-beta pathway activation","IFN-gamma response","Wound healing"]

In [ ]:
CIBERSORT_comp = pd.read_csv("path\to\Thorsson\TCGA.Kallisto.fullIDs.cibersort.relative.tsv", delimiter = "\t")
CIBERSORT_comp["SampleID"] = CIBERSORT_comp["SampleID"].str.replace(".","-").apply(lambda x: x[:15])
CIBERSORT_comp = CIBERSORT_comp.set_index("SampleID").drop(["CancerType","P.value","Correlation","RMSE"],axis=1)
CIBERSORT_comp.columns = CIBERSORT_comp.columns.str.replace("."," ")
CIBERSORT_comp = CIBERSORT_comp[~CIBERSORT_comp.index.duplicated()]

In [ ]:
thorsson_pat_info = pd.read_csv("path\to\thorsson_pat_info.txt",sep="\t")
thorsson_pat_info["TCGA Participant Barcode"] = thorsson_pat_info["TCGA Participant Barcode"].apply(lambda x: x+"-01")
thorsson_pat_info = thorsson_pat_info.set_index("TCGA Participant Barcode")

In [ ]:
thorsson_corr = thorsson_pat_info[['Leukocyte Fraction',
       'Stromal Fraction', 'Intratumor Heterogeneity',
       'Proliferation', 'Wound Healing', 'Macrophage Regulation',
       'Lymphocyte Infiltration Signature Score',]]
for col in thorsson_corr.columns:
    thorsson_corr[col] = thorsson_corr[col].str.replace(",",".")

In [ ]:
xCell_comp = pd.read_csv("path\to\xCell\info",sep="\t").set_index("Unnamed: 0")
xCell_comp.columns = xCell_comp.columns.str.replace(".","-")
xCell_comp = xCell_comp.T

In [ ]:
immune_sign_pc,immune_sign_pc_nocorrec = pd.DataFrame(columns=thorsson_corr.columns),pd.DataFrame(columns=thorsson_corr.columns)
# compute the spearman r associated with the signatures/immune characteristics
for cancer in tqdm(sorted(CIMP_cancers)):
    if cancer=="LAML":
        continue
    r_list,p_list = [],[]
    common_pat = immune_scores.index.intersection(new_meth_data_pc[cancer].index)
    cancer_xcell = thorsson_corr.loc[common_pat].astype(float)
    pat_cimp = CIMPness[cancer].loc[common_pat]
    df_comp = pd.concat([cancer_xcell,pat_cimp],axis=1)
    cimp = df_comp.iloc[:,-1].values.ravel()
    for i in range(cancer_xcell.shape[1]):
        comp = df_comp.iloc[:,i].values.ravel()
        r,p = spearmanr(cimp,comp)
        r_list.append(r)
        p_list.append(p)
    # multiple testing correction
    corrected_p = multipletests(p_list,method='fdr_bh')
    assoc_bool = corrected_p[0]

    assoc_r = [r_list[i] if assoc else 0 for i,assoc in enumerate(assoc_bool)]

    df = pd.DataFrame(np.array(assoc_r).reshape(1,-1),columns=cancer_xcell.columns,index=[cancer])
    df_nocorrec = pd.DataFrame(np.array(r_list).reshape(1,-1),columns=cancer_xcell.columns,index=[cancer])
    # with and without FDR correction
    immune_sign_pc = pd.concat([immune_sign_pc,df])
    immune_sign_pc_nocorrec = pd.concat([immune_sign_pc_nocorrec,df_nocorrec])

In [ ]:
immune_sign_pc_nocorrec = immune_sign_pc_nocorrec.fillna(0)

In [ ]:
plt.rc("font", size=20)
fig,ax = plt.subplots(figsize=(25,10))
sns.heatmap(immune_sign_pc_nocorrec.T.round(2),center=0,ax=ax,annot=immune_sign_pc.T.round(2).replace(0,'').astype(str).values,
            annot_kws={"fontsize": 23},fmt = '',cbar_kws={'label': 'Spearman R'}, vmin=-0.5, vmax=0.5,
           cmap=sns.diverging_palette(220, 20, as_cmap=True))
ax.set_ylabel("")
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,rotation_mode="default",fontsize=30)
ax.set_yticklabels(ax.get_yticklabels(),rotation=0,rotation_mode="default",fontsize=30)
fig.savefig("immune_score.png",bbox_inches="tight")
fig.savefig("immune_score.svg",bbox_inches="tight")

In [ ]:
def get_immune_correlations(immune_comp_df, new_meth_data_pc, CIMPness):
#     Parameters: immune_comp_df (pd.DataFrame): dataframe containing the patients as rows and the deconvolved immune composition (1 col = 1 cell subtype) as columns
#             new_meth_data_pc (dict): keys  are cancer types, values are pd.Df (n_patients, n_cpg_probes)
#             CIMPness (dict): keys are cancer types, values are pd.Df (n_patients,1) (rows=patients,columns=CIMP score)
#     Returns: xcell_pc (pd.DataFrame): dataframe containing the masked correlation values (only FDR q>0.05) 
#             xcell_pc_nocorrec (pd.DataFrame): same as xcell_pc but without the masking with the q value 

    xcell_pc,xcell_pc_nocorrec = pd.DataFrame(columns=immune_comp_df.columns),pd.DataFrame(columns=immune_comp_df.columns)
    # same as above but computing the associated spearman R for cell composition
    for cancer in tqdm(sorted(CIMP_cancers)):
        if cancer=="LAML":
            continue
        r_list,p_list = [],[]

        common_pat = immune_comp_df.index.intersection(new_meth_data_pc[cancer].index)
        cancer_xcell = immune_comp_df.loc[common_pat]
        pat_cimp = CIMPness[cancer].loc[common_pat]
        df_comp = pd.concat([cancer_xcell,pat_cimp],axis=1)
        cimp = df_comp.iloc[:,-1].values.ravel()
        for i in range(cancer_xcell.shape[1]):
            comp = df_comp.iloc[:,i].values.ravel()
            r,p = spearmanr(cimp,comp)
            if np.isnan(r):
                r,p=0,0
            r_list.append(r)
            p_list.append(p)
        corrected_p = multipletests(p_list,method='fdr_bh')
        assoc_bool = corrected_p[0]

        assoc_r = [r_list[i] if assoc else 0 for i,assoc in enumerate(assoc_bool)]

        df = pd.DataFrame(np.array(assoc_r).reshape(1,-1),columns=cancer_xcell.columns,index=[cancer])
        df_nocorrec = pd.DataFrame(np.array(r_list).reshape(1,-1),columns=cancer_xcell.columns,index=[cancer])
        xcell_pc = pd.concat([xcell_pc,df])
        xcell_pc_nocorrec = pd.concat([xcell_pc_nocorrec,df_nocorrec])
    
    return xcell_pc, xcell_pc_nocorrec

In [ ]:
CIBERSORT_pc, CIBERSORT_pc_nocorrec = get_immune_correlations(CIBERSORT_comp, new_meth_data_pc, CIMPness)
xcell_pc, xcell_pc_nocorrec = get_immune_correlations(xCell_comp, new_meth_data_pc, CIMPness)

In [ ]:
CIBERSORT_pc_nocorrec = xcell_pc_nocorrec.fillna(0)
xcell_pc_nocorrec = xcell_pc_nocorrec.fillna(0)

In [ ]:
plt.rc("font", size=20)
fig,ax = plt.subplots(figsize=(25,20))
sns.heatmap(CIBERSORT_pc_nocorrec.T.round(2),center=0,ax=ax,annot=CIBERSORT_pc.T.round(2).replace(0,'').astype(str).values,
            annot_kws={"fontsize": 23},fmt = '',cbar_kws={'label': 'Spearman r'}, vmin=-0.5, vmax=0.5,
           cmap=sns.diverging_palette(220, 20, as_cmap=True))
ax.set_ylabel("")
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,rotation_mode="default",fontsize=30)
ax.set_yticklabels(ax.get_yticklabels(),rotation=0,rotation_mode="default",fontsize=30)
fig.savefig("cibersort_corr.png",bbox_inches="tight")
fig.savefig("cibersort_corr.svg",bbox_inches="tight")

In [ ]:
plt.rc("font", size=20)
fig,ax = plt.subplots(figsize=(25,35))
sns.heatmap(xcell_pc_nocorrec.T.round(2),center=0,ax=ax,annot=xcell_pc.T.round(2).replace(0,'').astype(str).values,
            annot_kws={"fontsize": 23},fmt = '',cbar_kws={'label': 'Spearman r'}, vmin=-0.5, vmax=0.5,
           cmap=sns.diverging_palette(220, 20, as_cmap=True))
ax.set_ylabel("")
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,rotation_mode="default",fontsize=30)
ax.set_yticklabels(ax.get_yticklabels(),rotation=0,rotation_mode="default",fontsize=30)
fig.savefig("xcell_corr.png",bbox_inches="tight")
fig.savefig("xcell_corr.svg",bbox_inches="tight")

In [ ]:
seapal = sns.color_palette("bright")
immune_subtypes = ["C1","C2","C3","C4","C5","C6"]
immune_subtypes_mapping = {"C1":1,"C2":2,"C3":3,"C4":4,"C5":5,"C6":6}
color_mapping = {1: seapal[3], 2: seapal[-2], 3: seapal[2], 4: seapal[-1], 5: seapal[0], 6: seapal[4], -1: seapal[5]}
immune_df = pd.DataFrame(data=np.zeros((6,19)),index=immune_subtypes,columns=sorted(CIMP_cancers))

In [ ]:
y,OR_l=[-1 for i in range(len(sorted(CIMP_cancers)))],[0 for i in range(len(sorted(CIMP_cancers)))]

for i,cancer in enumerate(sorted(CIMP_cancers)):
    if cancer=="LAML":
        continue
    low_cl = consensus_clustering_spectral[cancer][consensus_clustering_spectral[cancer]["cluster"]==1].index
    if cancer in triple_cluster_cancers_aff:
        high_cl = consensus_clustering_spectral[cancer][consensus_clustering_spectral[cancer]["cluster"]==3].index
    else:
        high_cl = consensus_clustering_spectral[cancer][consensus_clustering_spectral[cancer]["cluster"]==2].index
    high_cl_immune = thorsson_pat_info["Immune Subtype"].loc[thorsson_pat_info.index.intersection(high_cl)].dropna()
    low_cl_immune = thorsson_pat_info["Immune Subtype"].loc[thorsson_pat_info.index.intersection(low_cl)].dropna()
    
    dict_high = Counter(high_cl_immune.values.ravel())
    size_high = high_cl_immune.shape[0]
    
    for subtype in immune_subtypes:
        immune_df.loc[subtype][cancer]=np.round(dict_high[subtype]/size_high*100)
    
    for subtype in immune_subtypes:
        count_high_subtype = (high_cl_immune==subtype).sum()
        count_low_subtype = (low_cl_immune==subtype).sum()
        size_high = high_cl_immune.shape[0]
        size_low = low_cl_immune.shape[0]
        contigency_table = np.array([[count_high_subtype,count_low_subtype],
                                     [size_high-count_high_subtype,size_low-count_low_subtype]])
        OR,p = fisher_exact(contigency_table)
        if OR>1 and p<0.05:
            print(cancer,subtype)

In [ ]:
plt.rc("font", size=20)
fig,ax = plt.subplots(figsize=(25,5))
sns.heatmap(immune_df.astype(int),center=0,ax=ax,annot=immune_df.astype(int),
            annot_kws={"fontsize": 22},fmt = '',cbar_kws={'label': 'Percentage'}, vmin=0, vmax=100,
#             cmap=sns.color_palette("vlag", as_cmap=True))
            cmap=sns.diverging_palette(220, 20, as_cmap=True))
ax.set_ylabel("")
ax.set_xticklabels(ax.get_xticklabels(),rotation=45,rotation_mode="default",fontsize=30)
ax.set_yticklabels(["Wound healing","IFN-gamma dominant","Inflammatory",
                    "Lymphocyte depleted","Immunologically quiet","TGF-beta dominant"],fontsize=25,rotation=0) 
fig.savefig("immune_subtype_breakdown.png",bbox_inches="tight")
fig.savefig("immune_subtype_breakdown.svg",bbox_inches="tight")

# Gene expression analysis

Finally, we analyze gene expression. Differential gene expression is calculated using the DESeq2 
package in R (code in script DESeq_GEX.R). 

In [ ]:
gencode_mapping_goi = {"TET1": "ENSG00000138336.8", "TET2": "ENSG00000168769.11", 
                       "TET3": "ENSG00000139372.13", "TDG": "ENSG00000139372.13", "TDGF1": "ENSG00000241186.6",
                       "MBD4": "ENSG00000129071.8", "APOBEC1": "ENSG00000111701.6", "CTCF": "ENSG00000102974.13",
                       "IDH1": "ENSG00000138413.12", "IDH2": "ENSG00000182054.8", "BAZ2A": "ENSG00000076108.10", 
                       "CTCFL": "ENSG00000124092.11", "DMAP1": "ENSG00000178028.12", "DNMT1": "ENSG00000130816.13",
                       "DNMT3A": "ENSG00000119772.15", "DNMT3B": "ENSG00000088305.17", "GADD45A": "ENSG00000116717.10",
                       "GADD45B": "ENSG00000099860.7", "GADD45G": "ENSG00000130222.9", "GADD45GIP1": "ENSG00000179271.2", 
                       "UHRF1BP1": "ENSG00000065060.15","UHRF1BP1L": "ENSG00000111647.11", "UHRF1": "ENSG00000276043.3", 
                       "MBD1": "ENSG00000141644.16", "MBD2": "ENSG00000134046.10"
                      }

In [ ]:
genes_meth_gex_gencode,genes_demeth_gex_gencode = [],[]
for gene in genes_meth_gex:
    genes_meth_gex_gencode.append(gencode_mapping_goi[gene])
for gene in genes_demeth_gex:
    genes_demeth_gex_gencode.append(gencode_mapping_goi[gene])

In [ ]:
# 450k Illumina annotation
Meth45k = []
with open("path\to\Illumina\annotation") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        Meth45k.append(row)
colnames_45k = Meth45k[7]
methylation_val_45k = Meth45k[8:]
Meth45k_df = pd.DataFrame(methylation_val_45k,columns=colnames_45k)
Meth45k_df = Meth45k_df.set_index("IlmnID")

In [ ]:
Counter(Meth45k_df['Relation_to_UCSC_CpG_Island'])

In [ ]:
# get genes associated with differentially methylated probes 
enriched_genes = {}
for cancer in sign_pos_spectral:
    sel_genes = []
    for gene in Meth45k_df.loc[sign_pos_spectral[cancer].values.ravel()]['UCSC_RefGene_Name'].values:
        if (gene=="" or gene==None):
            continue
        else:
            sel_genes += gene.split(";")
    enriched_genes[cancer]=np.unique(sel_genes)
    enriched_genes[cancer]=merged_table.loc[merged_table.index.intersection(enriched_genes[cancer])]["gencode_id_gex"].values

In [ ]:
# dictionnary from cpg probe to gene associated or cpg probe to position (promoter, gene body, TSS)
dict_cpg_to_gene = Meth45k_df['UCSC_RefGene_Name'].str.split(";").to_dict()
dict_cpg_to_position = Meth45k_df['UCSC_RefGene_Group'].str.split(";").to_dict()

In [ ]:
# dictionnary from gene to cpg probe
dict_gene_to_cpg = {}
n = len(dict_cpg_to_gene)
i=0
for cpg, genes in tqdm(dict_cpg_to_gene.items()):
    if i==(n-1):
        continue
    for gene in genes:
        if gene in list(dict_gene_to_cpg.keys()):
            dict_gene_to_cpg[gene].append(cpg)
        else:
            dict_gene_to_cpg[gene] = [cpg]
    i+=1

In [ ]:
# gencode annotations of histone writer and eraser genes
gencode_gene_writer_Hmeth = merged_table.loc[merged_table.index.intersection(all_genes_writer_Hmeth)]["gencode_id_gex"].values
gencode_gene_eraser_Hmeth = merged_table.loc[merged_table.index.intersection(all_genes_eraser_Hmeth)]["gencode_id_gex"].values

In [ ]:
all_meth_genes_gencode = merged_table.loc[merged_table.index.intersection(genes_to_keep)]["gencode_id_gex"].values

In [ ]:
inv_merged_table = merged_table.reset_index().set_index("gencode_id_gex")

In [ ]:
# differentially expressed genes computed with DESeq2
diff_gex_dir = "path\to\diff_expression_DESeq"
diff_gex_pc = {}
for cancer in sorted(CIMP_cancers):
    diff_gex_pc[cancer] = pd.read_csv(os.path.join(diff_gex_dir,cancer+"_diff_gene_expression.csv")).set_index("Unnamed: 0")
    diff_gex_pc[cancer].columns = ["logfold","p"]
    diff_gex_pc[cancer] = diff_gex_pc[cancer].reset_index()
    diff_gex_pc[cancer].columns = ["gene","logfold","p"]
    diff_gex_pc[cancer] = diff_gex_pc[cancer].iloc[np.where(multipletests(diff_gex_pc[cancer]["p"].values.ravel())[0])[0]]

In [ ]:
n_diff_gex = []
for cancer in sorted(CIMP_cancers):
    n_diff_gex.append(diff_gex_pc[cancer].shape[0])
print(np.mean(n_diff_gex),np.quantile(n_diff_gex,0.25),np.quantile(n_diff_gex,0.75))

In [ ]:
# diff genes expression in histone methylation genes
for cancer in diff_gex_pc:
    print(cancer)
    diff_genes_meth = list(diff_gex_pc[cancer].set_index("gene").index.intersection(gencode_gene_writer_Hmeth))
    print("Diff expressed histone methylase genes : ",inv_merged_table.loc[diff_genes_meth]["index"].values)
    print("Log2 fold cl2 vs cl1: ",diff_gex_pc[cancer].set_index("gene").loc[diff_genes_meth]["logfold"].values)
    print("Genes with diff promoter meth : ",set(enriched_genes[cancer]) & set(diff_genes_meth))

In [ ]:
# diff genes expression in histone demethylation genes
for cancer in diff_gex_pc:
    print(cancer)
    diff_genes_meth = list(diff_gex_pc[cancer].set_index("gene").index.intersection(gencode_gene_eraser_Hmeth))
    print("Diff expressed meth genes : ",inv_merged_table.loc[diff_genes_meth]["index"].values)
    print("Log2 fold cl2 vs cl1: ",diff_gex_pc[cancer].set_index("gene").loc[diff_genes_meth]["logfold"].values)
    print("Genes with diff promoter meth : ",set(enriched_genes[cancer]) & set(diff_genes_meth))

In [ ]:
# diff genes expression in DNA methylation/demethylation genes
for cancer in diff_gex_pc:
    print(cancer)
    diff_genes_meth = list(diff_gex_pc[cancer].set_index("gene").index.intersection(genes_meth_gex_gencode))
    diff_genes_demeth = list(diff_gex_pc[cancer].set_index("gene").index.intersection(genes_demeth_gex_gencode))
    print("Diff expressed meth genes : ",inv_merged_table.loc[diff_genes_meth]["index"].values)
    print("Log2 fold cl2 vs cl1: ",diff_gex_pc[cancer].set_index("gene").loc[diff_genes_meth]["logfold"].values)
    print("Genes with diff promoter meth : ",set(enriched_genes[cancer]) & set(diff_genes_meth))
    print("Diff expressed demeth genes : ",inv_merged_table.loc[diff_genes_demeth]["index"].values)
    print("Log2 fold cl2 vs cl1: ",diff_gex_pc[cancer].set_index("gene").loc[diff_genes_demeth]["logfold"].values)
    print("Genes with diff promoter meth : ",set(enriched_genes[cancer]) & set(diff_genes_demeth))

In [ ]:
# get candidate downstream genes (genes with differential GEX and hypermethylated promoters in the high methylated group)
diff_gex_offgene_dir = "path\to\diff_expression_offgene"
downstream_genes_pc, body_diff_genes_pc, promoter_diff_genes_pc = {},{},{}
for cancer in tqdm(sorted(CIMP_cancers)):
    assoc_cpg = []
    common_genes = inv_merged_table.index.intersection(diff_gex_pc[cancer]["gene"].values.ravel())
    list_gene = inv_merged_table.loc[common_genes]["index"].values.ravel()
    for gene in list_gene:
        if not(gene in list(dict_gene_to_cpg.keys())):
            continue
        assoc_cpg+=dict_gene_to_cpg[gene]

    meth_exp = new_meth_data_pc[cancer][sign_pos_spectral[cancer].set_index("0").index.intersection(assoc_cpg)]
    common_pat_clustering = consensus_clustering_spectral[cancer]
    ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
    ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
    ind_cl_3 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
    if cancer in triple_cluster_cancers_aff:
        ind_cl_2 = ind_cl_3
    cl1_gex = meth_exp.loc[ind_cl_1].mean()
    cl2_gex = meth_exp.loc[ind_cl_2].mean()

    hypermethylated_cpgs = cl1_gex.index[np.where(cl1_gex-cl2_gex<-0.1)[0]]
    downstream_genes, body_genes, promoter_genes = [],[],[]
    for cpg in list(hypermethylated_cpgs):
        if "Body" in dict_cpg_to_position[cpg]:
            body_genes+=dict_cpg_to_gene[cpg]
        else:
            promoter_genes+=dict_cpg_to_gene[cpg]
        downstream_genes+=dict_cpg_to_gene[cpg]

    downstream_genes_pc[cancer] = np.unique(downstream_genes)
    body_diff_genes_pc[cancer] = np.unique(body_genes)
    promoter_diff_genes_pc[cancer] = np.unique(promoter_genes)
    pd.DataFrame(downstream_genes_pc[cancer]).to_csv(os.path.join(diff_gex_offgene_dir,cancer+"_downstream_genes_gex.csv"),index=False)

In [ ]:
all_downstream_genes = []
for cancer in downstream_genes_pc:
    all_downstream_genes+=list(downstream_genes_pc[cancer])

In [ ]:
Counter(all_downstream_genes).most_common(10)

## Find signatures for different etiologies

Finally, we compute here the average beta values over all CpG Islands, shores and shelves to uncover methylation signatures linked to CIMP status (strong, weak and non CIMP)

In [ ]:
# break according to methylation status
sCIMP1 = ["ACC","COAD","KIRC","KIRP","MESO","STAD","THCA",]
sCIMP2 = ["CESC","GBM","HNSC","LAML","LGG","LIHC","LUAD","LUSC","PCPG","READ","SARC","SKCM",]
nonCIMP = ["BLCA","BRCA","ESCA","PAAD","PRAD","THYM","UCEC"]

In [ ]:
def get_methylation_sign_CIMP(flatax,cancer_list,Meth45k_df,meth_data_pc,consensus_clustering_spectral,
                              high_conf_patients,triple_cluster_cancers_aff):
    #     Parameters: flatax (matplotlib.pyplot.Axes.axe): flattened axis of the subplot object
    #                cancer_list (list): list of the cancer types to consider
    #                Meth45k_df (pd.DataFrame): df of the Illumina 450k annotation 
    #                meth_data_pc (dict): keys are cancer types, values of pd.DataFrame of beta values (n_patients,n_cpg_ids)
    #                consensus_clustering (dict): keys are cancer types, values are pd.Df (n_patients,1) (rows=patients,columns=cluster membership)
    #                high_conf_patients (dict): keys are cancer types, values are list of patients with SSC>0
    #                triple_cluster_cancers_aff (list): list of cancer types with 3 methyaltion clusters
    #     Returns: flatax (matplotlib.pyplot.Axes.axe): updated/filled axis for subplot object
    seapal = sns.color_palette("bright")
    fignum = 0
    mean_cgi_pc = {}
    for cancer in tqdm(cancer_list):
        # get the information for probes: position, strand polarity, CGI position
        cpg_info_df = Meth45k_df[['CHR', 'MAPINFO', 'Strand',"UCSC_CpG_Islands_Name",'Relation_to_UCSC_CpG_Island']]
        common_cpg = cpg_info_df.index.intersection(meth_data_pc[cancer].columns)
        cpg_info_df = cpg_info_df.loc[common_cpg]
        cpg_island_df = cpg_info_df["UCSC_CpG_Islands_Name"].str.split(":")
        CGI_sizes, CGI_start, CGI_end = [],[],[]

        # for every position in CGI, get start, end and size to break into bins
        for cpg in cpg_island_df.index:
            pos = cpg_island_df.loc[cpg][1].split("-")
            size  = int(pos[1])-int(pos[0])
            start, end = int(pos[0]),int(pos[1])
            CGI_sizes.append(size)
            CGI_start.append(start)
            CGI_end.append(end)
        cpg_info_df["CGI_sizes"] = CGI_sizes
        cpg_info_df["CGI_start"] = CGI_start
        cpg_info_df["CGI_end"] = CGI_end
        bin_membership = []
        for cpg in cpg_info_df.index:
            cpg_inf = cpg_info_df.loc[cpg]
            typ = cpg_inf["Relation_to_UCSC_CpG_Island"]
            strand = cpg_inf["Strand"]
            if typ=="Island":
                bin_size = cpg_inf["CGI_sizes"]//10+1
            else:
                bin_size = 200
            pos = int(cpg_inf["MAPINFO"])

            # there will be 50 bins all total, indexed starting at the N_shelf at 0, max 50 at the S_shelf
            # eg if bin 2 of the S_shore, the bin membership will be 40 (beginning of S_shore) + 2 (bin within the shore) = 42
            # if the strand is forward, then computation is straightforward ; if it is reverse, we must reverse the roles (S_shore ~ N_shore)

            if typ=="Island":
                rel_pos = pos-int(cpg_inf["CGI_start"])
                if strand=="F":
                    bin_membership.append(rel_pos//bin_size+1+20)
                else:
                    bin_membership.append(10-rel_pos//bin_size+20)
            elif typ=="N_Shelf":
                rel_pos = int(cpg_inf["CGI_start"])-pos-2000
                if strand=="F":
                    bin_membership.append(10-rel_pos//bin_size)
                else:
                    bin_membership.append(rel_pos//bin_size+1+40)
            elif typ=="S_Shelf":
                rel_pos = pos-int(cpg_inf["CGI_end"])-2000
                if strand=="F":
                    bin_membership.append(rel_pos//bin_size+1+40)
                else:
                    bin_membership.append(10-rel_pos//bin_size)
            elif typ=="N_Shore":
                rel_pos = int(cpg_inf["CGI_start"])-pos
                if strand=="F":
                    bin_membership.append(10-rel_pos//bin_size+10)
                else:
                    bin_membership.append(rel_pos//bin_size+1+30)
            elif typ=="S_Shore":
                rel_pos = pos-int(cpg_inf["CGI_end"])
                if strand=="F":
                    bin_membership.append(rel_pos//bin_size+1+30)
                else:
                    bin_membership.append(10-rel_pos//bin_size+10)

        cpg_info_df["bin"] = bin_membership
        common_pat_clustering = consensus_clustering_spectral[cancer].loc[consensus_clustering_spectral[cancer].index.intersection(high_conf_patients[cancer])]
        ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
        ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
        ind_cl_3 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
        cl_1_mean = new_meth_data_pc[cancer].loc[ind_cl_1].mean()
        cl_2_mean = new_meth_data_pc[cancer].loc[ind_cl_2].mean()
        cl_3_mean = new_meth_data_pc[cancer].loc[ind_cl_3].mean()
        normal_mean = normal_meth[cancer].mean()

        # get the methylation for low, int and high methylated cluster, as well as normal
        if cancer in triple_cluster_cancers_aff:
            full_cpg_island_df= pd.concat([cpg_info_df,cl_1_mean,cl_2_mean,cl_3_mean,normal_mean],axis=1).dropna()
            full_cpg_island_df.columns = list(cpg_info_df.columns) + ["Low meth", "Intermediate meth", "High meth", "Normal"]
        else:
            full_cpg_island_df= pd.concat([cpg_info_df,cl_1_mean,cl_2_mean,normal_mean],axis=1).dropna()
            full_cpg_island_df.columns = list(cpg_info_df.columns) + ["Low meth", "High meth", "Normal"]
            
        mean_cgi_meth = full_cpg_island_df.groupby(["bin"]).mean().reset_index()
        print(cancer)
        mean_cgi_pc[cancer]=mean_cgi_meth
        # plot avg meth on same plot
        sns.lineplot(data=mean_cgi_meth,x="bin",y="Low meth",color=seapal[0],ax=flatax[fignum],label="Low meth cluster")
        if cancer in triple_cluster_cancers_aff:
            sns.lineplot(data=mean_cgi_meth,x="bin",y="Intermediate meth",color=seapal[8],ax=flatax[fignum], label="Intermediate meth cluster")
        sns.lineplot(data=mean_cgi_meth,x="bin",y="High meth",color=seapal[3],ax=flatax[fignum], label="High meth cluster")
        sns.lineplot(data=mean_cgi_meth,x="bin",y="Normal",color=seapal[7],ax=flatax[fignum], label="Normal tissue")
        flatax[fignum].set_title(cancer+" average methylation over CGI",fontsize=18)
        flatax[fignum].set_ylabel("Methylation (beta)",fontsize=15)
        flatax[fignum].set_xlim([0,50])
        flatax[fignum].set_ylim([0,0.85])
        fignum+=1
    return flatax,mean_cgi_pc

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(13,6))
flatax = axs.flatten()

In [ ]:
flatax,mean_cgi_pc = get_methylation_sign_CIMP(flatax,sCIMP1,Meth45k_df,meth_data_pc,consensus_clustering_spectral,
                              high_conf_patients,triple_cluster_cancers_aff)

In [ ]:
# visualization
lines, labels = flatax[0].get_legend_handles_labels()
for i,cancer in enumerate(sCIMP1):
    a = flatax[i]
    a.set_ylabel('')
    if not (i in [0,4]):
        a.set_yticklabels([])
    a.set_xticks([5,15,25,35,45])
    if i in [4,5,6]:
        a.set_xticklabels(["N_Shelf","N_Shore","CGI","S_Shore","S_Shelf"],rotation=55,rotation_mode="default",fontsize=20)
    else:
        a.set_xticklabels([])
    a.set_title(cancer,fontsize=20)
    a.set_xlabel('')
    a.legend_=None
fig.text(0.5, -0.02, 'Region', ha='center',fontsize=20)
fig.text(-0.01, 0.5, 'Methylation (beta)', va='center', rotation='vertical',fontsize=20)
flatax[-1].axis("off")
fig.legend(lines, labels, loc = 'center right', bbox_to_anchor = (1.5,0.05,1,1),
            bbox_transform = plt.gcf().transFigure,fontsize=20,frameon=False)
fig.tight_layout()
fig

In [ ]:
fig.savefig("signature_sCIMP1_wnormal.svg",bbox_inches="tight")
fig.savefig("signature_sCIMP1_wnormal.png",bbox_inches="tight")

In [ ]:
mean_cgi_full = pd.DataFrame(columns=["bin","Low meth","Intermediate meth","High meth","Normal"])
for cancer in mean_cgi_pc:
    if cancer in triple_cluster_cancers_aff:
        mean_cgi_full= pd.concat([mean_cgi_full,mean_cgi_pc[cancer][["bin","Low meth","Intermediate meth","High meth","Normal"]]])
    else:
        mean_cgi_full= pd.concat([mean_cgi_full,mean_cgi_pc[cancer][["bin","Low meth","High meth","Normal"]]])
mean_cgi_full = mean_cgi_full.groupby("bin").mean()

In [ ]:
seapal=sns.color_palette("bright")
fig,ax=plt.subplots(1,1,figsize=(7,5))
sns.lineplot(data=mean_cgi_full,x="bin",y="Low meth",color=seapal[0],ax=ax,label="Low meth cluster")
sns.lineplot(data=mean_cgi_full,x="bin",y="Intermediate meth",color=seapal[8],ax=ax, label="Intermediate meth cluster")
sns.lineplot(data=mean_cgi_full,x="bin",y="High meth",color=seapal[3],ax=ax, label="High meth cluster")
sns.lineplot(data=mean_cgi_full,x="bin",y="Normal",color=seapal[7],ax=ax, label="Normal tissue")
ax.set_title("Signature 1",fontsize=25)
ax.set_ylabel("Methylation (beta)",fontsize=20)
ax.set_xlim([0,50])
ax.set_ylim([0,0.85])
ax.legend_=None
ax.set_xticks([5,15,25,35,45])
ax.set_xticklabels(["N_Shelf","N_Shore","CGI","S_Shore","S_Shelf"],rotation=45,rotation_mode="default",fontsize=20)
ax.set_xlabel('')
fig.savefig("signature_1_cartoon.png",bbox_inches="tight")
fig.savefig("signature_1_cartoon.svg",bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(13,9))
flatax = axs.flatten()

In [ ]:
flatax,mean_cgi_pc = get_methylation_sign_CIMP(flatax,sCIMP2,Meth45k_df,meth_data_pc,consensus_clustering_spectral,
                              high_conf_patients,triple_cluster_cancers_aff)

In [ ]:
# visualization
# lines, labels = flatax[0].get_legend_handles_labels()

for i,cancer in enumerate(sCIMP2):
    a = flatax[i]
    a.set_ylabel('')
    if not (i in [0,4,8]):
        a.set_yticklabels([])
    a.set_xticks([5,15,25,35,45])
    if i in [8,9,10,11]:
        a.set_xticklabels(["N_Shelf","N_Shore","CGI","S_Shore","S_Shelf"],rotation=55,rotation_mode="default",fontsize=20)
    else:
        a.set_xticklabels([])
    a.set_title(cancer,fontsize=20)
    a.set_xlabel('')
    a.legend_=None
fig.text(0.5, -0.02, 'Region', ha='center',fontsize=20)
fig.text(-0.01, 0.5, 'Methylation (beta)', va='center', rotation='vertical',fontsize=20)
# fig.legend(lines, labels, loc = 'center right', bbox_to_anchor = (1.2,-0.1,1,1),
#             bbox_transform = plt.gcf().transFigure,fontsize=15,frameon=False)
fig.tight_layout()
fig


In [ ]:
fig.savefig("signature_sCIMP2_wnormal.svg",bbox_inches="tight")
fig.savefig("signature_sCIMP2_wnormal.png",bbox_inches="tight")

In [ ]:
mean_cgi_full = pd.DataFrame(columns=["bin","Low meth","Intermediate meth","High meth","Normal"])
for cancer in mean_cgi_pc:
    if cancer in triple_cluster_cancers_aff:
        mean_cgi_full= pd.concat([mean_cgi_full,mean_cgi_pc[cancer][["bin","Low meth","Intermediate meth","High meth","Normal"]]])
    else:
        mean_cgi_full= pd.concat([mean_cgi_full,mean_cgi_pc[cancer][["bin","Low meth","High meth","Normal"]]])
mean_cgi_full = mean_cgi_full.groupby("bin").mean()

In [ ]:
seapal=sns.color_palette("bright")
fig,ax=plt.subplots(1,1,figsize=(7,5))
sns.lineplot(data=mean_cgi_full,x="bin",y="Low meth",color=seapal[0],ax=ax,label="Low meth cluster")
sns.lineplot(data=mean_cgi_full,x="bin",y="Intermediate meth",color=seapal[8],ax=ax, label="Intermediate meth cluster")
sns.lineplot(data=mean_cgi_full,x="bin",y="High meth",color=seapal[3],ax=ax, label="High meth cluster")
sns.lineplot(data=mean_cgi_full,x="bin",y="Normal",color=seapal[7],ax=ax, label="Normal tissue")
ax.set_title("Signature 2",fontsize=25)
ax.set_ylabel("Methylation (beta)",fontsize=20)
ax.set_xlim([0,50])
ax.set_ylim([0,0.85])
ax.legend_=None
ax.set_xticks([5,15,25,35,45])
ax.set_xticklabels(["N_Shelf","N_Shore","CGI","S_Shore","S_Shelf"],rotation=45,rotation_mode="default",fontsize=20)
ax.set_xlabel('')
fig.savefig("signature_2_cartoon.png",bbox_inches="tight")
fig.savefig("signature_2_cartoon.svg",bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(13,6))
flatax = axs.flatten()

In [ ]:
flatax,mean_cgi_pc = get_methylation_sign_CIMP(flatax,nonCIMP,Meth45k_df,meth_data_pc,consensus_clustering_spectral,
                              high_conf_patients,triple_cluster_cancers_aff)

In [ ]:
lines, labels = flatax[-2].get_legend_handles_labels()
for i,cancer in enumerate(nonCIMP):
    a = flatax[i]
    a.set_ylabel('')
    if not (i in [0,4]):
        a.set_yticklabels([]) 
    a.set_xticks([5,15,25,35,45])
    if i in [4,5,6]:
        a.set_xticklabels(["N_Shelf","N_Shore","CGI","S_Shore","S_Shelf"],rotation=55,rotation_mode="default",fontsize=20)
    else:
        a.set_xticklabels([])
    a.set_title(cancer,fontsize=20)
    a.set_xlabel('')
    a.legend_=None
    
fig.text(0.5, -0.02, 'Region', ha='center',fontsize=20)
fig.text(-0.01, 0.5, 'Methylation (beta)', va='center', rotation='vertical',fontsize=20)
flatax[-1].axis("off")
fig.legend(lines, labels, loc = 'right', bbox_to_anchor = (1.5,0.1,1,1),
            bbox_transform = plt.gcf().transFigure,fontsize=20,frameon=False)
fig.tight_layout()
fig

In [ ]:
fig.savefig("signature_nonCIMP_wnormal.svg",bbox_inches="tight")
fig.savefig("signature_nonCIMP_wnormal.png",bbox_inches="tight")

In [ ]:
mean_cgi_full = pd.DataFrame(columns=["bin","Low meth","Intermediate meth","High meth","Normal"])
for cancer in mean_cgi_pc:
    if cancer in triple_cluster_cancers_aff:
        mean_cgi_full= pd.concat([mean_cgi_full,mean_cgi_pc[cancer][["bin","Low meth","Intermediate meth","High meth","Normal"]]])
    else:
        mean_cgi_full= pd.concat([mean_cgi_full,mean_cgi_pc[cancer][["bin","Low meth","High meth","Normal"]]])
mean_cgi_full = mean_cgi_full.astype(np.float64).groupby("bin").mean()

In [ ]:
seapal=sns.color_palette("bright")
fig,ax=plt.subplots(1,1,figsize=(7,5))
sns.lineplot(data=mean_cgi_full,x="bin",y="Low meth",color=seapal[0],ax=ax,label="Low meth cluster")
sns.lineplot(data=mean_cgi_full,x="bin",y="Intermediate meth",color=seapal[8],ax=ax, label="Intermediate meth cluster")
sns.lineplot(data=mean_cgi_full,x="bin",y="High meth",color=seapal[3],ax=ax, label="High meth cluster")
sns.lineplot(data=mean_cgi_full,x="bin",y="Normal",color=seapal[7],ax=ax, label="Normal tissue")
ax.set_title("non-CIMP",fontsize=25)
ax.set_ylabel("Methylation (beta)",fontsize=20)
ax.set_xlim([0,50])
ax.set_ylim([0,0.85])
ax.legend_=None
ax.set_xticks([5,15,25,35,45])
ax.set_xticklabels(["N_Shelf","N_Shore","CGI","S_Shore","S_Shelf"],rotation=45,rotation_mode="default",fontsize=20)
ax.set_xlabel('')
fig.savefig("nonCIMP_cartoon.png",bbox_inches="tight")
fig.savefig("nonCIMP_cartoon.svg",bbox_inches="tight")

# Etiology group chart

In [ ]:
import plotly as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import orca

In [ ]:
analyzed_cancers = {0: ["GBM","LAML","LGG","SKCM"], 1: ["ACC","CESC","COAD","GBM","HNSC","KIRC","LGG","LIHC","LUSC","MESO"],
                   2: ["COAD"], 3:["CESC","COAD","HNSC","KIRC","LGG","LIHC","LUAD","LUSC","MESO","SARC"]}
ls = np.linspace(0,1,27)
send = {}
for j in range(len(analyzed_cancers)):
    send[j] = []
    for i,cancer in enumerate(cancer_to_keep):
        if cancer in analyzed_cancers[j]:
            send[j].append([ls[i],ls[i+1]])

In [ ]:
def get_theta(pct, num_points):
    start = pct[0] * 360
    length = (pct[1] - pct[0]) * 360
    step = 360 / num_points
    return np.arange(start, start + length + step, step)

num_points = 360
r_gap = 0.05

max_r = 1 

In [ ]:
color_pal = pl.colors.qualitative.Alphabet
color_mapping = {CIMP_cancers[i]: color_pal[i] for i in range(len(CIMP_cancers))}

In [ ]:
fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'polar'}]*2]*2)
count=0
for i in [1,2]:
    for j in [1,2]:
        fig.add_trace(go.Scatterpolar(
          r=[1] * num_points,
          theta=get_theta([0, 1], num_points),
          mode='lines',
          line_color='black',
          line_width=4,
          showlegend=False
        ),i,j)


        for index, seg in enumerate(send[count]):
            fig.add_trace(go.Scatterpolar(
            r = [1 + r_gap] * num_points,
            theta = get_theta(seg, num_points),
            mode='lines',
            line_width=10,
            name=str(analyzed_cancers[count][index]),
            line_color=color_mapping[analyzed_cancers[count][index]],
          ),i,j)
        count+=1
        
        fig.update_layout(
          polar1=dict(
              angularaxis=dict(
                rotation=90,
                direction="clockwise",
                showticklabels=False,
                showgrid=False
              ),
              radialaxis=dict(
                range=[0, 1 + (len(send) + 1) * r_gap],
                showticklabels=False,
                visible=False
              )
          ),
        polar2=dict(
              angularaxis=dict(
                rotation=90,
                direction="clockwise",
                showticklabels=False,
                showgrid=False
              ),
              radialaxis=dict(
                range=[0, 1 + (len(send) + 1) * r_gap],
                showticklabels=False,
                visible=False
              )
          ),
        polar3=dict(
              angularaxis=dict(
                rotation=90,
                direction="clockwise",
                showticklabels=False,
                showgrid=False
              ),
              radialaxis=dict(
                range=[0, 1 + (len(send) + 1) * r_gap],
                showticklabels=False,
                visible=False
              )
          ),
        polar4=dict(
              angularaxis=dict(
                rotation=90,
                direction="clockwise",
                showticklabels=False,
                showgrid=False
              ),
              radialaxis=dict(
                range=[0, 1 + (len(send) + 1) * r_gap],
                showticklabels=False,
                visible=False
              )
          ))
# # Configure the layout based on the requirements.


fig.add_annotation(x=0.1, y=1.1,
            text="Demethylation (mut)",
            showarrow=False,font=dict(
        family='Arial',
        size=18,
        color="Black"
    ))
fig.add_annotation(x=0.92, y=1.1,
            text="Histone methylation (mut)",
            showarrow=False,font=dict(
        family='Arial',
        size=18,
        color="Black"
    ))
fig.add_annotation(x=0.2, y=0.48,
            text="MSI",
            showarrow=False,font=dict(
        family='Arial',
        size=18,
        color="Black"
    ))
fig.add_annotation(x=0.82, y=0.48,
            text="Diverse",
            showarrow=False,font=dict(
        family='Arial',
        size=18,
        color="Black"
    ))
names = set()
fig.for_each_trace(
    lambda trace:
        trace.update(showlegend=False)
        if (trace.name in names) else names.add(trace.name))
fig.write_image("etiology_indicators.png")
fig.write_image("etiology_indicators.svg")

In [ ]:
fig

# Transcription start sites

We perform the same analysis around transcription start sites to find potential signatures
We do not have information on the distance to the TSS or its position in the illumina annotation so we must take the values from gencode annotations

In [ ]:
tss_dir = "path\to\TSS_distances"

In [ ]:
from gtfparse import read_gtf
gencode_annot_dir = "path\to\gencode\annotation\gtf_file" 
gencode_annot = read_gtf(os.path.join(gencode_annot_dir,"INSERTGENCODEFILE.gtf"))

In [ ]:
gene_start_gencode_annot = gencode_annot[gencode_annot["feature"]=="gene"].set_index("transcript_name")

In [ ]:
gene_start_gencode_annot

In [ ]:
# we get the distance to the TSS for each position
for cancer in tqdm(cancer_to_keep):
    print(cancer)
    # info from the Illumina 450k annotation
    cpg_info_df = Meth45k_df[['CHR', 'MAPINFO', 'Strand',"UCSC_RefGene_Name","UCSC_RefGene_Group"]]
    common_cpg = cpg_info_df.index.intersection(meth_data_pc[cancer].columns)
    cpg_info_df = cpg_info_df.loc[common_cpg]
    df_tss_distance = []
    for cpg in tqdm(cpg_info_df.index):
        info = cpg_info_df.loc[cpg]
        # strand polarity and absolute position of the cpg
        strand = info["Strand"]
        cpg_pos = int(info["MAPINFO"])
        # get the genes related to that probe
        rel_genes = np.unique(dict_cpg_to_gene[cpg])
        rel_genes = gene_start_gencode_annot.index.intersection(rel_genes)
        for gene in rel_genes:
            # get the info related to the gene
            gene_info = gene_start_gencode_annot.loc[gene]
            # start depending on strand polarity
            if strand=="F":
                start = gene_info[["start"]]
            else:
                start = gene_info[["end"]]
            # get the closest beginning in case there are several starts for the gene 
            ind_distance = start.apply(lambda x: np.abs(cpg_pos-x)).squeeze().argmin()
            if strand=="F":
                distance = cpg_pos - int(start.iloc[ind_distance])
            else:
                distance = int(start.iloc[ind_distance]) - cpg_pos
            df = pd.DataFrame(np.array([cpg,distance])).T
            df.columns = ["cpg","distance"]
            # relative distance to TSS (- for upstream, + for downstream)
            if len(df_tss_distance)==0:
                df_tss_distance = df
            else:
                df_tss_distance = pd.concat([df_tss_distance,df])
    df_tss_distance.to_csv(os.path.join(tss_dir,cancer+"_tss_distance.csv"))

In [ ]:
# methylation status is different for TSS
sCIMP1 = ["ACC","COAD","KIRP","LAML","LGG","MESO","SKCM","STAD"]
sCIMP2 = ["CESC","GBM","HNSC","KIRC","LIHC","LUAD","LUSC","PCPG","READ","SARC","THCA"]
nonCIMP = ["BLCA","BRCA","ESCA","PAAD","PRAD","THYM","UCEC"]

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(13,6))
flatax = axs.flatten()

In [ ]:
seapal = sns.color_palette("bright")
fignum = 0
for cancer in tqdm(sCIMP1):
    df_tss_distance = pd.read_csv(os.path.join(tss_dir,cancer+"_tss_distance.csv"))
    df_tss_distance = df_tss_distance.drop("Unnamed: 0",axis=1).set_index("cpg").astype(int)
    cpg_tss = df_tss_distance[df_tss_distance<5000].dropna().astype(int)
    cpg_tss = cpg_tss[cpg_tss>-1500].dropna()
    cpg_tss = cpg_tss[~cpg_tss.index.duplicated()]
    
    common_pat_clustering = consensus_clustering_spectral[cancer].loc[consensus_clustering_spectral[cancer].index.intersection(high_conf_patients[cancer])]
    ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
    ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
    ind_cl_3 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
    cl_1_mean = new_meth_data_pc[cancer].loc[ind_cl_1].mean()
    cl_2_mean = new_meth_data_pc[cancer].loc[ind_cl_2].mean()
    cl_3_mean = new_meth_data_pc[cancer].loc[ind_cl_3].mean()
    normal_mean = normal_meth[cancer].mean()
    if cancer in triple_cluster_cancers_aff:
        full_cpg_island_df= pd.concat([cpg_tss,cl_1_mean,cl_2_mean,cl_3_mean,normal_mean],axis=1,join="inner").dropna()
        full_cpg_island_df.columns = list(cpg_tss.columns) + ["Low meth", "Intermediate meth", "High meth", "Normal"]
    else:
        full_cpg_island_df= pd.concat([cpg_tss,cl_1_mean,cl_2_mean,normal_mean],axis=1,join="inner").dropna()
        full_cpg_island_df.columns = list(cpg_tss.columns) + ["Low meth", "High meth", "Normal"]
    mean_cgi_meth = full_cpg_island_df.groupby(["distance"]).mean().reset_index()
    mean_cgi_meth["distance"] = mean_cgi_meth["distance"].astype("int").apply(lambda x: x//100)
    mean_cgi_meth = mean_cgi_meth.groupby(["distance"]).mean()
    print(cancer)
    
    sns.lineplot(data=mean_cgi_meth,x="distance",y="Low meth",color=seapal[0],ax=flatax[fignum],label="Low meth cluster")
    if cancer in triple_cluster_cancers_aff:
        sns.lineplot(data=mean_cgi_meth,x="distance",y="Intermediate meth",color=seapal[8],ax=flatax[fignum], label="Intermediate meth cluster")
    sns.lineplot(data=mean_cgi_meth,x="distance",y="High meth",color=seapal[3],ax=flatax[fignum], label="High meth cluster")
    sns.lineplot(data=mean_cgi_meth,x="distance",y="Normal",color=seapal[7],ax=flatax[fignum], label="Normal tissue")
    flatax[fignum].set_title(cancer+" average methylation over CGI",fontsize=18)
    flatax[fignum].set_ylabel("Methylation (beta)",fontsize=15)
    flatax[fignum].set_xlim([0,50])
    flatax[fignum].set_ylim([0,0.85])
    fignum+=1

In [ ]:
# visualization
# lines, labels = flatax[0].get_legend_handles_labels()
for i,cancer in enumerate(sCIMP1):
    a = flatax[i]
    a.set_ylabel('') 
    a.set_xticks([-15,-10,0,10,20,30,40,50])
    if i in [4,5,6,7]:
        a.set_xticklabels(["","-1","TSS","1","2","3","4","5"],rotation=45,rotation_mode="default")
    else:
        a.set_xticklabels([])
    
    if not(i in [0,4]):
        a.set_yticklabels([])
       
    a.set_title(cancer,fontsize=15)
    a.set_xlabel('')
    a.legend_=None
fig.text(0.5, -0.02, 'Distance (kb)', ha='center')
fig.text(-0.01, 0.5, 'Methylation (beta)', va='center', rotation='vertical')
# flatax[-1].axis("off")
# fig.legend(lines, labels, loc = 'center right', bbox_to_anchor = (1.2,0,1,1),
#             bbox_transform = plt.gcf().transFigure,fontsize=15,frameon=False)
fig.tight_layout()
fig

In [ ]:
fig.savefig("TSS_signature_sCIMP1.png",bbox_inches='tight')
fig.savefig("TSS_signature_sCIMP1.svg",bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(13,9))
flatax = axs.flatten()

In [ ]:
seapal = sns.color_palette("bright")
fignum = 0
for cancer in tqdm(sCIMP2):
    df_tss_distance = pd.read_csv(os.path.join(tss_dir,cancer+"_tss_distance.csv"))
    df_tss_distance = df_tss_distance.drop("Unnamed: 0",axis=1).set_index("cpg").astype(int)
    cpg_tss = df_tss_distance[df_tss_distance<5000].dropna().astype(int)
    cpg_tss = cpg_tss[cpg_tss>-1500].dropna()
    cpg_tss = cpg_tss[~cpg_tss.index.duplicated()]
    
    common_pat_clustering = consensus_clustering_spectral[cancer].loc[consensus_clustering_spectral[cancer].index.intersection(high_conf_patients[cancer])]
    ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
    ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
    ind_cl_3 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
    cl_1_mean = new_meth_data_pc[cancer].loc[ind_cl_1].mean()
    cl_2_mean = new_meth_data_pc[cancer].loc[ind_cl_2].mean()
    cl_3_mean = new_meth_data_pc[cancer].loc[ind_cl_3].mean()
    normal_mean = normal_meth[cancer].mean()
    if cancer in triple_cluster_cancers_aff:
        full_cpg_island_df= pd.concat([cpg_tss,cl_1_mean,cl_2_mean,cl_3_mean,normal_mean],axis=1,join="inner").dropna()
        full_cpg_island_df.columns = list(cpg_tss.columns) + ["Low meth", "Intermediate meth", "High meth", "Normal"]
    else:
        full_cpg_island_df= pd.concat([cpg_tss,cl_1_mean,cl_2_mean,normal_mean],axis=1,join="inner").dropna()
        full_cpg_island_df.columns = list(cpg_tss.columns) + ["Low meth", "High meth", "Normal"]
    mean_cgi_meth = full_cpg_island_df.groupby(["distance"]).mean().reset_index()
    mean_cgi_meth["distance"] = mean_cgi_meth["distance"].astype("int").apply(lambda x: x//100)
    mean_cgi_meth = mean_cgi_meth.groupby(["distance"]).mean()
    print(cancer)
    
    sns.lineplot(data=mean_cgi_meth,x="distance",y="Low meth",color=seapal[0],ax=flatax[fignum],label="Low meth cluster")
    if cancer in triple_cluster_cancers_aff:
        sns.lineplot(data=mean_cgi_meth,x="distance",y="Intermediate meth",color=seapal[8],ax=flatax[fignum], label="Intermediate meth cluster")
    sns.lineplot(data=mean_cgi_meth,x="distance",y="High meth",color=seapal[3],ax=flatax[fignum], label="High meth cluster")
    sns.lineplot(data=mean_cgi_meth,x="distance",y="Normal",color=seapal[7],ax=flatax[fignum], label="Normal tissue")
    flatax[fignum].set_title(cancer+" average methylation over CGI",fontsize=18)
    flatax[fignum].set_ylabel("Methylation (beta)",fontsize=15)
    flatax[fignum].set_xlim([0,50])
    flatax[fignum].set_ylim([0,0.85])
    fignum+=1

In [ ]:
# visualization
lines, labels = flatax[0].get_legend_handles_labels()
for i,cancer in enumerate(sCIMP2):
    a = flatax[i]
    a.set_ylabel('') 
    a.set_xticks([-15,-10,0,10,20,30,40,50])
    if i in [7,8,9,10]:
        a.set_xticklabels(["","-1","TSS","1","2","3","4","5"],rotation=45,rotation_mode="default")
    else:
        a.set_xticklabels([])
    
    if not(i in [0,4,8]):
        a.set_yticklabels([])
       
    a.set_title(cancer,fontsize=15)
    a.set_xlabel('')
    a.legend_=None
fig.text(0.5, -0.02, 'Distance (kb)', ha='center')
fig.text(-0.01, 0.5, 'Methylation (beta)', va='center', rotation='vertical')
flatax[-1].axis("off")
fig.legend(lines, labels, loc = 'center right', bbox_to_anchor = (1.3,0,1,1),
            bbox_transform = plt.gcf().transFigure,fontsize=15,frameon=False)
fig.tight_layout()
fig

In [ ]:
fig.savefig("TSS_signature_sCIMP2.png",bbox_inches='tight')
fig.savefig("TSS_signature_sCIMP2.svg",bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(6,13))

In [ ]:
flatax = axs.flatten()
seapal = sns.color_palette("bright")
fignum = 0
for cancer in tqdm(nonCIMP):
    df_tss_distance = pd.read_csv(os.path.join(tss_dir,cancer+"_tss_distance.csv"))
    df_tss_distance = df_tss_distance.drop("Unnamed: 0",axis=1).set_index("cpg").astype(int)
    cpg_tss = df_tss_distance[df_tss_distance<5000].dropna().astype(int)
    cpg_tss = cpg_tss[cpg_tss>-1500].dropna()
    cpg_tss = cpg_tss[~cpg_tss.index.duplicated()]
    
    common_pat_clustering = consensus_clustering_spectral[cancer].loc[consensus_clustering_spectral[cancer].index.intersection(high_conf_patients[cancer])]
    ind_cl_1 = common_pat_clustering[common_pat_clustering["cluster"]==1].index
    ind_cl_2 = common_pat_clustering[common_pat_clustering["cluster"]==2].index
    ind_cl_3 = common_pat_clustering[common_pat_clustering["cluster"]==3].index
    cl_1_mean = new_meth_data_pc[cancer].loc[ind_cl_1].mean()
    cl_2_mean = new_meth_data_pc[cancer].loc[ind_cl_2].mean()
    cl_3_mean = new_meth_data_pc[cancer].loc[ind_cl_3].mean()
    normal_mean = normal_meth[cancer].mean()
    if cancer in triple_cluster_cancers_aff:
        full_cpg_island_df= pd.concat([cpg_tss,cl_1_mean,cl_2_mean,cl_3_mean,normal_mean],axis=1,join="inner").dropna()
        full_cpg_island_df.columns = list(cpg_tss.columns) + ["Low meth", "Intermediate meth", "High meth", "Normal"]
    else:
        full_cpg_island_df= pd.concat([cpg_tss,cl_1_mean,cl_2_mean,normal_mean],axis=1,join="inner").dropna()
        full_cpg_island_df.columns = list(cpg_tss.columns) + ["Low meth", "High meth", "Normal"]
    mean_cgi_meth = full_cpg_island_df.groupby(["distance"]).mean().reset_index()
    mean_cgi_meth["distance"] = mean_cgi_meth["distance"].astype("int").apply(lambda x: x//100)
    mean_cgi_meth = mean_cgi_meth.groupby(["distance"]).mean()
    print(cancer)
    sns.lineplot(data=mean_cgi_meth,x="distance",y="Low meth",color=seapal[0],ax=flatax[fignum],label="Low meth cluster")
    if cancer in triple_cluster_cancers_aff:
        sns.lineplot(data=mean_cgi_meth,x="distance",y="Intermediate meth",color=seapal[8],ax=flatax[fignum], label="Intermediate meth cluster")
    sns.lineplot(data=mean_cgi_meth,x="distance",y="High meth",color=seapal[3],ax=flatax[fignum], label="High meth cluster")
    sns.lineplot(data=mean_cgi_meth,x="distance",y="Normal",color=seapal[7],ax=flatax[fignum], label="Normal meth cluster")
    flatax[fignum].set_title(cancer+" average methylation over CGI",fontsize=15)
    flatax[fignum].set_ylabel("Methylation (beta)",fontsize=15)
    flatax[fignum].set_xlim([0,50])
    flatax[fignum].set_ylim([0,0.85])
    fignum+=1

In [ ]:
# visualization
lines, labels = flatax[-2].get_legend_handles_labels()
for i,cancer in enumerate(nonCIMP):
    a = flatax[i]
    a.set_ylabel('') 
    a.set_xticks([-15,-10,0,10,20,30,40,50])
    if i in [5,6]:
        a.set_xticklabels(["","-1","TSS","1","2","3","4","5"],rotation=45,rotation_mode="default")
    else:
        a.set_xticklabels([])
    
    if not(i in [0,2,4,6]):
        a.set_yticklabels([])
       
    a.set_title(cancer,fontsize=15)
    a.set_xlabel('')
    a.legend_=None
fig.text(0.5, -0.02, 'Distance (kb)', ha='center')
fig.text(-0.01, 0.5, 'Methylation (beta)', va='center', rotation='vertical')
flatax[-1].axis("off")
fig.legend(lines, labels, loc = 'center right', bbox_to_anchor = (0.2,0.05,1,1),
            bbox_transform = plt.gcf().transFigure,fontsize=15,frameon=False)
fig.tight_layout()
fig

In [ ]:
fig.savefig("TSS_signature_nonCIMP.png",bbox_inches='tight')
fig.savefig("TSS_signature_nonCIMP.svg",bbox_inches='tight')

## Clustering method comparison

In [ ]:
from scipy.spatial.distance import cdist, hamming
import networkx as nx
from community import community_louvain
import snf
from snf import metrics

In [ ]:
def get_consensus_matrix_community(meth_data_pc,graphs_snf,cancer_to_keep,res_list,n_runs):
    #     Parameters: meth_data_pc (dict): keys are cancer types, values are pd.DataFrame (n_pat,n_cpg) (rows=patients,columns=cpg)
    #             cancer_to_keep (list): names of cancer types
    #             graphs_snf (dict): keys are cancer types, values are networkx graphs 
    #             res_list (list): list of resolution to use in the Louvain community detection algorithm for each cancer type (type float)
    #             n_runs (int): number of runs for consensus clustering 
    #     Returns: consensus_matrix (pd.DataFrame): df with the cluster membership for each run (n_patients, n_runs)
    consensus_matrix = {cancer: [] for cancer in cancer_to_keep}
    for r in tqdm(range(n_runs)):
        clusters_louvain_snf = {}
        for i,cancer in enumerate(cancer_to_keep):
            print(cancer)
            clusters_louvain_snf[cancer]=compute_clusters_community(graphs_snf[cancer], res_list[i])
        df_louvain_snf = {}
        for cancer in cancer_to_keep:
            df_louvain_snf[cancer] = pd.DataFrame(np.zeros((len(list(meth_data_pc[cancer].index)),1)),index=list(meth_data_pc[cancer].index),columns=["cluster"])
            for cluster in clusters_louvain_snf[cancer]:
                for pat in clusters_louvain_snf[cancer][cluster]:
                    df_louvain_snf[cancer].loc[pat]=cluster
            df_louvain_snf[cancer]=df_louvain_snf[cancer].reset_index(drop=True)
            if len(consensus_matrix[cancer])==0:
                consensus_matrix[cancer] = df_louvain_snf[cancer]
            else:
                consensus_matrix[cancer] = pd.concat([consensus_matrix[cancer],df_louvain_snf[cancer]],axis=1)
    return consensus_matrix

In [ ]:
def graph_of_patients(patient_ids,sim_matrix):
#     Constructs the graph of UDN patients using the similarity matrix computed: nodes are patients, 
#                 edges between patient i and j is proportional to the similarity between these two patients
#     Parameters: patients (list): list of UDN IDs of patients to consider
#                 sim_matrix (np.ndarray): array, similarity matrix of pairwise similarity between each patient
#     Returns : G (networkx.Graph): networkx graph of UDN patients 
#               pos (np.ndarray): positions of nodes 
    G= nx.Graph()
    elist=[]
    for i in range(sim_matrix.shape[0]):
        G.add_node(patient_ids[i])
        for j in range(i,sim_matrix.shape[1]):
            elist.append((patient_ids[i],patient_ids[j],sim_matrix[i,j]))
    G.add_weighted_edges_from(elist)
    return G

In [ ]:
def compute_clusters_community(graph,resolution):
#     Compute the clusters in a graph using Louvain's community detection method
#     Parameters : graph (networkx.Graph): graph of UDN patients computed using the pairwise 
#                                 similarity between patients
#                 resolution (float): resolution for the Louvain method
#     Returns: clusters (dict): dictionary with the cluster number as key and a list containing all 
#                                 the patients in the cluster as value
    
    partition = community_louvain.best_partition(graph,resolution=resolution)
    clusters={}
    for node in partition.keys():
        if not(partition[node] in clusters.keys()):
            clusters[partition[node]]=[node]
        else:
            clusters[partition[node]].append(node)
    count=0
    for cluster in clusters.keys():
        print("Length of cluster {} : {}".format(cluster,len(clusters[cluster])))
        if len(clusters[cluster])<=1:
            count+=1
    print("Number of clusters with less than 3 patients (outliers) : {}".format(count))
    return clusters

In [ ]:
# get the affinity matrix with manhattan distance
aff_mat = {}
for cancer in tqdm(cancer_to_keep):
    aff_mat[cancer] = snf.make_affinity(new_meth_data_pc[cancer].values,metric="cityblock",K=50)

graphs_snf = {}
for cancer in cancer_to_keep:
    graphs_snf[cancer] = graph_of_patients(list(new_meth_data_pc[cancer].index),aff_mat[cancer])

In [ ]:
# if need to test what resolution for what cancer type
clusters_louvain_snf_test=compute_clusters_community(graphs_snf["CESC"], 1.15)

In [ ]:
res_list = [1.27, 1.25, 1.15, 1.15, 1.15, 1.2, 1.25, 1.15, 1.12, 1.3, 1.3, 1.3, 1.1, 1.1, 1.1, 1.35, 1.25, 1.16, 1.3, 1.25, 1.3, 1.1, 1.35, 1.1, 1.25, 1.1]

consensus_matrix_meth = get_consensus_matrix_community(new_meth_data_pc,graphs_snf,cancer_to_keep,res_list,10)

In [ ]:
from netneurotools import cluster
louvain_clustering_pc = {}
for cancer in cancer_to_keep:
    louvain_clustering_pc[cancer] = pd.DataFrame(cluster.find_consensus(consensus_matrix_meth[cancer].values, seed=1234),columns=["cluster"])
louvain_clustering_pc["STAD"] = louvain_clustering_pc["STAD"].replace({3: 2})
louvain_clustering_pc["PCPG"] = louvain_clustering_pc["PCPG"].replace({4: 3})

In [ ]:
for cancer in cancer_to_keep:
    print(cancer,Counter(louvain_clustering_pc[cancer]["cluster"].values.ravel()))

Get the membership for all clustering methods

In [ ]:
# 5D UMAP for GMM
meth_data_red = {}
for cancer in tqdm(cancer_to_keep):
    reducer = umap.UMAP(n_components=5)
    meth_data_red[cancer] = reducer.fit_transform(df_transf_data[cancer])

In [ ]:
agglomerative_clustering_pc = {}
for cancer in tqdm(cancer_to_keep):
    if cancer in triple_cluster_cancers_aff:
        agglomerative = AgglomerativeClustering(n_clusters=3)
    else:
        agglomerative = AgglomerativeClustering(n_clusters=2)
    agglomerative.fit(new_meth_data_pc[cancer])
    agglomerative_clustering_pc[cancer] = pd.DataFrame(agglomerative.labels_,columns=["cluster"])
    agglomerative_clustering_pc[cancer] = agglomerative_clustering_pc[cancer]+1

In [ ]:
kmeans_clustering_pc = {}
for cancer in tqdm(cancer_to_keep):
    if cancer in triple_cluster_cancers_aff:
        kmeans = KMeans(n_clusters=3)
    else:
        kmeans = KMeans(n_clusters=2)
    kmeans.fit(new_meth_data_pc[cancer])
    kmeans_clustering_pc[cancer] = pd.DataFrame(kmeans.labels_,columns=["cluster"])
    kmeans_clustering_pc[cancer] = kmeans_clustering_pc[cancer]+1

In [ ]:
gmm_clustering_pc = []
for cancer in tqdm(cancer_to_keep):
    if cancer in triple_cluster_cancers_aff:
        bgm = GaussianMixture(n_components=3, random_state=42, covariance_type="diag")
    else:
        bgm = GaussianMixture(n_components=2, random_state=42, covariance_type="diag")
    labels = bgm.fit_predict(meth_data_red[cancer])
    prob_labels = bgm.predict_proba(meth_data_red[cancer])
    labels_conf = [prob_labels[i][labels[i]]*100 for i in range(len(labels))]
    labels = labels+1
    if len(gmm_clustering_pc)==0:
        gmm_clustering_pc = pd.concat([pd.DataFrame(umap_red_data[cancer],index=new_meth_data_pc[cancer].index,columns=["Comp1","Comp2"]),
           pd.DataFrame(labels,index=new_meth_data_pc[cancer].index,columns=["Cluster"]),
            pd.DataFrame(labels_conf,index=new_meth_data_pc[cancer].index,columns=["Confidence"]),
            pd.DataFrame([cancer]*new_meth_data_pc[cancer].shape[0],index=new_meth_data_pc[cancer].index,columns=["Cancer"])],axis=1)
    else:
        df = pd.concat([pd.DataFrame(umap_red_data[cancer],index=new_meth_data_pc[cancer].index,columns=["Comp1","Comp2"]),
           pd.DataFrame(labels,index=new_meth_data_pc[cancer].index,columns=["Cluster"]),
            pd.DataFrame(labels_conf,index=new_meth_data_pc[cancer].index,columns=["Confidence"]),
            pd.DataFrame([cancer]*new_meth_data_pc[cancer].shape[0],index=new_meth_data_pc[cancer].index,columns=["Cancer"])],axis=1)
        gmm_clustering_pc = pd.concat([gmm_clustering_pc,df])

In [ ]:
gmm_clustering_pc = {cancer: pd.DataFrame(gmm_clustering_pc[gmm_clustering_pc["Cancer"]==cancer]["Cluster"].values.ravel(),index=new_meth_data_pc[cancer].index,columns=["cluster"]) for cancer in cancer_to_keep}

In [ ]:
sign_cpg_dir = "path\to\gisign_cpg_spectral"
def get_sign_kw_louvain(meth_data_pc, cancer_to_keep, consensus_clustering):
#     Parameters: meth_data_pc (dict): keys are cancer types, values are pd.DataFrame (n_pat,n_cpg) (rows=patients,columns=cpg)
#             cancer_to_keep (list): names of cancer types
#             consensus_clustering (dict): keys are cancer types, values are pd.DataFrame (n_pat,1) (rows=patients, column=cluster membership)
#     Returns: kw (dict): keys are cancer types, values are list of sign. cpg probes for 
    kw = {}
    for cancer in cancer_to_keep:
        kw[cancer]=[]
        positions = meth_data_pc[cancer].columns
        ind_cl_1 = np.where(consensus_clustering[cancer]["cluster"].values==1)[0]
        ind_cl_2 = np.where(consensus_clustering[cancer]["cluster"].values==2)[0]
        ind_cl_3 = np.where(consensus_clustering[cancer]["cluster"].values==3)[0]
        print(cancer)
        if len(np.unique(consensus_clustering[cancer]["cluster"].values))==2:
            meth_cl_1 = meth_data_pc[cancer].iloc[ind_cl_1].copy()
            meth_cl_2 = meth_data_pc[cancer].iloc[ind_cl_2].copy()
            for pos in tqdm(positions):
                kw[cancer].append(kruskal(meth_cl_1[pos],
                        meth_cl_2[pos])[1])
        elif len(np.unique(consensus_clustering[cancer]["cluster"].values))==3:
            meth_cl_1 = meth_data_pc[cancer].iloc[ind_cl_1].copy()
            meth_cl_2 = meth_data_pc[cancer].iloc[ind_cl_2].copy()
            meth_cl_3 = meth_data_pc[cancer].iloc[ind_cl_3].copy()
            for pos in tqdm(positions):
                kw[cancer].append(kruskal(meth_cl_1[pos],
                        meth_cl_2[pos],
                        meth_cl_3[pos])[1])
        else:
            print("The cancer {} is not taken into consideration (too many clusters)".format(cancer))
            continue
        ind_sign = np.where(np.array(kw[cancer])<(0.05/len(kw[cancer])))[0]
        print("There are {} significantly different positions between clusters for cancer type {}".format(len(ind_sign),cancer))
        sign_pos=[meth_data_pc[cancer].columns[i] for i in ind_sign]
    return kw

In [ ]:
kw_agg = get_sign_kw_louvain(new_meth_data_pc, cancer_to_keep, agglomerative_clustering_pc)

In [ ]:
kw_kmeans = get_sign_kw_louvain(new_meth_data_pc, cancer_to_keep, kmeans_clustering_pc)

In [ ]:
kw_louvain = get_sign_kw_louvain(new_meth_data_pc, cancer_to_keep, louvain_clustering_pc)

In [ ]:
kw_gmm = get_sign_kw_louvain(new_meth_data_pc, cancer_to_keep, gmm_clustering_pc)

In [ ]:
sign_pos_agglomerative = {}
for cancer in tqdm(cancer_to_keep):
    ind_sign = np.where(np.array(kw_agg[cancer])<(0.05/len(kw_agg[cancer])))[0]
    sign_pos_agglomerative[cancer] = pd.DataFrame([new_meth_data_pc[cancer].columns[i] for i in ind_sign])

In [ ]:
sign_pos_kmeans = {}
for cancer in tqdm(cancer_to_keep):
    ind_sign = np.where(np.array(kw_kmeans[cancer])<(0.05/len(kw_kmeans[cancer])))[0]
    sign_pos_kmeans[cancer] = pd.DataFrame([new_meth_data_pc[cancer].columns[i] for i in ind_sign])

In [ ]:
sign_pos_gmm = {}
for cancer in tqdm(cancer_to_keep):
    ind_sign = np.where(np.array(kw_gmm[cancer])<(0.05/len(kw_gmm[cancer])))[0]
    sign_pos_gmm[cancer] = pd.DataFrame([new_meth_data_pc[cancer].columns[i] for i in ind_sign])

In [ ]:
sign_pos_louvain = {}
for cancer in tqdm(cancer_to_keep):
    ind_sign = np.where(np.array(kw_louvain[cancer])<(0.05/len(kw_louvain[cancer])))[0]
    sign_pos_louvain[cancer] = pd.DataFrame([new_meth_data_pc[cancer].columns[i] for i in ind_sign])

In [ ]:
avg_meth_snf_agglomerative = get_avg_methylation_snf(sign_pos_agglomerative, new_meth_data_pc, agglomerative_clustering_pc, cancer_to_keep)
avg_meth_snf_agglomerative.columns=[1,2,3]

avg_meth_snf_kmeans = get_avg_methylation_snf(sign_pos_kmeans, new_meth_data_pc, kmeans_clustering_pc, cancer_to_keep)
avg_meth_snf_kmeans.columns=[1,2,3]

avg_meth_snf_gmm = get_avg_methylation_snf(sign_pos_gmm, new_meth_data_pc, gmm_clustering_pc, cancer_to_keep)
avg_meth_snf_gmm.columns=[1,2,3]

avg_meth_snf_louvain = get_avg_methylation_snf(sign_pos_louvain, new_meth_data_pc, louvain_clustering_pc, cancer_to_keep)
avg_meth_snf_louvain.columns=[1,2,3]
avg_meth_snf_louvain

In [ ]:
# make sure the cluster number corresponds to the average beta value (1=lowest, 3=highest)
for cancer in avg_meth_snf_agglomerative.index:
    sorted_arg = np.argsort(avg_meth_snf_agglomerative.loc[cancer])
    cl_order = {}
    for i in range(avg_meth_snf_agglomerative.loc[cancer].dropna().shape[0]):
        cl_order[sorted_arg[i+1]+1] = i+1
    agglomerative_clustering_pc[cancer] = agglomerative_clustering_pc[cancer].replace({"cluster": cl_order})
    agglomerative_clustering_pc[cancer].index = new_meth_data_pc[cancer].index

In [ ]:
for cancer in avg_meth_snf_kmeans.index:
    sorted_arg = np.argsort(avg_meth_snf_kmeans.loc[cancer])
    cl_order = {}
    for i in range(avg_meth_snf_kmeans.loc[cancer].dropna().shape[0]):
        cl_order[sorted_arg[i+1]+1] = i+1
    kmeans_clustering_pc[cancer] = kmeans_clustering_pc[cancer].replace({"cluster": cl_order})
    kmeans_clustering_pc[cancer].index = new_meth_data_pc[cancer].index

In [ ]:
for cancer in avg_meth_snf_gmm.index:
    sorted_arg = np.argsort(avg_meth_snf_gmm.loc[cancer])
    cl_order = {}
    for i in range(avg_meth_snf_gmm.loc[cancer].dropna().shape[0]):
        cl_order[sorted_arg[i+1]+1] = i+1
    gmm_clustering_pc[cancer] = gmm_clustering_pc[cancer].replace({"cluster": cl_order})
    gmm_clustering_pc[cancer].index = new_meth_data_pc[cancer].index

In [ ]:
for cancer in avg_meth_snf_louvain.index:
    sorted_arg = np.argsort(avg_meth_snf_louvain.loc[cancer])
    cl_order = {}
    for i in range(avg_meth_snf_louvain.loc[cancer].dropna().shape[0]):
        cl_order[sorted_arg[i+1]+1] = i+1
    louvain_clustering_pc[cancer] = louvain_clustering_pc[cancer].replace({"cluster": cl_order})
    louvain_clustering_pc[cancer].index = new_meth_data_pc[cancer].index

In [ ]:
# create a df with umap reduced data and cluster membership
df_umap_agglomerative,umap_red_data_pd_agg = {},{}
for cancer in cancer_to_keep:
    umap_red_data_pd_agg[cancer] = pd.DataFrame(umap_red_data[cancer],index=agglomerative_clustering_pc[cancer].index,columns=["Comp1","Comp2"])
    if cancer in triple_cluster_cancers_aff:
        df_umap_agglomerative[cancer] = pd.concat([umap_red_data_pd_agg[cancer],agglomerative_clustering_pc[cancer]],axis=1).replace({1: "Low cluster", 2: "Intermediate cluster", 3: "High cluster"})
    else:
        df_umap_agglomerative[cancer] = pd.concat([umap_red_data_pd_agg[cancer],agglomerative_clustering_pc[cancer]],axis=1).replace({1: "Low cluster", 2: "High cluster"})
    df_umap_agglomerative[cancer].columns = ["Comp1","Comp2","Cluster"]

In [ ]:
# compute the associated silhouette score in the 2D UMAP reduced space
sil_scores_agg = []
sil_scores_means_agg = []
for cancer in tqdm(cancer_to_keep):
    sil_pd = pd.concat([new_meth_data_pc[cancer],agglomerative_clustering_pc[cancer]],axis=1).dropna()
    sil = silhouette_samples(sil_pd.drop("cluster",axis=1),sil_pd["cluster"].values.ravel(),metric="euclidean")
    mn = np.mean(sil)
    print(cancer,mn)
    sil_scores_agg.append(sil)
    sil_scores_means_agg.append(mn)
print("Mean {} IQR [{}-{}]".format(np.mean(sil_scores_means_agg),np.quantile(sil_scores_means_agg,0.25),np.quantile(sil_scores_means_agg,0.75)))

In [ ]:
# compute the associated silhouette score in the 2D UMAP reduced space
sil_scores_gmm = []
sil_scores_means_gmm = []
for cancer in tqdm(cancer_to_keep):
    sil_pd = pd.concat([new_meth_data_pc[cancer],gmm_clustering_pc[cancer]],axis=1).dropna()
    sil = silhouette_samples(sil_pd.drop("cluster",axis=1),sil_pd["cluster"].values.ravel(),metric="euclidean")
    mn = np.mean(sil)
    print(cancer,mn)
    sil_scores_gmm.append(sil)
    sil_scores_means_gmm.append(mn)
print("Mean {} IQR [{}-{}]".format(np.mean(sil_scores_means_gmm),np.quantile(sil_scores_means_gmm,0.25),np.quantile(sil_scores_means_gmm,0.75)))

In [ ]:
# compute the associated silhouette score in the 2D UMAP reduced space
sil_scores_kmeans = []
sil_scores_means_kmeans = []
for cancer in tqdm(cancer_to_keep):
    sil_pd = pd.concat([new_meth_data_pc[cancer],kmeans_clustering_pc[cancer]],axis=1).dropna()
    sil = silhouette_samples(sil_pd.drop("cluster",axis=1),sil_pd["cluster"].values.ravel(),metric="euclidean")
    mn = np.mean(sil)
    print(cancer,mn)
    sil_scores_kmeans.append(sil)
    sil_scores_means_kmeans.append(mn)
print("Mean {} IQR [{}-{}]".format(np.mean(sil_scores_means_kmeans),np.quantile(sil_scores_means_kmeans,0.25),np.quantile(sil_scores_means_kmeans,0.75)))

In [ ]:
# compute the associated silhouette score in the 2D UMAP reduced space
sil_scores_louvain = []
sil_scores_means_louvain = []
for cancer in tqdm(cancer_to_keep):
    sil_pd = pd.concat([new_meth_data_pc[cancer],louvain_clustering_pc[cancer]],axis=1).dropna()
    sil = silhouette_samples(sil_pd.drop("cluster",axis=1),sil_pd["cluster"].values.ravel(),metric="euclidean")
    mn = np.mean(sil)
    print(cancer,mn)
    sil_scores_louvain.append(sil)
    sil_scores_means_louvain.append(mn)
print("Mean {} IQR [{}-{}]".format(np.mean(sil_scores_means_louvain),np.quantile(sil_scores_means_louvain,0.25),np.quantile(sil_scores_means_louvain,0.75)))

In [ ]:
# compute the associated silhouette score in the 2D UMAP reduced space
sil_scores = []
sil_scores_means= []
for cancer in tqdm(cancer_to_keep):
    sil_pd = pd.concat([new_meth_data_pc[cancer],consensus_clustering_spectral[cancer]],axis=1).dropna()
    sil = silhouette_samples(sil_pd.drop("cluster",axis=1),sil_pd["cluster"].values.ravel(),metric="euclidean")
    mn = np.mean(sil)
    print(cancer,mn)
    sil_scores.append(sil)
    sil_scores_means.append(mn)
print("Mean {} IQR [{}-{}]".format(np.mean(sil_scores_means),np.quantile(sil_scores_means,0.25),np.quantile(sil_scores_means,0.75)))

In [ ]:
pd.DataFrame(np.array([sil_scores_means,sil_scores_means_agg,sil_scores_means_kmeans,sil_scores_means_gmm,sil_scores_means_louvain]).T,columns=["spectral","agg","kmeans","gmm","louvain"],index=cancer_to_keep).round(2)